# Analisi esplorativa dei dati (EDA)

Questo notebook esegue un'analisi esplorativa rapida sui file dati presenti nella cartella `data/raw/surveys/`.

- Rileva automaticamente file CSV/TSV/XLS/XLSX/JSON
- Carica il dataset selezionato
- Mostra struttura, valori mancanti e statistiche
- Crea alcuni grafici di base e salva output in `analysis/exports/latest/`

## Cella 2: Installazione dipendenze

Questa cella verifica e installa automaticamente le librerie Python necessarie per l'analisi (pandas, numpy, matplotlib, seaborn, openpyxl). Se una libreria manca, viene installata automaticamente tramite pip.

In [ ]:
# Se necessario, installa dipendenze (esegui una volta)
import sys, subprocess
def ensure(pkg):
    try:
        __import__(pkg)
        return True
    except Exception:
        print(f'Installing {pkg} ...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
        return True

for p in ['pandas', 'numpy', 'matplotlib', 'seaborn', 'openpyxl', 'sciencePlots']:
    ensure(p)
print('Dipendenze pronte.')

## Cella 3: Import e configurazione librerie

Importa le librerie principali (pandas, numpy, matplotlib, seaborn) e configura le opzioni di visualizzazione per mostrare più righe e colonne. Imposta anche il tema grafico 'whitegrid' di seaborn e stampa le versioni delle librerie utilizzate.

In [ ]:
# Import e configurazione
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
sns.set_theme(style='whitegrid')
print('Versioni:', {
    'pandas': pd.__version__,
    'numpy': np.__version__
})

## Cella 4: Definizione percorsi

Definisce i percorsi delle cartelle principali del progetto: la cartella dati di input (`data/raw/surveys/`) e la cartella di output per i risultati esplorativi (`analysis/exports/latest/`). Crea automaticamente la cartella di output se non esiste e verifica l'esistenza della cartella dati.

In [ ]:
# Percorsi: individua la root del repository e carica i path da config/paths.json
import json
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / '.git').exists():
            return candidate
    raise RuntimeError('Impossibile trovare la root del repository')

REPO_ROOT = find_repo_root(Path.cwd())
with (REPO_ROOT / 'config' / 'paths.json').open() as fp:
    PATHS = json.load(fp)

DATA_DIR = (REPO_ROOT / PATHS['data']['raw_surveys_dir']).resolve()
INTERIM_DIR = (REPO_ROOT / PATHS['data']['interim_dir']).resolve()
OUTPUT_DIR = (REPO_ROOT / PATHS['analysis']['exports_latest_dir']).resolve()

INTERIM_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('DATA_DIR:', DATA_DIR)
print('INTERIM_DIR:', INTERIM_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)
assert DATA_DIR.exists(), f'Cartella dati non trovata: {DATA_DIR}'


## Cella 5: Lista file disponibili

Cerca e elenca tutti i file di dati disponibili nella cartella `data/raw/surveys/`. Supporta formati CSV, TSV, Excel (xlsx, xls) e JSON. Mostra un elenco numerato di tutti i file trovati.

In [ ]:
# Elenco file disponibili
from itertools import chain
patterns = ['*.csv', '*.tsv', '*.xlsx', '*.xls', '*.json']
files = list(chain.from_iterable(DATA_DIR.glob(p) for p in patterns))
files = sorted(files)
for i, f in enumerate(files, 1):
    print(f'{i:2d}.', f.name)
if not files:
    print('Nessun file trovato in', DATA_DIR)

## Cella 6: Identificazione file insegnanti e studenti

Identifica automaticamente i due file Excel principali (insegnanti e studenti) usando pattern di riconoscimento nei nomi dei file (es. "insegn", "student", ecc.). Se i pattern non corrispondono, usa i primi due file Excel trovati in ordine alfabetico.

In [ ]:
# Seleziona i due file Excel (insegnanti e studenti)
from pathlib import Path
import re
from itertools import chain

# Rileva i file Excel direttamente dalla cartella dati (evita dipendenze dall'ordine di esecuzione di altre celle)
excel_files = sorted(chain.from_iterable(DATA_DIR.glob(p) for p in ('*.xlsx', '*.xls')))
if not excel_files:
    raise FileNotFoundError(f'Nessun file Excel trovato in {DATA_DIR}')

# Heuristics per riconoscere i file
insegn_patterns = [r'insegn', r'docent', r'teacher']
studenti_patterns = [r'student', r'alunn', r'alliev']

def match_any(path: Path, patterns):
    name = path.name.lower()
    return any(re.search(p, name) for p in patterns)

file_insegnanti = next((p for p in excel_files if match_any(p, insegn_patterns)), None)
file_studenti = next((p for p in excel_files if match_any(p, studenti_patterns)), None)

# Fallback: se non riconosciamo i nomi, prendiamo i primi due file
if file_insegnanti is None or file_studenti is None:
    if len(excel_files) >= 2:
        # Ordina per nome per stabilità
        ef_sorted = sorted(excel_files)
        file_insegnanti = file_insegnanti or ef_sorted[0]
        file_studenti = file_studenti or ef_sorted[1]
    else:
        raise RuntimeError('Meno di due file Excel trovati, impossibile proseguire con insegnanti e studenti.')

print('File insegnanti:', file_insegnanti.name)
print('File studenti  :', file_studenti.name)

## Cella 7: Caricamento dati da Excel

Carica i file Excel di insegnanti e studenti. Utilizza la prima riga come intestazioni (domande del questionario), gestisce nomi di colonna duplicati, rimuove righe e colonne completamente vuote, e unifica le strutture dati aggiungendo una colonna "Gruppo" per distinguere insegnanti e studenti.

In [ ]:
# Carica i file usando la prima riga come domande (intestazioni)
import pandas as pd
import numpy as np

SHEET_NAME = 0  # modifica se serve un foglio diverso

# Utility per deduplicare i nomi colonna preservando l'ordine
from collections import Counter

def dedupe(names):
    seen = {}
    out = []
    for n in names:
        key = ('' if n is None or (isinstance(n, float) and np.isnan(n)) else str(n).strip())
        key = key if key else 'Senza_nome'
        if key not in seen:
            seen[key] = 0
            out.append(key)
        else:
            seen[key] += 1
            out.append(f"{key}__{seen[key]}")
    return out


def load_survey_excel(path: Path) -> tuple[pd.DataFrame, list]:
    df_raw = pd.read_excel(path, sheet_name=SHEET_NAME, header=None)
    # Prima riga con le domande
    questions = df_raw.iloc[0].tolist()
    questions = [str(x).strip() if not (isinstance(x, float) and np.isnan(x)) else '' for x in questions]
    questions = [q if q else 'Senza_nome' for q in questions]
    questions = dedupe(questions)

    # Applica le intestazioni e rimuovi la prima riga
    df = df_raw.iloc[1:].copy()
    df.columns = questions

    # Drop colonne completamente vuote e righe completamente vuote
    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=0, how='all')
    # Reset indice per sicurezza
    df = df.reset_index(drop=True)
    return df, questions

# Caricamento
_df_ins, q_insegnanti = load_survey_excel(file_insegnanti)
_df_stu, q_studenti = load_survey_excel(file_studenti)

# Uniforma le colonne facendo l'unione, riempiendo con NaN dove mancano
all_cols = list(dict.fromkeys(list(_df_ins.columns) + list(_df_stu.columns)))  # preserve order
_df_ins = _df_ins.reindex(columns=all_cols)
_df_stu = _df_stu.reindex(columns=all_cols)

# Aggiungi colonna Gruppo
_df_ins['Gruppo'] = 'insegnanti'
_df_stu['Gruppo'] = 'studenti'

# Concatena
DF = pd.concat([_df_ins, _df_stu], ignore_index=True)
print('Shape insegnanti:', _df_ins.shape)
print('Shape studenti  :', _df_stu.shape)
print('Shape totale    :', DF.shape)

# Verifica differenze di domande fra i due file
set_ins, set_stu = set(q_insegnanti), set(q_studenti)
solo_insegnanti = [c for c in set_ins - set_stu]
solo_studenti = [c for c in set_stu - set_ins]
if solo_insegnanti:
    print('\nDomande presenti solo in insegnanti (conteggio:', len(solo_insegnanti), '):')
    for x in sorted(solo_insegnanti):
        print(' -', x)
if solo_studenti:
    print('\nDomande presenti solo in studenti (conteggio:', len(solo_studenti), '):')
    for x in sorted(solo_studenti):
        print(' -', x)

# Salva il dataframe combinato
out_csv = OUTPUT_DIR / 'combined_insegnanti_studenti.csv'
DF.to_csv(out_csv, index=False)
print('\nSalvato CSV combinato in:', out_csv)

## Cella 8: Concatenazione e verifica differenze

Concatena i dataframe di insegnanti e studenti in un unico dataframe (DF). Verifica e stampa le domande presenti solo in uno dei due gruppi. Salva il dataframe combinato in formato CSV per analisi successive.

In [ ]:
# Estrai e stampa la lista completa delle domande (non troncata) e salvala su file
# Le domande sono le colonne (esclusa la colonna 'Gruppo')
questions_all = [c for c in DF.columns if c != 'Gruppo']

print('Numero totale di domande:', len(questions_all))
for idx, q in enumerate(questions_all, 1):
    print(f"{idx:3d}. {q}")

# Salva su CSV
q_out = OUTPUT_DIR / 'domande_prima_riga_insegnanti_studenti.csv'
(
    pd.DataFrame({'n': range(1, len(questions_all)+1), 'domanda': questions_all})
    .to_csv(q_out, index=False)
)
print('\nSalvata lista domande in:', q_out)

## Cella 9: Estrazione lista domande

Estrae e stampa la lista completa di tutte le domande del questionario (nomi delle colonne esclusa "Gruppo"). Salva la lista numerata in un file CSV per riferimento e documentazione.

In [ ]:
# Costruisce DF_plot con 'GruppoDettaglio' per le 4 categorie richieste
import pandas as pd
import numpy as np
import re

assert 'DF' in globals(), 'DF non trovato: esegui prima le celle di caricamento dati.'

ORDER = [
    'studenti - secondaria',
    'studenti - universitari',
    'insegnanti - non in servizio',
    'insegnanti - in servizio',
]

# Individua colonna stato insegnamento (per "attualmente insegni o hai intenzione di insegnare")
STATUS_COL = None
for c in DF.columns:
    s = str(c).lower()
    if all(tok in s for tok in ('insegn',)) and any(tok in s for tok in ('attualmente','adesso','ora','intenz','insegnare','stai')):
        STATUS_COL = c
        break

# Token per classificazione
TOK_STU_UNIV = re.compile(r"univers|ateneo|laure|trienn|magistral", re.I)
TOK_STU_PRIM = re.compile(r"primaria|primarie", re.I)
TOK_STU_SECO = re.compile(r"secondaria|superior|liceo|istituto", re.I)

# Stato insegnamento: mappe comuni (case-insensitive)
TOK_TCH_STATUS_NON = re.compile(r"ancora\s+non\s+insegno|non\s+insegno|no,?\s+ma\s+ho\s+intenz|no\s+\(intenzione\)|pre\s*-?serv|tirocin|tfa|sfp|laureand|studente|abilitaz", re.I)
TOK_TCH_STATUS_IN  = re.compile(r"s[ìi],?\s*attualmente\s*insegno|attualmente\s*insegno|ora\s*insegno|in\s+servizio", re.I)

# Utility: testo unificato per riga (valori non null)
def row_text(r: pd.Series) -> str:
    parts = []
    for v in r.values:
        if pd.isna(v):
            continue
        s = str(v).strip()
        if not s:
            continue
        if re.fullmatch(r"\d+(\.\d+)?", s):
            continue
        parts.append(s)
    return (" ".join(parts)).lower()

# Classificatore riga -> GruppoDettaglio
def classify_row(r: pd.Series) -> str:
    g = str(r.get('Gruppo', '')).strip().lower()

    if g == 'studenti':
        text = row_text(r)
        if TOK_STU_UNIV.search(text):
            return 'studenti - universitari'
        if TOK_STU_PRIM.search(text):
            return 'studenti - primaria'
        if TOK_STU_SECO.search(text):
            return 'studenti - secondaria'
        return 'studenti - secondaria'

    if g == 'insegnanti':
        # Se disponibile, usa la colonna stato insegnamento (più affidabile)
        if STATUS_COL is not None:
            val = str(r.get(STATUS_COL, '')).strip().lower()
            if TOK_TCH_STATUS_NON.search(val):
                return 'insegnanti - non in servizio'
            if TOK_TCH_STATUS_IN.search(val):
                return 'insegnanti - in servizio'
        # Fallback: pattern generali nel testo della riga
        text = row_text(r)
        if TOK_TCH_STATUS_NON.search(text):
            return 'insegnanti - non in servizio'
        if TOK_TCH_STATUS_IN.search(text):
            return 'insegnanti - in servizio'
        return 'insegnanti - in servizio'

    return g or 'sconosciuto'

DF_plot = DF.copy()
DF_plot['GruppoDettaglio'] = DF_plot.apply(classify_row, axis=1)

# Categoria ordinata (includiamo eventuali classi extra senza ordinarle)
DF_plot['GruppoDettaglio'] = pd.Categorical(
    DF_plot['GruppoDettaglio'],
    categories=ORDER + [c for c in DF_plot['GruppoDettaglio'].unique() if c not in ORDER],
    ordered=True,
)

# Report rapido di controllo
print('Colonna stato insegnamento:', STATUS_COL)
print('Distribuzione by Gruppo:')
print(DF_plot['Gruppo'].value_counts(dropna=False))
print('\nDistribuzione by GruppoDettaglio:')
print(DF_plot['GruppoDettaglio'].value_counts(dropna=False))

# Salva di servizio
(OUTPUT_DIR / 'classificazione').mkdir(parents=True, exist_ok=True)
DF_plot[['Gruppo','GruppoDettaglio']].value_counts().to_csv(OUTPUT_DIR / 'classificazione' / 'conteggi_gruppi.csv')

## Cella 10: Classificazione in 4 gruppi dettagliati

Classifica i partecipanti in 4 categorie dettagliate:
- studenti - secondaria
- studenti - universitari  
- insegnanti - non in servizio
- insegnanti - in servizio

Utilizza pattern di ricerca testuale per identificare il livello scolastico degli studenti e lo stato lavorativo degli insegnanti. Crea la colonna "GruppoDettaglio" nel dataframe.

In [ ]:
# Verifica classificazione: distribuzione risposte stato insegnamento e crosstab con GruppoDettaglio
assert 'DF_plot' in globals(), 'DF_plot non trovato: esegui la cella di classificazione.'

if STATUS_COL is None:
    print('Nessuna colonna di stato insegnamento trovata.')
else:
    vc = (
        DF.loc[DF['Gruppo'].eq('insegnanti'), STATUS_COL]
          .astype(str)
          .str.strip()
          .value_counts(dropna=False)
    )
    print('Valori distinti (insegnanti) per', STATUS_COL, ':')
    print(vc)

    # Crosstab fra stato testuale e categoria assegnata
    tab = pd.crosstab(
        DF_plot.loc[DF_plot['Gruppo'].eq('insegnanti'), 'GruppoDettaglio'],
        DF.loc[DF['Gruppo'].eq('insegnanti'), STATUS_COL].astype(str).str.strip(),
        dropna=False
    )
    display(tab)

    out = OUTPUT_DIR / 'classificazione' / 'crosstab_gruppo_vs_stato_insegnamento.csv'
    tab.to_csv(out)
    print('Crosstab salvata in:', out)


## Configurazioni condivise per grafici bilingue e ordine gruppi

Definisce costanti globali utilizzate in tutto il notebook per garantire coerenza tra i grafici in italiano e inglese:

- **Palette colori** (`PALETTE_4GROUPS`): Assegna colori specifici a ciascun gruppo (rosso=studenti secondaria, verde=universitari, blu=insegnanti in servizio, giallo=non in servizio)
- **Ordine gruppi** (`ORDER`, `ORDER_4`): Sequenza standard per visualizzazioni e analisi
- **Etichette bilingue** (`GROUP_LABELS`): Mappatura italiano↔inglese dei nomi dei gruppi
- **Etichette assi** (`LANG_AXES`): Titoli degli assi in italiano e inglese per grafici

Queste configurazioni vengono richiamate nelle celle successive per garantire uniformità visiva e semantica in tutti i grafici generati.

In [ ]:
# Configurazioni condivise per grafici bilingue e ordine gruppi
ORDER_4 = [
    'studenti - secondaria',
    'studenti - universitari',
    'insegnanti - non in servizio',
    'insegnanti - in servizio',
]

GROUP_LABELS = {
    'it': {
        'studenti - secondaria': 'Studenti - secondaria',
        'studenti - universitari': 'Studenti - universitari',
        'insegnanti - non in servizio': 'Insegnanti - non in servizio',
        'insegnanti - in servizio': 'Insegnanti - in servizio',
    },
    'en': {
        'studenti - secondaria': 'Students - secondary',
        'studenti - universitari': 'Students - university',
        'insegnanti - non in servizio': 'Teachers - pre-service',
        'insegnanti - in servizio': 'Teachers - in-service',
    }
}

LANG_AXES = {
    'it': {'xlabel': 'Gruppo', 'ylabel': 'Valore'},
    'en': {'xlabel': 'Group', 'ylabel': 'Value'},
}

PALETTE_4GROUPS = {
    'studenti - secondaria': '#e41a1c',
    'studenti - universitari': '#4daf4a',
    'insegnanti - non in servizio': '#ffd31a',
    'insegnanti - in servizio': '#377eb8',
}

def map_group_label(value, lang='it'):
    return GROUP_LABELS.get(lang, {}).get(value, str(value))

def map_group_series(series, lang='it'):
    mapping = GROUP_LABELS.get(lang, {})
    return series.map(lambda v: mapping.get(v, str(v)))

def _iqr_mask(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    if pd.isna(iqr) or iqr == 0:
        return series.between(q1, q3)
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.between(lower, upper)

def bilingual_violin_box(
    df,
    value_col,
    group_col='GruppoDettaglio',
    order=None,
    palette=None,
    slug=None,
    title_it='',
    title_en='',
    ylabel_it=None,
    ylabel_en=None,
    xlabel_it=None,
    xlabel_en=None,
    trim_outliers=False,
    save_csv=False,
    summary_suffix='_summary.csv',
    data_suffix='_data.csv',
    show_strip=True,
):
    """Crea violin e box plot bilingue (IT/EN) e salva gli output richiesti."""
    import matplotlib.pyplot as plt
    import seaborn as sns

    order = order or ORDER_4
    palette = palette or PALETTE_4GROUPS

    data = df[[group_col, value_col]].dropna().copy()
    data = data[data[group_col].isin(order)]
    if data.empty:
        print(f"⚠️ Nessun dato disponibile per {slug or value_col}")
        return data, None

    data[group_col] = pd.Categorical(data[group_col], categories=order, ordered=True)

    if trim_outliers:
        filtered = []
        for grp in order:
            subset = data[data[group_col] == grp]
            if subset.empty:
                continue
            mask = _iqr_mask(subset[value_col])
            filtered.append(subset[mask])
        if filtered:
            data = pd.concat(filtered, ignore_index=True)

    stats = (
        data.groupby(group_col, observed=False)[value_col]
            .agg(
                conteggio='count',
                media='mean',
                mediana='median',
                sd='std',
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75),
            )
            .round(2)
    )

    print(f"\nStatistiche per {slug or value_col}:")
    print(stats)

    def _save_csv(payload, suffix):
        if slug and save_csv:
            out_path = OUTPUT_DIR / f"{slug}{suffix}"
            payload.to_csv(out_path, index=False)
            print('  ↳ Salvato', out_path)

    if save_csv:
        _save_csv(data[[group_col, value_col]].copy(), data_suffix)
        _save_csv(stats.reset_index(), summary_suffix)

    for lang in ('it', 'en'):
        lang_title = title_it if lang == 'it' else (title_en or title_it)
        ylabel = ylabel_it if lang == 'it' else (ylabel_en or ylabel_it or LANG_AXES[lang]['ylabel'])
        xlabel = xlabel_it if lang == 'it' else (xlabel_en or LANG_AXES[lang]['xlabel'])
        display_df = data.copy()
        display_df['__group_display'] = map_group_series(display_df[group_col], lang)
        order_lang = [map_group_label(g, lang) for g in order if g in data[group_col].cat.categories]
        palette_lang = {map_group_label(k, lang): v for k, v in palette.items() if k in order}

        fig, ax = plt.subplots(figsize=(10, 5.5))
        sns.violinplot(
            data=display_df,
            x='__group_display',
            y=value_col,
            order=order_lang,
            palette=palette_lang,
            hue='__group_display',
            legend=False,
            inner='quartile',
            cut=0,
            density_norm='width',
            ax=ax,
        )
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        if lang_title:
            ax.set_title(lang_title)
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        ax.set_axisbelow(True)
        for tick in ax.get_xticklabels():
            tick.set_rotation(10)
            tick.set_ha('right')
        sns.despine(ax=ax)
        plt.tight_layout()
        if slug:
            fig.savefig(OUTPUT_DIR / f"{slug}_{lang}_violin.png", dpi=150)
            fig.savefig(OUTPUT_DIR / f"{slug}_{lang}_violin.svg", dpi=150)
        plt.show()
        plt.close(fig)

        fig_box, ax_box = plt.subplots(figsize=(10, 5.5))
        sns.boxplot(
            data=display_df,
            x='__group_display',
            y=value_col,
            order=order_lang,
            palette=palette_lang,
            hue='__group_display',
            legend=False,
            showfliers=not trim_outliers,
            ax=ax_box,
        )
        if show_strip:
            sns.stripplot(
                data=display_df,
                x='__group_display',
                y=value_col,
                order=order_lang,
                hue='__group_display',
                palette=palette_lang,
                dodge=False,
                size=3,
                alpha=0.7,
                linewidth=0,
                ax=ax_box,
            )
        if ax_box.get_legend():
            ax_box.get_legend().remove()
        ax_box.set_xlabel(xlabel)
        ax_box.set_ylabel(ylabel)
        title_box = f"{lang_title} — box" if lang_title else 'Box plot'
        ax_box.set_title(title_box)
        ax_box.grid(axis='y', alpha=0.3, linestyle='--')
        ax_box.set_axisbelow(True)
        for tick in ax_box.get_xticklabels():
            tick.set_rotation(10)
            tick.set_ha('right')
        sns.despine(ax=ax_box)
        plt.tight_layout()
        if slug:
            fig_box.savefig(OUTPUT_DIR / f"{slug}_{lang}_box.png", dpi=150)
            fig_box.savefig(OUTPUT_DIR / f"{slug}_{lang}_box.svg", dpi=150)
        plt.show()
        plt.close(fig_box)

    return data, stats



def ensure_df_age(force=False, verbose=True):
    """Garantisce che i dataframe sull'età siano disponibili."""
    import re
    import pandas as pd

    global df_age, df_age_trimmed, age_stats_full, age_stats_trimmed

    already_available = (
        'df_age' in globals() and 'df_age_trimmed' in globals()
        and df_age is not None and df_age_trimmed is not None
    )
    if already_available and not force:
        if verbose:
            print('df_age già disponibile (usa force=True per rigenerare).')
        return df_age, df_age_trimmed, age_stats_full, age_stats_trimmed

    assert 'DF' in globals(), 'DF non trovato: esegui la cella di caricamento dati.'
    assert 'DF_plot' in globals(), 'DF_plot non trovato: esegui le celle precedenti.'

    col_eta = None
    for tokens in (('quanti', 'anni'), ('eta',), ('età',)):
        for col in DF.columns:
            label = str(col).lower()
            if all(tok in label for tok in tokens):
                col_eta = col
                break
        if col_eta:
            break
    if col_eta is None:
        raise RuntimeError('Colonna età non trovata nel dataset')

    col_insegn_ordine = None
    for col in DF.columns:
        txt = str(col).lower()
        if all(token in txt for token in ('ordine', 'scuola')):
            col_insegn_ordine = col
            break

    num_re = re.compile(r'(\d{1,3})')

    def to_age(value):
        if pd.isna(value):
            return pd.NA
        if isinstance(value, (int, float)):
            num = float(value)
        else:
            match = num_re.search(str(value))
            if not match:
                return pd.NA
            num = float(match.group(1))
        return num if 5 <= num <= 100 else pd.NA

    age_base = DF_plot.copy()
    age_base['Eta'] = DF[col_eta].apply(to_age)

    mask_primary = age_base['GruppoDettaglio'].astype(str).str.contains('studenti - primaria', case=False, na=False)
    mask_univ = pd.Series(False, index=age_base.index)
    if col_insegn_ordine is not None:
        teacher_mask = age_base['Gruppo'].astype(str).str.contains('insegnanti', case=False, na=False)
        school_text = DF[col_insegn_ordine].astype(str).str.lower()
        mask_univ = teacher_mask & school_text.str.contains('univers|ateneo|univ', regex=True, na=False)

    filtered = age_base.loc[~mask_primary & ~mask_univ].copy()
    filtered = filtered[filtered['GruppoDettaglio'].isin(ORDER_4)]
    filtered = filtered[filtered['Eta'].notna()]
    filtered['GruppoDettaglio'] = pd.Categorical(filtered['GruppoDettaglio'], categories=ORDER_4, ordered=True)

    if verbose:
        print(f'Esempi validati: {len(filtered)} osservazioni')

    df_full = filtered[['GruppoDettaglio', 'Eta']].copy()

    def compute_stats(frame):
        if frame.empty:
            return pd.DataFrame(columns=['GruppoDettaglio', 'conteggio', 'media', 'mediana', 'sd', 'q1', 'q3'])
        return (
            frame.groupby('GruppoDettaglio', observed=False)['Eta']
                 .agg(
                     conteggio='count',
                     media='mean',
                     mediana='median',
                     sd='std',
                     q1=lambda s: s.quantile(0.25),
                     q3=lambda s: s.quantile(0.75),
                 )
                 .round(2)
        )

    trimmed_parts = []
    for grp in ORDER_4:
        subset = df_full[df_full['GruppoDettaglio'] == grp]
        if subset.empty:
            continue
        eta_vals = subset['Eta']
        q1 = eta_vals.quantile(0.25)
        q3 = eta_vals.quantile(0.75)
        iqr = q3 - q1
        if pd.isna(iqr) or iqr == 0:
            trimmed_parts.append(subset)
            continue
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        trimmed_parts.append(subset[(subset['Eta'] >= lower) & (subset['Eta'] <= upper)])

    df_trim = pd.concat(trimmed_parts, ignore_index=True) if trimmed_parts else df_full.copy()
    stats_full = compute_stats(df_full)
    stats_trim = compute_stats(df_trim)

    df_age = df_full.copy()
    df_age_trimmed = df_trim.copy()
    age_stats_full = stats_full.copy()
    age_stats_trimmed = stats_trim.copy()

    if verbose:
        print('I dataframe df_age e df_age_trimmed sono pronti per le celle successive.')

    return df_age, df_age_trimmed, age_stats_full, age_stats_trimmed


## Cella 11: Verifica classificazione

Verifica la classificazione creata nella cella precedente. Mostra la distribuzione delle risposte alla domanda sullo stato dell'insegnamento e crea una crosstab tra Gruppo e GruppoDettaglio per controllare la coerenza della classificazione.

In [ ]:
# Versioni publication-ready per i grafici sull'età
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.ticker import MultipleLocator
import unicodedata

# Helper per garantire df_age disponibile

def _normalize(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c)).lower().replace(' ', '')

def ensure_df_age(verbose: bool = False, force: bool = False):
    global df_age
    if not force and 'df_age' in globals() and isinstance(df_age, pd.DataFrame) and {'GruppoDettaglio', 'Eta'}.issubset(df_age.columns):
        if verbose:
            print('[ensure_df_age] df_age già presente.')
        return df_age
    if 'DF_plot' not in globals() or not isinstance(DF_plot, pd.DataFrame):
        raise RuntimeError('DF_plot non disponibile: esegui prima le celle di classificazione (cella con DF_plot).')
    if 'DF' not in globals() or not isinstance(DF, pd.DataFrame):
        raise RuntimeError('DF non disponibile: esegui le celle di caricamento dati.')

    # Identifica colonna età: scegli quella con molti valori tra 10 e 100
    age_candidates = []
    for c in DF.columns:
        s = pd.to_numeric(DF[c], errors='coerce')
        valid = s.between(10, 100).sum()
        if valid >= 50:  # soglia
            age_candidates.append((c, valid))
    age_candidates.sort(key=lambda x: -x[1])
    if not age_candidates:
        raise RuntimeError('Nessuna colonna con valori plausibili di età trovata.')
    age_col = age_candidates[0][0]

    # Usa DF_plot che ha GruppoDettaglio già classificato
    if 'GruppoDettaglio' not in DF_plot.columns:
        raise RuntimeError('Colonna GruppoDettaglio non trovata in DF_plot.')

    if verbose:
        print(f"[ensure_df_age] Colonna età selezionata: {age_col}")
        print(f"[ensure_df_age] Uso DF_plot con GruppoDettaglio già classificato")

    tmp = DF_plot[['GruppoDettaglio']].copy()
    tmp['Eta'] = pd.to_numeric(DF[age_col], errors='coerce')
    df_age = tmp.dropna(subset=['Eta', 'GruppoDettaglio'])
    if verbose:
        print(f"[ensure_df_age] Righe valide: {len(df_age)}")
        print(df_age['GruppoDettaglio'].value_counts())
    return df_age

# Stile
try:
    plt.style.reload_library()
    plt.style.use(['science', 'no-latex', 'grid'])
except Exception:
    plt.style.use('default')
    mpl.rcParams.update({'axes.grid': True, 'grid.alpha': 0.3, 'grid.linestyle': '--'})

mpl.rcParams.update({'text.usetex': False,'font.family': 'DejaVu Sans','font.size': 8,'savefig.dpi': 300})

# df_age
ensure_df_age(verbose=True)

# Filtra solo i 4 gruppi principali
df_age_4groups = df_age[df_age['GruppoDettaglio'].isin(ORDER_4)].copy()

# Limita ai valori plausibili per i grafici
AGE_MIN, AGE_MAX = 10, 80
df_age_plot = df_age_4groups[(df_age_4groups['Eta'] >= AGE_MIN) & (df_age_4groups['Eta'] <= AGE_MAX)].copy()

# Usa l'ordine e la palette dei 4 gruppi
order = ORDER_4
palette = PALETTE_4GROUPS

# Calcola outlier con metodo IQR (senza limiti di età per identificare tutti gli outlier)
stats_publication = (df_age_4groups.groupby('GruppoDettaglio', observed=False)['Eta'].agg(q1=lambda s: s.quantile(0.25), q3=lambda s: s.quantile(0.75)))
stats_publication['iqr'] = stats_publication['q3'] - stats_publication['q1']
stats_publication['lower'] = stats_publication['q1'] - 1.5 * stats_publication['iqr']
stats_publication['upper'] = stats_publication['q3'] + 1.5 * stats_publication['iqr']
merged = df_age_4groups.merge(stats_publication, on='GruppoDettaglio', how='left')

inliers = merged[(merged['Eta'] >= merged['lower']) & (merged['Eta'] <= merged['upper'])]
outliers = merged[(merged['Eta'] < merged['lower']) | (merged['Eta'] > merged['upper'])]

# Boxplot orizzontale con outlier
fig, ax = plt.subplots(figsize=(5, 3))
sns.boxplot(data=inliers, x='Eta', y='GruppoDettaglio', order=order, palette=palette, 
            hue='GruppoDettaglio', legend=False, ax=ax, showfliers=False)
for grp in order:
    grp_out = outliers[outliers['GruppoDettaglio'] == grp]['Eta']
    if not grp_out.empty:
        y_pos = order.index(grp)
        ax.scatter(grp_out, [y_pos]*len(grp_out), color=palette[grp], s=20, alpha=0.7, 
                   edgecolor='black', linewidth=0.5, marker='o', zorder=3)
ax.set_xlabel('Età')
ax.set_ylabel('')
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.set_xlim(AGE_MIN, AGE_MAX)
plt.tight_layout()
plt.savefig(OUTPUT_DIR/'age_boxplot_publication_h.png', dpi=300, bbox_inches='tight')
plt.show()

# Limita ai valori plausibili per l'istogramma
df_age_plot = df_age_4groups[(df_age_4groups['Eta'] >= AGE_MIN) & (df_age_4groups['Eta'] <= AGE_MAX)].copy()

# Istogramma
fig, ax = plt.subplots(figsize=(5, 3))
bins = range(AGE_MIN, AGE_MAX + 1, 5)
ax.hist(df_age_plot['Eta'], bins=bins, color='steelblue', edgecolor='black', linewidth=0.4, alpha=0.75)
ax.set_xlabel('Età')
ax.set_ylabel('Frequenza')
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.set_xlim(AGE_MIN, AGE_MAX)
plt.tight_layout()
plt.savefig(OUTPUT_DIR/'age_histogram_publication.png', dpi=300, bbox_inches='tight')
plt.show()

print('[age_plots] Salvati grafici pub-ready.')


# 0. Profilo del campione

Panoramica demografica delle quattro categorie principali e delle categorie escluse, necessaria per leggere le differenze d'uso nelle sezioni successive.

## 0.1 Distribuzione dell'età nei quattro cluster

Applichiamo i filtri (categorie escluse comprese) e produciamo, nella stessa cella, violin plot e box plot in italiano e in inglese.

In [ ]:

# Età – dataset filtrato e grafici bilingui
print('=== Controllo categorie ===')
excluded_counts = (
    DF_plot[~DF_plot['GruppoDettaglio'].isin(ORDER_4)]['GruppoDettaglio']
    .value_counts(dropna=False)
)
if excluded_counts.empty:
    print('Nessuna categoria extra oltre alle quattro principali')
else:
    print('Categorie escluse monitorate:')
    print(excluded_counts)

ensure_df_age(force=True, verbose=True)

# FILTRO CRITICO: Rimuovi valori di età assurdi (errori di input)
# Mantieni solo valori ragionevoli tra 10 e 100 anni
df_age_clean = df_age[(df_age['Eta'] >= 10) & (df_age['Eta'] <= 100)].copy()

# Filtra solo i 4 gruppi principali
df_age_clean = df_age_clean[df_age_clean['GruppoDettaglio'].isin(ORDER_4)].copy()

print(f"Dati puliti: {len(df_age_clean)} righe valide (da {len(df_age)} originali)")
print(f"Righe rimosse per età anomala: {len(df_age) - len(df_age_clean)}")

# Grafici bilingui principali
df_age_full, age_stats = bilingual_violin_box(
    df_age_clean[['GruppoDettaglio', 'Eta']].copy(),
    value_col='Eta',
    slug='eta_full',
    title_it='Distribuzione età per gruppo (campione completo)',
    title_en='Age distribution by group (full sample)',
    ylabel_it='Età (anni)',
    ylabel_en='Age (years)',
    save_csv=True,
)

df_age_trimmed, age_stats_trimmed = bilingual_violin_box(
    df_age_clean[['GruppoDettaglio', 'Eta']].copy(),
    value_col='Eta',
    slug='eta_trimmed',
    title_it='Distribuzione età per gruppo (senza outlier)',
    title_en='Age distribution by group (trimmed)',
    ylabel_it='Età (anni)',
    ylabel_en='Age (years)',
    trim_outliers=True,
)

# Rende disponibili i dataframe alle celle successive
df_age = df_age_full.copy()
df_age_trimmed = df_age_trimmed.copy()
age_stats_full = age_stats.copy()
age_stats_trimmed = age_stats_trimmed.copy()
print('I dataframe df_age e df_age_trimmed sono pronti per le celle successive.')


## 0.2 Distribuzione di genere

Conteggi e grafici bilingui (violin e box non applicabili; usiamo barre per categorie nominali).

In [ ]:
# Conteggi GENERE per Gruppo (pulito: solo questa cella per il genere)
import re
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Utility: normalizza stringhe
_def_na = {None, np.nan}

def _norm(s):
    if s is None:
        return ""
    try:
        s = str(s).strip()
    except Exception:
        return ""
    return s

def _find_col(tokens, cols):
    """Trova una colonna contenente TUTTI i token (case-insensitive)."""
    toks = [t.lower() for t in tokens]
    for c in cols:
        lc = c.lower()
        if all(t in lc for t in toks):
            return c
    return None

# Ricostruisco un base filtrato coerente con le analisi età
base_g = DF_plot.copy()
all_cols = list(DF.columns)

# Colonna genere (preferisce quella con 'genere', altrimenti 'sesso')
col_genere = _find_col(["genere"], all_cols) or _find_col(["sesso"], all_cols)
if col_genere is None:
    raise RuntimeError("Colonna del genere non trovata (atteso 'genere' o 'sesso').")

# Porto la colonna nel base (allineamento per indice)
if col_genere not in base_g.columns:
    base_g[col_genere] = DF[col_genere]

# Colonna ordine di scuola per escludere docenti universitari (se disponibile)
col_insegn_ordine = _find_col(["ordine", "scuola"], all_cols)
if col_insegn_ordine and col_insegn_ordine not in base_g.columns:
    base_g[col_insegn_ordine] = DF[col_insegn_ordine]

# Filtri: rimuovo studenti primaria e docenti universitari
mask_primaria = base_g["GruppoDettaglio"].str.contains("studenti - primaria", case=False, na=False)
mask_univ = False
if col_insegn_ordine:
    mask_univ = (
        base_g["Gruppo"].str.contains("insegnanti", case=False, na=False)
        & base_g[col_insegn_ordine].astype(str).str.contains("univers|ateneo|univ", case=False, na=False, regex=True)
    )
base_gf = base_g.loc[~mask_primaria & ~mask_univ].copy()

# Normalizzazione GENERE in tre categorie
GEN_ORDER = ["Maschio", "Femmina", "Non risponde"]

def _norm_genere(v):
    s = _norm(v).lower()
    if s == "" or "non risp" in s or "preferisc" in s:
        return "Non risponde"
    if s in {"m", "maschio"} or "masch" in s or "uomo" in s:
        return "Maschio"
    if s in {"f", "femmina"} or "femm" in s or "donna" in s:
        return "Femmina"
    # qualunque altra voce la riconduco a Non risponde per la tabella richiesta
    return "Non risponde"

base_gf["GenereCat"] = base_gf[col_genere].map(_norm_genere)

cont_genere_main = (
    pd.crosstab(base_gf["GruppoDettaglio"], base_gf["GenereCat"]).reindex(columns=GEN_ORDER, fill_value=0)
)

# Esporta CSV
out_dir = Path("output")/"exploratory"/"demographics"/"gender"
out_dir.mkdir(parents=True, exist_ok=True)
(p_gen_main := out_dir/"gender_counts_by_group.csv")
cont_genere_main.to_csv(p_gen_main)

print("Colonna GENERE:", col_genere)
print("✓ CSV salvato:", p_gen_main)
print("\nConteggi GENERE per Gruppo:")
print(cont_genere_main)

# Controllo somme per riga rispetto al totale per gruppo
row_sum = cont_genere_main.sum(axis=1)
check_df = pd.concat({"totale": base_gf.groupby("GruppoDettaglio").size(), "somma_tab": row_sum}, axis=1)
print("\nVerifica somma per gruppo (totale vs somma_tab):")
print(check_df)

# ============================================================================
# GENERAZIONE GRAFICI BILINGUE (EN + IT)
# ============================================================================

# Traduzioni
labels_it_en = {
    'studenti - secondaria': 'students - secondary',
    'studenti - universitari': 'students - university',
    'insegnanti - non in servizio': 'teachers - pre-service',
    'insegnanti - in servizio': 'teachers - in-service',
}

# Colori per genere
color_m = '#4A90E2'  # Blu per maschi
color_f = '#E94B3C'  # Rosso per femmine

# Prepara i dati
gender_plot = cont_genere_main[["Maschio", "Femmina"]].copy()
gender_order = list(ORDER_4)  # Ordine standard dei 4 gruppi

# Calcola percentuali
totals = gender_plot.sum(axis=1)
gender_perc = gender_plot.div(totals, axis=0) * 100

# ============================================================================
# GRAFICO INGLESE (sempre per primo)
# ============================================================================

fig_en, ax_en = plt.subplots(figsize=(12, 6))
x = np.arange(len(gender_order))
width = 0.35

# Converti i gruppi in inglese
gender_order_en = [labels_it_en[g] for g in gender_order]

# Barre
bars1 = ax_en.bar(x - width/2, gender_plot.loc[gender_order, 'Maschio'], width, 
                   label='Male', color=color_m, alpha=0.8)
bars2 = ax_en.bar(x + width/2, gender_plot.loc[gender_order, 'Femmina'], width, 
                   label='Female', color=color_f, alpha=0.8)

# Aggiungi etichette con numeri assoluti e percentuali
for i, grp in enumerate(gender_order):
    # Maschi
    count_m = gender_plot.loc[grp, 'Maschio']
    perc_m = gender_perc.loc[grp, 'Maschio']
    if count_m > 0:
        ax_en.text(i - width/2, count_m + 2, f'{count_m}\n({perc_m:.1f}%)', 
                   ha='center', va='bottom', fontsize=9)
    
    # Femmine
    count_f = gender_plot.loc[grp, 'Femmina']
    perc_f = gender_perc.loc[grp, 'Femmina']
    if count_f > 0:
        ax_en.text(i + width/2, count_f + 2, f'{count_f}\n({perc_f:.1f}%)', 
                   ha='center', va='bottom', fontsize=9)

ax_en.set_xlabel('Group', fontsize=12)
ax_en.set_ylabel('Count', fontsize=12)
ax_en.set_title('Gender Distribution by Group', fontsize=14, fontweight='bold')
ax_en.set_xticks(x)
ax_en.set_xticklabels(gender_order_en, rotation=15, ha='right')
ax_en.legend()
ax_en.grid(axis='y', alpha=0.3)

plt.tight_layout()

# Salva grafici EN
p_gender_en_png = out_dir / "gender_distribution_4groups_en.png"
p_gender_en_svg = out_dir / "gender_distribution_4groups_en.svg"
fig_en.savefig(p_gender_en_png, dpi=300, bbox_inches='tight')
fig_en.savefig(p_gender_en_svg, bbox_inches='tight')
plt.show()

print(f'\n✓ Grafici INGLESE salvati in:')
print(f'  PNG: {p_gender_en_png}')
print(f'  SVG: {p_gender_en_svg}')

# ============================================================================
# GRAFICO ITALIANO
# ============================================================================

fig_it, ax_it = plt.subplots(figsize=(12, 6))

# Barre
bars1_it = ax_it.bar(x - width/2, gender_plot.loc[gender_order, 'Maschio'], width, 
                      label='Maschio', color=color_m, alpha=0.8)
bars2_it = ax_it.bar(x + width/2, gender_plot.loc[gender_order, 'Femmina'], width, 
                      label='Femmina', color=color_f, alpha=0.8)

# Aggiungi etichette con numeri assoluti e percentuali
for i, grp in enumerate(gender_order):
    # Maschi
    count_m = gender_plot.loc[grp, 'Maschio']
    perc_m = gender_perc.loc[grp, 'Maschio']
    if count_m > 0:
        ax_it.text(i - width/2, count_m + 2, f'{count_m}\n({perc_m:.1f}%)', 
                   ha='center', va='bottom', fontsize=9)
    
    # Femmine
    count_f = gender_plot.loc[grp, 'Femmina']
    perc_f = gender_perc.loc[grp, 'Femmina']
    if count_f > 0:
        ax_it.text(i + width/2, count_f + 2, f'{count_f}\n({perc_f:.1f}%)', 
                   ha='center', va='bottom', fontsize=9)

ax_it.set_xlabel('Gruppo', fontsize=12)
ax_it.set_ylabel('Conteggio', fontsize=12)
ax_it.set_title('Distribuzione per genere per gruppo', fontsize=14, fontweight='bold')
ax_it.set_xticks(x)
ax_it.set_xticklabels(gender_order, rotation=15, ha='right')
ax_it.legend()
ax_it.grid(axis='y', alpha=0.3)

plt.tight_layout()

# Salva grafici IT
p_gender_it_png = out_dir / "gender_distribution_4groups_it.png"
p_gender_it_svg = out_dir / "gender_distribution_4groups_it.svg"
fig_it.savefig(p_gender_it_png, dpi=300, bbox_inches='tight')
fig_it.savefig(p_gender_it_svg, bbox_inches='tight')
plt.show()

print(f'\n✓ Grafici ITALIANO salvati in:')
print(f'  PNG: {p_gender_it_png}')
print(f'  SVG: {p_gender_it_svg}')

## 0.3 Area disciplinare (STEM vs Umanistiche)

Conteggi e grafici bilingui dedicati.

In [ ]:
# Distribuzione area disciplinare – grafici bilingui
import matplotlib.patches as mpatches

if col_stud_percorso is None or col_doc_materia is None:
    print('Colonne area disciplinare mancanti. Esegui la cella precedente.')
else:
    area_plot = area_counts[area_counts['Area_norm'].isin(['STEM', 'Umanistiche'])].copy()
    area_order = ['STEM', 'Umanistiche']
    area_plot['Area_norm'] = pd.Categorical(area_plot['Area_norm'], area_order, ordered=True)

    for lang in ('it', 'en'):
        fig, ax = plt.subplots(figsize=(14, 6))
        x = np.arange(len(ORDER_4))
        width = 0.35
        colors = {'STEM': '#1f77b4', 'Umanistiche': '#ff7f0e'}
        label_stem = 'Discipline STEM' if lang == 'it' else 'STEM fields'
        label_hum = 'Discipline umanistiche' if lang == 'it' else 'Humanities'
        xlabel = 'Gruppo' if lang == 'it' else 'Group'
        ylabel = 'Frequenza' if lang == 'it' else 'Count'
        title = 'Area disciplinare nei 4 gruppi' if lang == 'it' else 'Disciplinary area across groups'

        for i, grp in enumerate(ORDER_4):
            sub = area_plot[area_plot['GruppoDettaglio'] == grp]
            for j, area in enumerate(area_order):
                data = sub[sub['Area_norm'] == area]
                if data.empty:
                    continue
                count = int(data['conteggio'].iloc[0])
                perc = float(data['perc'].iloc[0])
                offset = -width/2 if area == 'STEM' else width/2
                ax.bar(i + offset, count, width, color=colors[area], label=(label_stem if area == 'STEM' else label_hum) if i == 0 else None)
                ax.text(i + offset, count + max(1, count*0.01), f"{count}\n({int(round(perc))}%)", ha='center', va='bottom', fontsize=10)

        ax.set_xticks(x)
        ax.set_xticklabels([map_group_label(g, lang) for g in ORDER_4])
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        legend_elements = [
            mpatches.Patch(facecolor=colors['STEM'], label=label_stem),
            mpatches.Patch(facecolor=colors['Umanistiche'], label=label_hum)
        ]
        ax.legend(handles=legend_elements, ncols=2)
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        plt.tight_layout()

        fp_png = OUT_EXPL / f'area_distribution_4groups_{lang}.png'
        fp_svg = OUT_EXPL / f'area_distribution_4groups_{lang}.svg'
        fig.savefig(fp_png, dpi=300)
        fig.savefig(fp_svg, dpi=300)
        plt.show()
        print(f'Salvati grafici area {lang.upper()}:', fp_png, 'e', fp_svg)


# 1. Usage patterns

Frequenza, intensità e varietà con cui gli strumenti di IA vengono utilizzati nei diversi profili educativi.

## 1.1 Adozione quotidiana (Sì/No)

Distribuzione binaria dell'uso dell'IA nella vita quotidiana per ciascun gruppo (grafici IT/EN).

In [ ]:
# Uso quotidiano dell'IA (sì/no) — istogrammi per 4 categorie (IT/EN)
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Stile coerente con il resto del notebook
try:
    import scienceplots  # noqa: F401
except Exception:
    try:
        ensure('SciencePlots')
        import scienceplots  # noqa: F401
    except Exception:
        pass
plt.style.use(['science', 'no-latex', 'grid'])
mpl.rcParams.update({
    'text.usetex': False,
    'mathtext.fontset': 'dejavusans',
    'font.family': 'DejaVu Sans',
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 12,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
})

# Helper per ricerca colonne

def find_col_any(cols, token_options):
    for c in cols:
        text = str(c).lower()
        for tokens in token_options:
            if all(tok in text for tok in tokens):
                return c
    return None

# Normalizza sì/no
_yes_re = re.compile(r'^(si|sì|yes|y|true|1)$', re.IGNORECASE)
_no_re  = re.compile(r'^(no|n|false|0)$', re.IGNORECASE)

def to_yes_no(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip().lower()
    s = s.replace('ì', 'i')  # gestisci accento per matching semplice
    if _yes_re.match(s):
        return True
    if _no_re.match(s):
        return False
    return np.nan

# Base di lavoro e ordine/palette gruppi
base = DF_plot.copy()
order = [
    'studenti - secondaria',
    'studenti - universitari',
    'insegnanti - non in servizio',
    'insegnanti - in servizio',
]
palette = {
    'studenti - secondaria': 'red',
    'studenti - universitari': 'forestgreen',
    'insegnanti - in servizio': 'royalblue',
    'insegnanti - non in servizio': 'gold',
}

# Individua colonna per la domanda “Nella tua vita quotidiana utilizzi l'intelligenza artificiale?”
# Potrebbero esserci due colonne (questionari diversi). Cerchiamo per token.
all_cols = list(DF.columns)
col_daily_stu = find_col_any(all_cols, [["nella", "vita", "quotidian", "intelligenza", "artificial"],
                                        ["vita", "quotidian", "utilizzi", "intelligenza"]])
col_daily_doc = col_daily_stu  # spesso il testo è identico; se necessario, differenziarlo qui
if col_daily_stu is None:
    # fallback: cerca inglese o varianti
    col_daily_stu = find_col_any(all_cols, [["daily", "life", "use", "ai"], ["use", "artificial", "intelligence", "daily"]])
if col_daily_stu is None:
    raise RuntimeError("Impossibile trovare la colonna della domanda: 'Nella tua vita quotidiana utilizzi l'intelligenza artificiale?' ")

# Assicura colonne nella base
for c in {col_daily_stu, col_daily_doc}:
    if c and c not in base.columns:
        base[c] = DF[c]

# Filtri coerenti con il resto (escludi primaria e docenti universitari)
col_insegn_ordine = find_col_any(
    DF.columns,
    [["ordine", "scuola", "insegni"], ["ordine", "scuola", "vorrest"], ["ordine", "scuola"]]
)
mask_primaria = base['GruppoDettaglio'].astype(str).str.contains('studenti - primaria', case=False, na=False)
mask_univ = False
if col_insegn_ordine is not None:
    m_teacher = base['Gruppo'].astype(str).str.contains('insegnanti', case=False, na=False)
    tvals = base.loc[m_teacher, col_insegn_ordine].astype(str).str.lower()
    m_univ = tvals.str.contains(r"univers|ateneo|univ", regex=True, na=False)
    mask_univ = pd.Series(False, index=base.index)
    mask_univ.loc[tvals[m_univ].index] = True
filt = (~mask_primaria) & (~mask_univ) if isinstance(mask_univ, pd.Series) else (~mask_primaria)
df_bin = base.loc[filt].copy()

# Colonna binaria sì/no per riga, distinguendo se serve studenti vs insegnanti
is_student = df_bin['Gruppo'].astype(str).str.contains('studenti', case=False, na=False)
df_bin['DailyAI'] = np.where(
    is_student,
    df_bin[col_daily_stu].apply(to_yes_no),
    df_bin[col_daily_doc].apply(to_yes_no) if col_daily_doc else np.nan,
)

# Tieni solo le 4 categorie ordinate
df_bin = df_bin[df_bin['GruppoDettaglio'].isin(order)].copy()
df_bin['GruppoDettaglio'] = pd.Categorical(df_bin['GruppoDettaglio'], categories=order, ordered=True)

# Tabella conteggi per gruppo (True/False)
counts = (
    df_bin.groupby(['GruppoDettaglio', 'DailyAI'], observed=False)
          .size()
          .rename('conteggio')
          .reset_index()
)
# Completa livelli mancanti (True/False) per ogni gruppo
all_idx = pd.MultiIndex.from_product([order, [False, True]], names=['GruppoDettaglio','DailyAI'])
counts = counts.set_index(['GruppoDettaglio','DailyAI']).reindex(all_idx, fill_value=0).reset_index()

# Calcola percentuali per gruppo
counts['totale_gruppo'] = counts.groupby('GruppoDettaglio')['conteggio'].transform('sum')
counts['perc'] = np.where(counts['totale_gruppo']>0, counts['conteggio'] / counts['totale_gruppo'], np.nan)

# Salvataggi CSV
OUT = (Path.cwd()/'../analysis/exports/latest').resolve()
OUT.mkdir(parents=True, exist_ok=True)
counts_csv = OUT / 'daily_ai_use_yesno_counts.csv'
counts.to_csv(counts_csv, index=False)
print('Conteggi salvati in:', counts_csv)

# Funzione per disegnare una griglia 2x2 (lasciata per riferimento)

def plot_grid(counts_df, lang='it'):
    labels = {'it': {'title': "Uso quotidiano dell'IA (sì/no) — per categoria",
                     'xlabel': 'Risposta', 'ylabel': 'Frequenza', 'yes': 'Sì', 'no': 'No'},
              'en': {'title': 'Daily AI use (yes/no) — by category',
                     'xlabel': 'Response', 'ylabel': 'Count', 'yes': 'Yes', 'no': 'No'}}
    lab = labels.get(lang, labels['it'])

    fig, axes = plt.subplots(2, 2, figsize=(9, 6), sharex=True)
    axes = axes.flatten()
    for i, grp in enumerate(order):
        ax = axes[i]
        sub = counts_df[counts_df['GruppoDettaglio'] == grp].copy()
        # Ordina sempre Sì prima di No
        sub = sub.sort_values('DailyAI', ascending=False)  # True (Sì), False (No)
        x = [lab['yes'], lab['no']]
        y = [int(sub.loc[sub['DailyAI'] == True, 'conteggio'].values[0]),
             int(sub.loc[sub['DailyAI'] == False, 'conteggio'].values[0])]
        color = palette['insegnanti - in servizio'] if 'insegnanti' in grp else palette['studenti - secondaria']
        if 'universitari' in grp:
            color = palette['studenti - universitari']
        if 'non in servizio' in grp:
            color = palette['insegnanti - non in servizio']
        ax.bar(x, y, color=color, alpha=0.9)
        ax.set_title(grp)
        ax.set_xlabel(lab['xlabel'])
        ax.set_ylabel(lab['ylabel'])
        # Aggiungi le percentuali nelle etichette
        for resp in [True, False]:
            val = int(sub.loc[sub['DailyAI']==resp, 'conteggio'].values[0])
            pct = float(sub.loc[sub['DailyAI']==resp, 'perc'].values[0]) * 100 if sub.loc[sub['DailyAI']==resp, 'totale_gruppo'].values[0]>0 else np.nan
            lbl = f"{val} ({pct:.0f}%)" if pct==pct else f"{val}"
            xi = lab['yes'] if resp else lab['no']
            ax.text(xi, val, lbl, ha='center', va='bottom', fontsize=10)
        ax.set_ylim(0, max(y)*1.25 + 1)
    fig.suptitle(lab['title'])
    plt.tight_layout(rect=[0, 0.02, 1, 0.96])
    return fig

# Nuovo: unico grafico con 4 gruppi e barre Sì/No (Sì sempre prima) + percentuali
from matplotlib.colors import to_rgb

def lighten(color, amount=0.5):
    r, g, b = to_rgb(color)
    return (1 - amount) + amount * r, (1 - amount) + amount * g, (1 - amount) + amount * b

def plot_combined(counts_df, lang='it'):
    labels = {'it': {'title': "Uso quotidiano dell'IA (sì/no) — 4 gruppi",
                     'xlabel': 'Gruppo', 'ylabel': 'Frequenza', 'yes': 'Sì', 'no': 'No',
                     'groups': order},
              'en': {'title': 'Daily AI use (yes/no) — 4 groups',
                     'xlabel': 'Group', 'ylabel': 'Count', 'yes': 'Yes', 'no': 'No',
                     'groups': ['students - secondary','students - university','teachers - not in service','teachers - in service']}}
    lab = labels.get(lang, labels['it'])

    # Preparazione dati: garantisci Sì (True) poi No (False)
    df = counts_df.copy().sort_values(['GruppoDettaglio','DailyAI'], ascending=[True, False])
    # Posizioni
    x = np.arange(len(order))
    width = 0.38

    fig, ax = plt.subplots(figsize=(11, 6))
    # Disegna per ciascun gruppo le due barre: Sì (a sinistra), No (a destra)
    bars = []
    for i, grp in enumerate(order):
        sub = df[df['GruppoDettaglio']==grp]
        yes = int(sub.loc[sub['DailyAI']==True, 'conteggio'].iloc[0])
        no  = int(sub.loc[sub['DailyAI']==False, 'conteggio'].iloc[0])
        yes_p = float(sub.loc[sub['DailyAI']==True, 'perc'].iloc[0]) * 100 if int(sub['totale_gruppo'].iloc[0])>0 else np.nan
        no_p  = float(sub.loc[sub['DailyAI']==False, 'perc'].iloc[0]) * 100 if int(sub['totale_gruppo'].iloc[0])>0 else np.nan
        base_col = palette['insegnanti - in servizio'] if 'insegnanti' in grp else palette['studenti - secondaria']
        if 'universitari' in grp:
            base_col = palette['studenti - universitari']
        if 'non in servizio' in grp:
            base_col = palette['insegnanti - non in servizio']
        col_yes = base_col
        col_no  = lighten(base_col, 0.7)
        b1 = ax.bar(x[i]-width/2, yes, width, color=col_yes)
        b2 = ax.bar(x[i]+width/2, no,  width, color=col_no)
        # Etichette con percentuali
        ax.text(x[i]-width/2, yes, f"{yes} ({yes_p:.0f}%)" if yes_p==yes_p else f"{yes}", ha='center', va='bottom', fontsize=10)
        ax.text(x[i]+width/2, no,  f"{no} ({no_p:.0f}%)"   if no_p==no_p else f"{no}",  ha='center', va='bottom', fontsize=10)
        bars.extend([b1,b2])
    ax.set_xticks(x)
    ax.set_xticklabels(lab['groups'], rotation=0)
    ax.set_xlabel(lab['xlabel'])
    ax.set_ylabel(lab['ylabel'])
    ax.set_title(lab['title'])
    # Leggenda con patch personalizzate (colori neutri)
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='darkgray', label=lab['yes']),
        Patch(facecolor='lightgray', label=lab['no'])
    ]
    leg = ax.legend(handles=legend_elements, ncols=2, frameon=False)
    ax.margins(x=0.02)
    ax.set_ylim(0, max(counts_df['conteggio'])*1.15 + 5)
    plt.tight_layout()
    return fig

# Disegna e salva figure IT/EN (griglia e grafico unico)
fig_it = plot_grid(counts, lang='it')
p_it_png = OUT / 'hist_daily_ai_use_it.png'
p_it_svg = OUT / 'hist_daily_ai_use_it.svg'
fig_it.savefig(p_it_png)
fig_it.savefig(p_it_svg)
plt.show()
plt.close(fig_it)

fig_en = plot_grid(counts, lang='en')
p_en_png = OUT / 'hist_daily_ai_use_en.png'
p_en_svg = OUT / 'hist_daily_ai_use_en.svg'
fig_en.savefig(p_en_png)
fig_en.savefig(p_en_svg)
plt.show()
plt.close(fig_en)

# Nuovi grafici unici
g_it = plot_combined(counts, lang='it')
out_it_png = OUT / 'hist_daily_ai_use_combined_it.png'
out_it_svg = OUT / 'hist_daily_ai_use_combined_it.svg'
g_it.savefig(out_it_png)
g_it.savefig(out_it_svg)
plt.show()
plt.close(g_it)

g_en = plot_combined(counts, lang='en')
out_en_png = OUT / 'hist_daily_ai_use_combined_en.png'
out_en_svg = OUT / 'hist_daily_ai_use_combined_en.svg'
g_en.savefig(out_en_png)
g_en.savefig(out_en_svg)
plt.show()
plt.close(g_en)

print('Salvati:', p_it_png, p_it_svg, p_en_png, p_en_svg, out_it_png, out_it_svg, out_en_png, out_en_svg)

## 1.2 Intensità settimanale di utilizzo

Ore dedicate all'IA nella vita quotidiana per ciascun gruppo (violin + box bilingui).

In [ ]:
# Ore settimanali di uso IA — grafici bilingui con violino/box
import re
from IPython.display import display

col_hours = None
for col in DF.columns:
    label = str(col).lower()
    if all(token in label for token in ('ore', 'settimana')) and 'ia' in label:
        col_hours = col
        break
if col_hours is None:
    raise RuntimeError('Colonna ore settimanali per uso IA non trovata')

col_insegn_ordine = None
for col in DF.columns:
    txt = str(col).lower()
    if all(token in txt for token in ('ordine', 'scuola')):
        col_insegn_ordine = col
        break

base_hours = DF_plot.copy()
if col_hours not in base_hours.columns:
    base_hours[col_hours] = DF[col_hours]
if col_insegn_ordine and col_insegn_ordine not in base_hours.columns:
    base_hours[col_insegn_ordine] = DF[col_insegn_ordine]

mask_primary = base_hours['GruppoDettaglio'].astype(str).str.contains('studenti - primaria', case=False, na=False)
mask_univ = pd.Series(False, index=base_hours.index)
if col_insegn_ordine is not None:
    teacher_mask = base_hours['Gruppo'].astype(str).str.contains('insegnanti', case=False, na=False)
    school_text = base_hours[col_insegn_ordine].astype(str).str.lower()
    mask_univ = teacher_mask & school_text.str.contains('univers|ateneo|univ', regex=True, na=False)

num_re = re.compile(r"(\d+[\.,]?\d*)")

def to_hours(value):
    if pd.isna(value):
        return pd.NA
    if isinstance(value, (int, float)):
        val = float(value)
    else:
        match = num_re.search(str(value))
        if not match:
            return pd.NA
        val = float(match.group(1).replace(',', '.'))
    return val if 0 <= val <= 168 else pd.NA

is_student = base_hours['Gruppo'].astype(str).str.contains('studenti', case=False, na=False)
base_hours['OreSettimanali'] = base_hours[col_hours].apply(to_hours)

hours_df = base_hours.loc[~mask_primary & ~mask_univ].copy()
hours_df = hours_df[hours_df['GruppoDettaglio'].isin(ORDER_4) & hours_df['OreSettimanali'].notna()]
hours_df['GruppoDettaglio'] = pd.Categorical(hours_df['GruppoDettaglio'], categories=ORDER_4, ordered=True)
print(f'Osservazioni valide per ore settimanali: {len(hours_df)}')

hours_full, hours_stats = bilingual_violin_box(
    hours_df[['GruppoDettaglio', 'OreSettimanali']],
    value_col='OreSettimanali',
    slug='ore_settimanali_ia',
    title_it='Uso IA nella vita quotidiana — ore settimanali',
    title_en='Daily-life AI usage — weekly hours',
    ylabel_it='Ore settimanali (uso IA)',
    ylabel_en='Weekly hours (AI use)',
    save_csv=True,
)

hours_trimmed, hours_stats_trimmed = bilingual_violin_box(
    hours_df[['GruppoDettaglio', 'OreSettimanali']],
    value_col='OreSettimanali',
    slug='ore_settimanali_ia_trimmed',
    title_it='Uso quotidiano IA — ore settimanali (senza outlier)',
    title_en='Daily-life AI usage — weekly hours (trimmed)',
    ylabel_it='Ore settimanali (uso IA)',
    ylabel_en='Weekly hours (AI use)',
    trim_outliers=True,
)

print(hours_stats)
display(hours_stats)

df_use = hours_full.copy()
df_use_trimmed = hours_trimmed.copy()
print('\nI dataframe df_use e df_use_trimmed sono aggiornati per le analisi successive.')


## 1.3 Uso per studio vs didattica

Confronto delle attivazioni (Sì/No) tra studenti e insegnanti nei rispettivi contesti.

In [ ]:
# Uso dell'IA nello studio (studenti) vs nella didattica (insegnanti) — sì/no con percentuali (IT/EN)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()
ASSETS = (Path.cwd()/"../assets/figures").resolve()
OUT_EXPL.mkdir(parents=True, exist_ok=True)
ASSETS.mkdir(parents=True, exist_ok=True)

# Trova colonne delle due domande principali

def find_col(df, tokens_all, tokens_any=None):
    cols = list(df.columns)
    toks_all = [t.lower() for t in tokens_all]
    for c in cols:
        s = str(c).lower()
        if all(t in s for t in toks_all):
            if not tokens_any:
                return c
            if any(t in s for t in [x.lower() for x in tokens_any]):
                return c
    # fallback: any of toks_all
    for c in cols:
        s = str(c).lower()
        if any(t in s for t in toks_all):
            return c
    return None

COL_STUDY = find_col(DF_plot, ["utilizzi", "intelligenza", "studio"])  # "Utilizzi l'intelligenza artificiale nello studio?"
COL_TEACH = find_col(DF_plot, ["utilizzi", "intelligenza", "didattica"]) # "Utilizzi l'intelligenza artificiale nella didattica?"
print("Colonne rilevate:", COL_STUDY, "|", COL_TEACH)

YES_IT, NO_IT = "Sì", "No"
YES_EN, NO_EN = "Yes", "No"

def norm_yesno(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip().lower()
    s = s.replace("ì","i")
    if s in {"si","yes","y","true","1"}:
        return YES_IT
    if s in {"no","n","false","0"}:
        return NO_IT
    return np.nan

ORDER_4 = [
    "studenti - secondaria",
    "studenti - universitari",
    "insegnanti - non in servizio",
    "insegnanti - in servizio",
]

base = DF_plot.copy()
mask_stu = base["GruppoDettaglio"].isin(ORDER_4[:2])
mask_tch = base["GruppoDettaglio"].isin(ORDER_4[2:])

base.loc[mask_stu, "UsoAI_studio_didattica"] = base.loc[mask_stu, COL_STUDY].map(norm_yesno) if COL_STUDY in base.columns else np.nan
base.loc[mask_tch, "UsoAI_studio_didattica"] = base.loc[mask_tch, COL_TEACH].map(norm_yesno) if COL_TEACH in base.columns else np.nan

use = base[base["UsoAI_studio_didattica"].isin([YES_IT, NO_IT]) & base["GruppoDettaglio"].isin(ORDER_4)].copy()
use["GruppoDettaglio"] = pd.Categorical(use["GruppoDettaglio"], ORDER_4, ordered=True)

counts = (use.groupby(["GruppoDettaglio","UsoAI_studio_didattica"]).size().reset_index(name="conteggio"))
# completa buchi
idx = pd.MultiIndex.from_product([ORDER_4,[YES_IT, NO_IT]], names=["GruppoDettaglio","UsoAI_studio_didattica"])

counts = (
    counts
    .set_index(["GruppoDettaglio","UsoAI_studio_didattica"])
    .reindex(idx, fill_value=0)
    .rename_axis(["GruppoDettaglio","UsoAI_studio_didattica"])
    .reset_index()
)

if "UsoAI_studio_didattica" not in counts.columns and "level_1" in counts.columns:
    counts = counts.rename(columns={"level_1": "UsoAI_studio_didattica"})
if "GruppoDettaglio" not in counts.columns and "level_0" in counts.columns:
    counts = counts.rename(columns={"level_0": "GruppoDettaglio"})
counts["totale_gruppo"] = counts.groupby("GruppoDettaglio")["conteggio"].transform("sum")
counts["perc"] = (counts["conteggio"]/counts["totale_gruppo"]*100).round(1)

counts_csv_it = OUT_EXPL/"ai_use_study_teaching_yesno_counts_it.csv"
counts.to_csv(counts_csv_it, index=False)
print("Conteggi IT salvati:", counts_csv_it)

palette = {
    "studenti - secondaria": "#e41a1c",
    "studenti - universitari": "#4daf4a",
    "insegnanti - non in servizio": "#ffd31a",
    "insegnanti - in servizio": "#3b6ce1",
}

import matplotlib.colors as mcolors

def lighten(c, a=0.7):
    r,g,b = mcolors.to_rgb(c)
    return (1 - a) + a*r, (1 - a) + a*g, (1 - a) + a*b

# Combined 4-groups IT
fig, ax = plt.subplots(figsize=(14,6))
x = np.arange(len(ORDER_4)); width = 0.36
for i, grp in enumerate(ORDER_4):
    sub = counts[counts["GruppoDettaglio"]==grp]
    c = palette[grp]
    yes = int(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "conteggio"].iloc[0])
    no  = int(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "conteggio"].iloc[0])
    yp  = float(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "perc"].iloc[0])
    np_ = float(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "perc"].iloc[0])
    ax.bar(i-width/2, yes, width, color=c)
    ax.bar(i+width/2, no,  width, color=lighten(c,0.7))
    ax.text(i-width/2, yes+max(1,yes*0.01), f"{yes} ({int(round(yp))}%)", ha='center', va='bottom', fontsize=11)
    ax.text(i+width/2, no+max(1,no*0.01),   f"{no} ({int(round(np_))}%)", ha='center', va='bottom', fontsize=11)
ax.set_xticks(x); ax.set_xticklabels(ORDER_4)
ax.set_title("Uso dell'IA nello studio/didattica (sì/no) — 4 gruppi")
ax.set_xlabel("Gruppo"); ax.set_ylabel("Frequenza")
# Leggenda con patch personalizzate
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='darkgray', label='Sì'),
    Patch(facecolor='lightgray', label='No')
]
ax.legend(handles=legend_elements, ncols=2)
ax.grid(axis='y', alpha=0.3, linestyle='--'); plt.tight_layout()
fp_it_png = OUT_EXPL/"ai_use_study_teaching_4groups_it.png"; fp_it_svg = OUT_EXPL/"ai_use_study_teaching_4groups_it.svg"
fig.savefig(fp_it_png, dpi=300); fig.savefig(fp_it_svg, dpi=300)
plt.show()
print("Salvati:", fp_it_png, fp_it_svg)

# EN copy (labels only)
map_it_en = {
    "studenti - secondaria": "students - secondary",
    "studenti - universitari": "students - university",
    "insegnanti - non in servizio": "teachers - not in service",
    "insegnanti - in servizio": "teachers - in service",
}
fig, ax = plt.subplots(figsize=(14,6))
x = np.arange(len(ORDER_4)); width = 0.36
for i, grp in enumerate(ORDER_4):
    sub = counts[counts["GruppoDettaglio"]==grp]
    c = palette[grp]
    yes = int(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "conteggio"].iloc[0])
    no  = int(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "conteggio"].iloc[0])
    yp  = float(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "perc"].iloc[0])
    np_ = float(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "perc"].iloc[0])
    ax.bar(i-width/2, yes, width, color=c)
    ax.bar(i+width/2, no,  width, color=lighten(c,0.7))
    ax.text(i-width/2, yes+max(1,yes*0.01), f"{yes} ({int(round(yp))}%)", ha='center', va='bottom', fontsize=11)
    ax.text(i+width/2, no+max(1,no*0.01),   f"{no} ({int(round(np_))}%)", ha='center', va='bottom', fontsize=11)
ax.set_xticks(x); ax.set_xticklabels([map_it_en[g] for g in ORDER_4])
ax.set_title("AI use in study/teaching (yes/no) — 4 groups")
ax.set_xlabel("Group"); ax.set_ylabel("Count")
# Leggenda con patch personalizzate
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='darkgray', label='Yes'),
    Patch(facecolor='lightgray', label='No')
]
ax.legend(handles=legend_elements, ncols=2)
ax.grid(axis='y', alpha=0.3, linestyle='--'); plt.tight_layout()
fp_en_png = OUT_EXPL/"ai_use_study_teaching_4groups_en.png"; fp_en_svg = OUT_EXPL/"ai_use_study_teaching_4groups_en.svg"
fig.savefig(fp_en_png, dpi=300); fig.savefig(fp_en_svg, dpi=300)
plt.show()
print("Saved:", fp_en_png, fp_en_svg)

# Copia in assets i due principali
import shutil
try:
    shutil.copyfile(fp_en_png, ASSETS/"ai_use_study_teaching_4groups_en.png")
    print("Copiati in assets.")
except Exception as e:
    print("Avviso copia assets:", e)



## 1.4 Sintesi combinata dei quattro gruppi

Vista unica con barre Sì/No per i quattro cluster (IT/EN) e salvataggi CSV.

In [ ]:
# Variante a due pannelli: a sinistra gli studenti (uso nello studio), a destra gli insegnanti (uso nella didattica)

import matplotlib.pyplot as plt
from pathlib import Path

ORDER_STU = ["studenti - secondaria", "studenti - universitari"]
ORDER_TCH = ["insegnanti - non in servizio", "insegnanti - in servizio"]

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
width = 0.36

for ax, groups, title in [
    (axes[0], ORDER_STU, "Studenti — uso nello studio"),
    (axes[1], ORDER_TCH, "Insegnanti — uso nella didattica"),
]:
    x = np.arange(len(groups))
    for i, grp in enumerate(groups):
        sub = counts[counts["GruppoDettaglio"]==grp]
        c = palette[grp]
        yes = int(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "conteggio"].iloc[0])
        no  = int(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "conteggio"].iloc[0])
        yp  = float(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "perc"].iloc[0])
        np_ = float(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "perc"].iloc[0])
        ax.bar(i-width/2, yes, width, color=c)
        ax.bar(i+width/2, no,  width, color=lighten(c,0.7))
        ax.text(i-width/2, yes+max(1,yes*0.01), f"{yes} ({int(round(yp))}%)", ha='center', va='bottom', fontsize=11)
        ax.text(i+width/2, no+max(1,no*0.01),   f"{no} ({int(round(np_))}%)", ha='center', va='bottom', fontsize=11)
    ax.set_xticks(x)
    ax.set_xticklabels(groups)
    ax.set_title(title)
    ax.set_xlabel("Gruppo")
    ax.set_ylabel("Frequenza")
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    # Leggenda con patch personalizzate
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='darkgray', label='Sì'),
        Patch(facecolor='lightgray', label='No')
    ]
    ax.legend(handles=legend_elements, ncols=2)

plt.suptitle("Uso dell'IA nello studio/didattica (sì/no) — studenti vs insegnanti")
plt.tight_layout(rect=[0,0.03,1,0.98])

fp_pan_it_png = OUT_EXPL/"ai_use_study_teaching_panels_it.png"
fp_pan_it_svg = OUT_EXPL/"ai_use_study_teaching_panels_it.svg"
fig.savefig(fp_pan_it_png, dpi=300); fig.savefig(fp_pan_it_svg, dpi=300)
plt.show()
print("Salvati (pannelli IT):", fp_pan_it_png, fp_pan_it_svg)

# EN labels
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
for ax, groups, title in [
    (axes[0], ORDER_STU, "Students — study use"),
    (axes[1], ORDER_TCH, "Teachers — teaching use"),
]:
    x = np.arange(len(groups))
    for i, grp in enumerate(groups):
        sub = counts[counts["GruppoDettaglio"]==grp]
        c = palette[grp]
        yes = int(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "conteggio"].iloc[0])
        no  = int(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "conteggio"].iloc[0])
        yp  = float(sub.loc[sub["UsoAI_studio_didattica"]==YES_IT, "perc"].iloc[0])
        np_ = float(sub.loc[sub["UsoAI_studio_didattica"]==NO_IT, "perc"].iloc[0])
        ax.bar(i-width/2, yes, width, color=c)
        ax.bar(i+width/2, no,  width, color=lighten(c,0.7))
        ax.text(i-width/2, yes+max(1,yes*0.01), f"{yes} ({int(round(yp))}%)", ha='center', va='bottom', fontsize=11)
        ax.text(i+width/2, no+max(1,no*0.01),   f"{no} ({int(round(np_))}%)", ha='center', va='bottom', fontsize=11)
    ax.set_xticks(x)
    ax.set_xticklabels([map_it_en[g] for g in groups])
    ax.set_title(title)
    ax.set_xlabel("Group")
    ax.set_ylabel("Count")
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    # Leggenda con patch personalizzate
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='darkgray', label='Yes'),
        Patch(facecolor='lightgray', label='No')
    ]
    ax.legend(handles=legend_elements, ncols=2)

plt.suptitle("AI use in study/teaching (yes/no) — students vs teachers")
plt.tight_layout(rect=[0,0.03,1,0.98])

fp_pan_en_png = OUT_EXPL/"ai_use_study_teaching_panels_en.png"
fp_pan_en_svg = OUT_EXPL/"ai_use_study_teaching_panels_en.svg"
fig.savefig(fp_pan_en_png, dpi=300); fig.savefig(fp_pan_en_svg, dpi=300)
plt.show()
print("Saved (panels EN):", fp_pan_en_png, fp_pan_en_svg)



## 1.6 Significatività statistica dei pattern di uso

Verifichiamo se le differenze osservate tra gruppi sono statisticamente rilevanti (proporzioni e ore).

### 1.6.a Test Chi-quadrato (uso quotidiano Sì/No)

In [ ]:
# === PREPARAZIONE DATI PER TEST STATISTICI ===
import pandas as pd
import numpy as np
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*80)
print("PREPARAZIONE DATI PER TEST STATISTICI")
print("="*80)

# 1. PREPARAZIONE tab_gen (Genere × Uso IA)
print("\n1. Creazione tab_gen (Genere × Uso IA)...")

# Usa col_daily_stu se disponibile
if 'col_daily_stu' in globals() and col_daily_stu is not None:
    col_use = col_daily_stu
else:
    col_use = next((c for c in DF.columns if 'utilizzi quotidianamente' in c.lower()), None)

if col_use and 'base_gf' in globals():
    yes_set = set(['si','sì','yes','y','1','true'])
    
    tab_gen = DF_plot[['GruppoDettaglio']].copy()
    tab_gen['GenereCat'] = base_gf['GenereCat'] if 'GenereCat' in base_gf.columns else 'Non risponde'
    tab_gen['uses_ai'] = DF[col_use].apply(lambda v: str(v).strip().lower() in yes_set if pd.notna(v) else False)
    
    # Aggregazione: per ogni (Gruppo, Genere) conta totale e utenti
    agg = (
        tab_gen[tab_gen['GruppoDettaglio'].isin(ORDER_4)]
        .groupby(['GruppoDettaglio', 'GenereCat'], observed=True)
        .agg(total=('uses_ai', 'size'), users=('uses_ai', 'sum'))
        .reset_index()
    )
    agg['perc_users_of_gender'] = (agg['users'] / agg['total'] * 100).round(1)
    tab_gen = agg
    print(f"   ✓ tab_gen creato: {len(tab_gen)} righe")
else:
    print("   ⚠️  Impossibile creare tab_gen (mancano col_use o base_gf)")
    tab_gen = None

# 2. PREPARAZIONE df_check (per test chi-quadrato differenze gruppi)
print("\n2. Creazione df_check (Uso IA per gruppo)...")

if col_use and 'base_gf' in globals():
    yes_set = set(['si','sì','yes','y','1','true'])
    
    df_check = DF_plot[['GruppoDettaglio']].copy()
    if 'GenereCat' in base_gf.columns:
        df_check['GenereCat'] = base_gf['GenereCat']
    df_check['uses_ai'] = DF[col_use].apply(lambda v: str(v).strip().lower() in yes_set if pd.notna(v) else False)
    df_check = df_check[df_check['GruppoDettaglio'].isin(ORDER_4)]
    print(f"   ✓ df_check creato: {len(df_check)} righe")
else:
    print("   ⚠️  Impossibile creare df_check")
    df_check = None

# 3. PREPARAZIONE df_use (Ore settimanali per gruppo)
print("\n3. Creazione df_use (Ore settimanali)...")

# Cerca colonna ore settimanali - pattern più flessibile
col_hours = None
for c in DF.columns:
    c_lower = c.lower()
    if 'ore' in c_lower and 'settiman' in c_lower and 'attività quotidiane' in c_lower:
        col_hours = c
        print(f"   Trovata colonna ore: {c[:70]}...")
        break

if col_hours:
    # Funzione per convertire in ore (gestisce testo e numeri)
    def to_hours(val):
        if pd.isna(val):
            return np.nan
        val_str = str(val).strip().lower()
        if val_str in ['non lo so', 'non rispondo', '']:
            return np.nan
        try:
            return float(val_str)
        except:
            return np.nan
    
    df_use = DF_plot[['GruppoDettaglio']].copy()
    df_use['OreSettimanali'] = DF[col_hours].apply(to_hours)
    df_use = df_use[df_use['GruppoDettaglio'].isin(ORDER_4)]
    df_use = df_use.dropna(subset=['OreSettimanali'])
    print(f"   ✓ df_use creato: {len(df_use)} righe valide")
else:
    print("   ⚠️  Impossibile creare df_use (colonna ore settimanali non trovata)")
    df_use = None

# 4. PREPARAZIONE tab_area (Area disciplinare × Uso IA)
print("\n4. Creazione tab_area (Area × Uso IA)...")

# Cerca colonna area disciplinare in DF
col_area = None
for c in DF.columns:
    c_lower = c.lower()
    if ('settore' in c_lower and 'scientific' in c_lower) or ('classe' in c_lower and 'concorso' in c_lower):
        col_area = c
        print(f"   Trovata colonna area: {c[:60]}...")
        break

if col_area and col_use:
    yes_set = set(['si','sì','yes','y','1','true'])
    
    # Normalizza le aree in STEM / Umanistiche
    def norm_area(val):
        if pd.isna(val):
            return 'Non risponde'
        s = str(val).strip().lower()
        # STEM
        if any(x in s for x in ['stem', 'scienz', 'matemat', 'fisic', 'chim', 'biolog', 'ingegner', 'tecnolog', 'informatic']):
            return 'STEM'
        # Umanistiche
        if any(x in s for x in ['uman', 'letter', 'lingu', 'artist', 'music', 'filosofi', 'stori', 'giuridic', 'giurispr', 'sociol', 'psicolog', 'pedagog', 'scienze dell\'educazione']):
            return 'Umanistiche'
        return 'Altro'
    
    # Prepara dataframe
    df_a = DF_plot[['GruppoDettaglio']].copy()
    df_a['Area_norm'] = DF[col_area].apply(norm_area)
    df_a['uses_ai'] = DF[col_use].apply(lambda v: str(v).strip().lower() in yes_set if pd.notna(v) else False)
    
    # Filtra per i 4 gruppi principali e solo STEM/Umanistiche
    df_a = df_a[df_a['GruppoDettaglio'].isin(ORDER_4)]
    df_a = df_a[df_a['Area_norm'].isin(['STEM', 'Umanistiche'])]
    
    # Crea tabella di contingenza: Gruppo × Area_norm
    tab_area = (
        df_a.groupby(['GruppoDettaglio','Area_norm'], observed=False)
        .agg(total=('uses_ai','size'), users=('uses_ai','sum'))
        .reset_index()
    )
    tab_area['perc_users_of_area'] = (tab_area['users'] / tab_area['total'] * 100).round(1)
    print(f"   ✓ tab_area creato: {len(tab_area)} righe")
else:
    if not col_area:
        print("   ⚠️  Colonna area disciplinare non trovata in DF")
    elif not col_use:
        print("   ⚠️  col_use non disponibile")
    tab_area = None

print("\n" + "="*80)
print("PREPARAZIONE COMPLETATA")
print("="*80)
print(f"  tab_gen:   {'✓' if tab_gen is not None else '✗'}")
print(f"  tab_area:  {'✓' if tab_area is not None else '✗'}")
print(f"  df_check:  {'✓' if df_check is not None else '✗'}")
print(f"  df_use:    {'✓' if df_use is not None else '✗'}")
print("="*80)

In [ ]:
# === TEST CHI-QUADRATO: GENERE × USO IA ===
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*80)
print("TEST CHI-QUADRATO: Genere × Uso dell'IA")
print("="*80)

# Verifica che tab_gen esista (dalla cella precedente)
if 'tab_gen' not in globals():
    print("ATTENZIONE: tab_gen non trovato. Esegui prima la cella 41 (incrocio genere × uso).")
else:
    # Test chi-quadrato per ogni gruppo separatamente
    results_gender = []
    
    for grp in ORDER_4:
        print(f"\n{'─'*60}")
        print(f"Gruppo: {grp}")
        print(f"{'─'*60}")
        
        # Filtra dati per questo gruppo (solo Maschio/Femmina)
        sub = tab_gen[(tab_gen['GruppoDettaglio']==grp) & 
                      (tab_gen['GenereCat'].isin(['Maschio','Femmina']))].copy()
        
        if len(sub) < 2:
            print("  ⚠️  Dati insufficienti per il test")
            continue
        
        # Crea tabella di contingenza: righe=genere, colonne=[non_users, users]
        contingency = []
        for gender in ['Maschio', 'Femmina']:
            row_data = sub[sub['GenereCat']==gender]
            if not row_data.empty:
                total = int(row_data['total'].iloc[0])
                users = int(row_data['users'].iloc[0])
                non_users = total - users
                contingency.append([non_users, users])
                print(f"  {gender:8s}: {users:3d}/{total:3d} usano IA ({row_data['perc_users_of_gender'].iloc[0]:.1f}%)")
        
        if len(contingency) == 2:
            contingency = np.array(contingency)
            
            # Test chi-quadrato
            try:
                chi2, p_value, dof, expected = chi2_contingency(contingency)
                
                # Se valori attesi < 5, usa Fisher's exact test
                if (expected < 5).any():
                    print("\n  ℹ️  Valori attesi < 5, uso Fisher's exact test")
                    oddsratio, p_value_fisher = fisher_exact(contingency)
                    print(f"  Fisher's exact test p-value: {p_value_fisher:.4f}")
                    sig = "✓ Significativo" if p_value_fisher < 0.05 else "✗ Non significativo"
                    print(f"  {sig} (α=0.05)")
                    results_gender.append({
                        'Gruppo': grp,
                        'Test': 'Fisher',
                        'p-value': p_value_fisher,
                        'Significativo (α=0.05)': p_value_fisher < 0.05
                    })
                else:
                    print(f"\n  χ² = {chi2:.3f}, df = {dof}, p-value = {p_value:.4f}")
                    sig = "✓ Significativo" if p_value < 0.05 else "✗ Non significativo"
                    print(f"  {sig} (α=0.05)")
                    results_gender.append({
                        'Gruppo': grp,
                        'Test': 'Chi-quadrato',
                        'Chi2': chi2,
                        'df': dof,
                        'p-value': p_value,
                        'Significativo (α=0.05)': p_value < 0.05
                    })
            except Exception as e:
                print(f"  ⚠️  Errore nel test: {e}")
    
    # Salva risultati
    if results_gender:
        df_results_gender = pd.DataFrame(results_gender)
        csv_gender_sig = OUT_EXPL / 'significance_gender_use.csv'
        df_results_gender.to_csv(csv_gender_sig, index=False)
        print(f"\n{'='*80}")
        print("RIEPILOGO TEST GENERE × USO")
        print(f"{'='*80}")
        print(df_results_gender.to_string(index=False))
        print(f"\nSalvato: {csv_gender_sig}")


In [ ]:
# === TEST CHI-QUADRATO: AREA DISCIPLINARE × USO IA ===
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*80)
print("TEST CHI-QUADRATO: Area disciplinare × Uso dell'IA")
print("="*80)

# Verifica che tab_area esista
if 'tab_area' not in globals() or tab_area is None:
    print("\n⚠️  ATTENZIONE: tab_area non disponibile.")
    print("   Per eseguire questo test, è necessario prima eseguire la cella 52")
    print("   che calcola Area_norm e crea la tabella di contingenza tab_area.\n")
    print("=" * 80)
elif len(tab_area) == 0:
    print("\n⚠️  ATTENZIONE: tab_area è vuoto. Verifica i dati.\n")
    print("=" * 80)
else:
    # Test chi-quadrato per ogni gruppo separatamente
    results_area = []
    
    for grp in ORDER_4:
        print(f"\n{'─'*60}")
        print(f"Gruppo: {grp}")
        print(f"{'─'*60}")
        
        # Filtra dati per questo gruppo (solo STEM/Umanistiche)
        sub = tab_area[(tab_area['GruppoDettaglio']==grp) & 
                       (tab_area['Area_norm'].isin(['STEM','Umanistiche']))].copy()
        
        if len(sub) < 2:
            print("  ⚠️  Dati insufficienti per il test")
            continue
        
        # Crea tabella di contingenza: righe=area, colonne=[non_users, users]
        contingency = []
        for area in ['STEM', 'Umanistiche']:
            row_data = sub[sub['Area_norm']==area]
            if not row_data.empty:
                total = int(row_data['total'].iloc[0])
                users = int(row_data['users'].iloc[0])
                non_users = total - users
                contingency.append([non_users, users])
                print(f"  {area:12s}: {users:3d}/{total:3d} usano IA ({row_data['perc_users_of_area'].iloc[0]:.1f}%)")
        
        if len(contingency) == 2:
            contingency = np.array(contingency)
            
            # Test chi-quadrato
            try:
                chi2, p_value, dof, expected = chi2_contingency(contingency)
                
                # Se valori attesi < 5, usa Fisher's exact test
                if (expected < 5).any():
                    print("\n  ℹ️  Valori attesi < 5, uso Fisher's exact test")
                    oddsratio, p_value_fisher = fisher_exact(contingency)
                    print(f"  Fisher's exact test p-value: {p_value_fisher:.4f}")
                    sig = "✓ Significativo" if p_value_fisher < 0.05 else "✗ Non significativo"
                    print(f"  {sig} (α=0.05)")
                    results_area.append({
                        'Gruppo': grp,
                        'Test': 'Fisher',
                        'p-value': p_value_fisher,
                        'Significativo (α=0.05)': p_value_fisher < 0.05
                    })
                else:
                    print(f"\n  χ² = {chi2:.3f}, df = {dof}, p-value = {p_value:.4f}")
                    sig = "✓ Significativo" if p_value < 0.05 else "✗ Non significativo"
                    print(f"  {sig} (α=0.05)")
                    results_area.append({
                        'Gruppo': grp,
                        'Test': 'Chi-quadrato',
                        'Chi2': chi2,
                        'df': dof,
                        'p-value': p_value,
                        'Significativo (α=0.05)': p_value < 0.05
                    })
            except Exception as e:
                print(f"  ⚠️  Errore nel test: {e}")
    
    # Salva risultati
    if results_area:
        df_results_area = pd.DataFrame(results_area)
        csv_area_sig = OUT_EXPL / 'significance_area_use.csv'
        df_results_area.to_csv(csv_area_sig, index=False)
        print(f"\n{'='*80}")
        print("RIEPILOGO TEST AREA × USO")
        print(f"{'='*80}")
        print(df_results_area.to_string(index=False))
        print(f"\nSalvato: {csv_area_sig}")


In [ ]:
# === TEST CHI-QUADRATO: DIFFERENZE TRA I 4 GRUPPI ===
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*80)
print("TEST CHI-QUADRATO: Differenze nell'uso dell'IA tra i 4 gruppi")
print("="*80)

# Verifica che df_check esista (dalla cella 41)
if 'df_check' not in globals():
    print("ATTENZIONE: df_check non trovato. Esegui prima la cella 41 (incrocio genere × uso).")
else:
    # Crea tabella di contingenza: righe=gruppo, colonne=[non_users, users]
    contingency_groups = []
    group_labels = []
    
    print("\nDistribuzione uso IA per gruppo:")
    print(f"{'─'*60}")
    
    for grp in ORDER_4:
        grp_data = df_check[df_check['GruppoDettaglio']==grp]
        total = len(grp_data)
        users = grp_data['uses_ai'].sum()
        non_users = total - users
        perc = (users / total * 100) if total > 0 else 0
        
        contingency_groups.append([non_users, users])
        group_labels.append(grp)
        
        print(f"{grp:35s}: {users:3d}/{total:3d} ({perc:.1f}%)")
    
    contingency_groups = np.array(contingency_groups)
    
    # Test chi-quadrato
    try:
        chi2, p_value, dof, expected = chi2_contingency(contingency_groups)
        
        print(f"\n{'─'*60}")
        print("Test chi-quadrato:")
        print(f"  χ² = {chi2:.3f}")
        print(f"  df = {dof}")
        print(f"  p-value = {p_value:.6f}")
        
        if p_value < 0.001:
            print(f"  ✓✓✓ ALTAMENTE significativo (p < 0.001)")
        elif p_value < 0.01:
            print(f"  ✓✓ Molto significativo (p < 0.01)")
        elif p_value < 0.05:
            print(f"  ✓ Significativo (p < 0.05)")
        else:
            print(f"  ✗ Non significativo (p ≥ 0.05)")
        
        # Verifica assunzioni del test
        print(f"\n{'─'*60}")
        print("Verifica assunzioni:")
        min_expected = expected.min()
        print(f"  Minimo valore atteso: {min_expected:.2f}")
        if min_expected < 5:
            print("  ⚠️  ATTENZIONE: alcuni valori attesi < 5, i risultati potrebbero non essere affidabili")
        else:
            print("  ✓ Tutti i valori attesi ≥ 5, assunzioni soddisfatte")
        
        # Salva risultati
        result_groups = {
            'Test': 'Chi-quadrato tra 4 gruppi',
            'Chi2': chi2,
            'df': dof,
            'p-value': p_value,
            'Significativo (α=0.05)': p_value < 0.05,
            'Significativo (α=0.01)': p_value < 0.01,
            'Significativo (α=0.001)': p_value < 0.001,
            'Min_expected': min_expected,
            'Assunzioni_OK': min_expected >= 5
        }
        
        df_result_groups = pd.DataFrame([result_groups])
        csv_groups_sig = OUT_EXPL / 'significance_4groups_use.csv'
        df_result_groups.to_csv(csv_groups_sig, index=False)
        
        print(f"\n{'='*80}")
        print("CONCLUSIONE:")
        if p_value < 0.05:
            print("Le differenze nell'uso dell'IA tra i 4 gruppi SONO statisticamente significative.")
            print("Questo indica che l'appartenenza al gruppo è associata all'uso dell'IA.")
        else:
            print("Le differenze nell'uso dell'IA tra i 4 gruppi NON sono statisticamente significative.")
        print(f"{'='*80}")
        
        print(f"\nSalvato: {csv_groups_sig}")
        
    except Exception as e:
        print(f"⚠️  Errore nel test: {e}")


### 1.6.b Sintesi test Chi-quadrato

### 1.6.c ANOVA sulle ore settimanali

In [ ]:
# === ANOVA: ORE SETTIMANALI USO IA TRA I 4 GRUPPI ===
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*80)
print("ANOVA: Ore settimanali di uso dell'IA tra i 4 gruppi")
print("="*80)

# Verifica che df_use esista (dalla cella ore settimanali)
if 'df_use' not in globals():
    print("ATTENZIONE: df_use non trovato. Esegui prima le celle sull'analisi ore settimanali.")
else:
    # Prepara dati per ANOVA
    groups_data = []
    group_names = []
    
    print("\nStatistiche descrittive per gruppo:")
    print(f"{'─'*80}")
    print(f"{'Gruppo':<35s} {'N':>6s} {'Media':>8s} {'Mediana':>8s} {'SD':>8s} {'Min':>6s} {'Max':>6s}")
    print(f"{'─'*80}")
    
    for grp in ORDER_4:
        grp_data = df_use[df_use['GruppoDettaglio']==grp]['OreSettimanali'].dropna()
        if len(grp_data) > 0:
            groups_data.append(grp_data.values)
            group_names.append(grp)
            
            print(f"{grp:<35s} {len(grp_data):>6d} {grp_data.mean():>8.2f} {grp_data.median():>8.2f} "
                  f"{grp_data.std():>8.2f} {grp_data.min():>6.1f} {grp_data.max():>6.1f}")
    
    # Esegui ANOVA one-way
    if len(groups_data) >= 2:
        print(f"\n{'─'*80}")
        print("ANOVA One-Way:")
        print(f"{'─'*80}")
        
        f_stat, p_value = stats.f_oneway(*groups_data)
        
        print(f"  F-statistic = {f_stat:.4f}")
        print(f"  p-value = {p_value:.6f}")
        
        if p_value < 0.001:
            print(f"  ✓✓✓ ALTAMENTE significativo (p < 0.001)")
        elif p_value < 0.01:
            print(f"  ✓✓ Molto significativo (p < 0.01)")
        elif p_value < 0.05:
            print(f"  ✓ Significativo (p < 0.05)")
        else:
            print(f"  ✗ Non significativo (p ≥ 0.05)")
        
        # Verifica assunzioni
        print(f"\n{'─'*80}")
        print("Verifica assunzioni ANOVA:")
        print(f"{'─'*80}")
        
        # Test di Levene per omogeneità delle varianze
        levene_stat, levene_p = stats.levene(*groups_data)
        print(f"  Test di Levene (omogeneità varianze):")
        print(f"    Statistica = {levene_stat:.4f}, p-value = {levene_p:.4f}")
        if levene_p >= 0.05:
            print(f"    ✓ Varianze omogenee (assunzione soddisfatta)")
        else:
            print(f"    ⚠️  Varianze NON omogenee (considera Welch's ANOVA)")
        
        # Test di normalità (Shapiro-Wilk) per ogni gruppo
        print(f"\n  Test di normalità (Shapiro-Wilk) per gruppo:")
        all_normal = True
        for i, (grp_name, data) in enumerate(zip(group_names, groups_data)):
            if len(data) >= 3:
                shapiro_stat, shapiro_p = stats.shapiro(data)
                normal = shapiro_p >= 0.05
                all_normal = all_normal and normal
                status = "✓" if normal else "⚠️"
                print(f"    {status} {grp_name:<35s}: p = {shapiro_p:.4f}")
        
        if not all_normal:
            print(f"\n  ⚠️  Alcuni gruppi non sono normali → considera Kruskal-Wallis (non parametrico)")
        
        # Salva risultati
        anova_result = {
            'Test': 'ANOVA One-Way',
            'F-statistic': f_stat,
            'p-value': p_value,
            'Significativo (α=0.05)': p_value < 0.05,
            'Significativo (α=0.01)': p_value < 0.01,
            'Significativo (α=0.001)': p_value < 0.001,
            'Levene_statistic': levene_stat,
            'Levene_p-value': levene_p,
            'Varianze_omogenee': levene_p >= 0.05
        }
        
        df_anova = pd.DataFrame([anova_result])
        csv_anova = OUT_EXPL / 'anova_hours_4groups.csv'
        df_anova.to_csv(csv_anova, index=False)
        
        print(f"\n{'='*80}")
        if p_value < 0.05:
            print("CONCLUSIONE: Le medie delle ore settimanali di uso IA differiscono")
            print("significativamente tra i 4 gruppi.")
            print("→ Prosegui con test post-hoc per identificare quali coppie differiscono.")
        else:
            print("CONCLUSIONE: Non ci sono differenze significative nelle medie delle ore")
            print("settimanali di uso IA tra i 4 gruppi.")
        print(f"{'='*80}")
        
        print(f"\nSalvato: {csv_anova}")
    else:
        print("⚠️  Dati insufficienti per ANOVA (servono almeno 2 gruppi)")


In [ ]:
# === TEST POST-HOC: Tukey HSD per confronti multipli ===
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
from itertools import combinations

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

# Verifica che l'ANOVA sia stata eseguita e sia significativa
if 'p_value' not in globals():
    print("⚠️  Esegui prima la cella ANOVA sopra.")
elif p_value >= 0.05:
    print("ℹ️  ANOVA non significativo (p ≥ 0.05), test post-hoc non necessari.")
    print("   Non ci sono differenze significative tra i gruppi.")
else:
    print("="*80)
    print("TEST POST-HOC: Confronti a coppie (Tukey HSD)")
    print("="*80)
    print("\nPoiché l'ANOVA è significativo, identifichiamo quali coppie differiscono.\n")
    
    # Implementazione manuale Tukey HSD (scipy non ha statsmodels di default)
    # Alternativa: usa test t con correzione Bonferroni
    
    # Prepara tutti i dati con etichette gruppo
    all_data = []
    all_labels = []
    for grp in ORDER_4:
        grp_data = df_use[df_use['GruppoDettaglio']==grp]['OreSettimanali'].dropna()
        all_data.extend(grp_data.values)
        all_labels.extend([grp] * len(grp_data))
    
    df_for_posthoc = pd.DataFrame({'Gruppo': all_labels, 'Ore': all_data})
    
    # Confronti a coppie con test t (correzione Bonferroni)
    n_comparisons = len(list(combinations(ORDER_4, 2)))
    alpha_bonferroni = 0.05 / n_comparisons
    
    print(f"Numero di confronti: {n_comparisons}")
    print(f"Soglia Bonferroni: α = {alpha_bonferroni:.4f} (0.05/{n_comparisons})")
    print(f"\n{'─'*90}")
    print(f"{'Gruppo 1':<35s} {'Gruppo 2':<35s} {'Diff Media':>10s} {'p-value':>10s} {'Sig?':>5s}")
    print(f"{'─'*90}")
    
    posthoc_results = []
    
    for grp1, grp2 in combinations(ORDER_4, 2):
        data1 = df_use[df_use['GruppoDettaglio']==grp1]['OreSettimanali'].dropna()
        data2 = df_use[df_use['GruppoDettaglio']==grp2]['OreSettimanali'].dropna()
        
        if len(data1) > 0 and len(data2) > 0:
            # Test t indipendente
            t_stat, p_val = stats.ttest_ind(data1, data2)
            diff_mean = data1.mean() - data2.mean()
            
            # Significativo con Bonferroni?
            sig = "✓" if p_val < alpha_bonferroni else "✗"
            
            print(f"{grp1:<35s} {grp2:<35s} {diff_mean:>10.2f} {p_val:>10.6f} {sig:>5s}")
            
            posthoc_results.append({
                'Gruppo_1': grp1,
                'Gruppo_2': grp2,
                'Media_Gruppo_1': data1.mean(),
                'Media_Gruppo_2': data2.mean(),
                'Differenza_medie': diff_mean,
                't-statistic': t_stat,
                'p-value': p_val,
                'p-value_Bonferroni': alpha_bonferroni,
                'Significativo_Bonferroni': p_val < alpha_bonferroni
            })
    
    print(f"{'─'*90}")
    
    # Salva risultati
    df_posthoc = pd.DataFrame(posthoc_results)
    csv_posthoc = OUT_EXPL / 'posthoc_tukey_hours_4groups.csv'
    df_posthoc.to_csv(csv_posthoc, index=False)
    
    # Riepilogo
    sig_comparisons = df_posthoc[df_posthoc['Significativo_Bonferroni']==True]
    
    print(f"\n{'='*80}")
    print("RIEPILOGO CONFRONTI SIGNIFICATIVI:")
    print(f"{'='*80}")
    
    if len(sig_comparisons) > 0:
        print(f"\nTrovate {len(sig_comparisons)} coppie con differenze significative:\n")
        for _, row in sig_comparisons.iterrows():
            print(f"  • {row['Gruppo_1']}")
            print(f"    vs {row['Gruppo_2']}")
            print(f"    Differenza: {row['Differenza_medie']:.2f} ore/settimana (p={row['p-value']:.6f})")
            print()
    else:
        print("\nNessuna coppia di gruppi differisce significativamente dopo")
        print("la correzione per confronti multipli (Bonferroni).")
    
    print(f"Salvato: {csv_posthoc}")


### 1.6.d Riepilogo ANOVA

In [ ]:
# === USO IA incrociato con GENERE ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
import shutil

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()
ASSETS = (Path.cwd()/"../assets/figures").resolve()
OUT_EXPL.mkdir(parents=True, exist_ok=True)
ASSETS.mkdir(parents=True, exist_ok=True)

# Usa col_daily_stu se disponibile (definita in celle precedenti)
if 'col_daily_stu' in globals() and col_daily_stu is not None:
    col_use = col_daily_stu
    print(f"Usiamo la colonna uso quotidiano: {col_use}")
elif 'col_daily_doc' in globals() and col_daily_doc is not None:
    col_use = col_daily_doc
    print(f"Usiamo la colonna uso quotidiano (docenti): {col_use}")
else:
    print("ATTENZIONE: col_daily_stu/col_daily_doc non trovate. Esegui prima le celle che le definiscono.")
    col_use = None

if col_use is not None:
    # Usa GenereCat già calcolata (dalla cella precedente del genere)
    gen_col = 'GenereCat'
    print('Usando GenereCat già calcolata.')

    # Usa yes_set già definito nel kernel (o fallback)
    if 'yes_set' not in globals():
        yes_set = set(['si','sì','yes','y','1','true'])
    
    # Costruisce tabella: per GruppoDettaglio e GenereCat -> conteggio utenti e % che usano IA
    df_check = DF_plot.copy()
    
    # Assicura le colonne usate
    if col_use not in df_check.columns:
        df_check[col_use] = DF[col_use]
    
    # Prendi GenereCat da base_gf (già calcolata)
    if 'base_gf' in globals() and gen_col in base_gf.columns:
        df_check[gen_col] = base_gf[gen_col]
    else:
        print("ATTENZIONE: base_gf o GenereCat non trovata. Esegui prima la cella che calcola il genere.")
        df_check[gen_col] = 'Non risponde'

    # Normalizza uso come boolean
    def is_yes(v):
        if pd.isna(v):
            return False
        return str(v).strip().lower() in yes_set

    df_check['uses_ai'] = df_check[col_use].apply(is_yes)

    # Filtra ai 4 gruppi
    groups = ORDER_4 if 'ORDER_4' in globals() else sorted(df_check['GruppoDettaglio'].unique())
    df_check['GruppoDettaglio'] = pd.Categorical(df_check['GruppoDettaglio'], categories=groups, ordered=True)

    # Tabella per genere
    tab_gen = (
        df_check.groupby(['GruppoDettaglio','GenereCat'], observed=False)
        .agg(total=('uses_ai','size'), users=('uses_ai','sum'))
        .reset_index()
    )
    tab_gen['perc_users_of_gender'] = (tab_gen['users'] / tab_gen['total'] * 100).round(1)

    csv_gen = OUT_EXPL / 'cross_gender_use_4groups.csv'
    tab_gen.to_csv(csv_gen, index=False)
    print('Salvato CSV incrocio Genere-Use:', csv_gen)
    display(tab_gen)

    # Grafico percentuale di uso tra Maschio/Femmina per ogni gruppo (IT)
    fig, ax = plt.subplots(figsize=(12,5))
    x = np.arange(len(groups))
    width = 0.35
    colors = {'Femmina':'#ff69b4','Maschio':'#4169e1'}

    for i, g in enumerate(groups):
        sub = tab_gen[tab_gen['GruppoDettaglio']==g]
        for j, gender in enumerate(['Femmina','Maschio']):
            row = sub[sub['GenereCat']==gender]
            if row.empty:
                val = 0.0
            else:
                val = float(row['perc_users_of_gender'].iloc[0])
            ax.bar(i + (j-0.5)*width, val, width, color=colors[gender], label=gender if i==0 else None)
            ax.text(i + (j-0.5)*width, val + 1, f"{val:.0f}%", ha='center', va='bottom', fontsize=9)

    ax.set_xticks(x)
    ax.set_xticklabels(groups, rotation=0)
    ax.set_ylabel('Percentuale utenti IA (tra il genere nel gruppo)')
    ax.set_title('Uso dell\'IA per Genere nei 4 gruppi')
    ax.legend(ncols=2)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    fp_it = OUT_EXPL / 'cross_gender_use_4groups_it.png'
    fig.savefig(fp_it, dpi=300, bbox_inches='tight')
    plt.show()
    print(f'Salvato: {fp_it}')

    # Versione EN
    fig, ax = plt.subplots(figsize=(12,5))
    for i, g in enumerate(groups):
        sub = tab_gen[tab_gen['GruppoDettaglio']==g]
        for j, gender in enumerate(['Femmina','Maschio']):
            row = sub[sub['GenereCat']==gender]
            val = float(row['perc_users_of_gender'].iloc[0]) if not row.empty else 0.0
            ax.bar(i + (j-0.5)*width, val, width, color=colors[gender], label=('Female' if gender=='Femmina' else 'Male') if i==0 else None)
            ax.text(i + (j-0.5)*width, val + 1, f"{val:.0f}%", ha='center', va='bottom', fontsize=9)
    ax.set_xticks(x)
    ax.set_xticklabels([map_it_en[g] if g in map_it_en else g for g in groups], rotation=0)
    ax.set_ylabel('Percent of AI users (within gender in group)')
    ax.set_title('AI use by Gender across 4 groups')
    ax.legend(ncols=2)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    fp_en = OUT_EXPL / 'cross_gender_use_4groups_en.png'
    fig.savefig(fp_en, dpi=300, bbox_inches='tight')
    plt.show()
    print(f'Salvato: {fp_en}')

    # copia in assets
    try:
        shutil.copyfile(fp_en, ASSETS / 'cross_gender_use_4groups_en.png')
        shutil.copyfile(fp_it, ASSETS / 'cross_gender_use_4groups_it.png')
        print('Copiati grafici in assets.')
    except Exception as e:
        print('Avviso copia assets:', e)


In [ ]:
# === USO IA incrociato con AREA (STEM vs Umanistiche) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
import shutil

# Verifica che tab_area sia già stata creata dalla cella di preparazione
if 'tab_area' not in globals() or tab_area is None:
    print('ATTENZIONE: tab_area non trovata. Esegui prima la cella 42 (preparazione dati).')
else:
    print(f"Usando tab_area già creato: {len(tab_area)} righe")
    
    # Salva CSV
    csv_area = OUT_EXPL / 'cross_area_use_4groups.csv'
    tab_area.to_csv(csv_area, index=False)
    print('Salvato CSV incrocio Area-Use:', csv_area)
    display(tab_area)

    # Grafico IT
    groups = ORDER_4 if 'ORDER_4' in globals() else sorted(tab_area['GruppoDettaglio'].unique())
    fig, ax = plt.subplots(figsize=(12,5))
    x = np.arange(len(groups))
    width = 0.35
    colors = {'STEM':'#2ecc71','Umanistiche':'#e74c3c'}
    for i, g in enumerate(groups):
        sub = tab_area[tab_area['GruppoDettaglio']==g]
        for j, area in enumerate(['STEM','Umanistiche']):
            row = sub[sub['Area_norm']==area]
            val = float(row['perc_users_of_area'].iloc[0]) if not row.empty else 0.0
            ax.bar(i + (j-0.5)*width, val, width, color=colors[area], label=area if i==0 else None)
            ax.text(i + (j-0.5)*width, val + 1, f"{val:.0f}%", ha='center', va='bottom', fontsize=9)
    ax.set_xticks(x)
    ax.set_xticklabels(groups, rotation=0)
    ax.set_ylabel('Percentuale utenti IA (tra area nel gruppo)')
    ax.set_title('Uso dell\'IA per Area disciplinare (STEM/Umanistiche) nei 4 gruppi')
    ax.legend(ncols=2)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    fp_it = OUT_EXPL / 'cross_area_use_4groups_it.png'
    fig.savefig(fp_it, dpi=300, bbox_inches='tight')
    plt.show()
    shutil.copy(fp_it, ASSETS / fp_it.name)
    print('Salvato grafico IT:', fp_it)

    # Grafico EN
    fig2, ax2 = plt.subplots(figsize=(12,5))
    for i, g in enumerate(groups):
        sub = tab_area[tab_area['GruppoDettaglio']==g]
        for j, area in enumerate(['STEM','Umanistiche']):
            row = sub[sub['Area_norm']==area]
            val = float(row['perc_users_of_area'].iloc[0]) if not row.empty else 0.0
            ax2.bar(i + (j-0.5)*width, val, width, color=colors[area], label=area if i==0 else None)
            ax2.text(i + (j-0.5)*width, val + 1, f"{val:.0f}%", ha='center', va='bottom', fontsize=9)
    ax2.set_xticks(x)
    ax2.set_xticklabels(groups, rotation=0)
    ax2.set_ylabel('Percentage of AI users (within area in group)')
    ax2.set_title('AI Use by Disciplinary Area (STEM/Humanities) across 4 groups')
    ax2.legend(ncols=2)
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    fp_en = OUT_EXPL / 'cross_area_use_4groups_en.png'
    fig2.savefig(fp_en, dpi=300, bbox_inches='tight')
    plt.show()
    shutil.copy(fp_en, ASSETS / fp_en.name)
    print('Salvato grafico EN:', fp_en)


## 🔬 Variazioni DENTRO i 4 gruppi: Genere e Area

Verifichiamo se **all'interno di ciascun gruppo** ci sono differenze significative nelle ore settimanali di uso IA per:
1. **Genere** (Maschi vs Femmine) → t-test indipendente
2. **Area disciplinare** (STEM vs Umanistiche) → t-test indipendente

In [ ]:
# === VARIAZIONI GENERE DENTRO OGNI GRUPPO (ore settimanali) ===
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*90)
print("DIFFERENZE DI GENERE NELLE ORE SETTIMANALI - PER OGNI GRUPPO")
print("="*90)

if 'df_use' not in globals():
    print("⚠️  df_use non trovato. Esegui prima le celle sull'analisi ore settimanali.")
elif 'base_gf' not in globals() or 'GenereCat' not in base_gf.columns:
    print("⚠️  base_gf o GenereCat non trovata. Esegui prima la cella che calcola il genere.")
else:
    # Aggiungi GenereCat a df_use tramite merge
    df_use_with_gender = df_use.merge(
        base_gf[['GenereCat']],
        left_index=True,
        right_index=True,
        how='left'
    )
    
    results_gender_hours = []
    
    print(f"\n{'─'*90}")
    print(f"{'Gruppo':<35s} {'N_M':>6s} {'Media_M':>9s} {'N_F':>6s} {'Media_F':>9s} {'Diff':>8s} {'p-value':>10s} {'Sig?':>5s}")
    print(f"{'─'*90}")
    
    for grp in ORDER_4:
        grp_data = df_use_with_gender[df_use_with_gender['GruppoDettaglio']==grp].copy()
        
        # Filtra per genere
        m_data = grp_data[grp_data['GenereCat']=='Maschio']['OreSettimanali'].dropna()
        f_data = grp_data[grp_data['GenereCat']=='Femmina']['OreSettimanali'].dropna()
        
        if len(m_data) >= 2 and len(f_data) >= 2:
            # T-test indipendente
            t_stat, p_val = stats.ttest_ind(m_data, f_data)
            diff = m_data.mean() - f_data.mean()
            
            # Significativo?
            if p_val < 0.001:
                sig = "✓✓✓"
            elif p_val < 0.01:
                sig = "✓✓"
            elif p_val < 0.05:
                sig = "✓"
            else:
                sig = "✗"
            
            print(f"{grp:<35s} {len(m_data):>6d} {m_data.mean():>9.2f} {len(f_data):>6d} {f_data.mean():>9.2f} "
                  f"{diff:>8.2f} {p_val:>10.6f} {sig:>5s}")
            
            results_gender_hours.append({
                'Gruppo': grp,
                'N_Maschi': len(m_data),
                'Media_Maschi': m_data.mean(),
                'SD_Maschi': m_data.std(),
                'N_Femmine': len(f_data),
                'Media_Femmine': f_data.mean(),
                'SD_Femmine': f_data.std(),
                'Differenza_medie': diff,
                't-statistic': t_stat,
                'p-value': p_val,
                'Significativo_0.05': p_val < 0.05,
                'Significativo_0.01': p_val < 0.01,
                'Significativo_0.001': p_val < 0.001
            })
        else:
            print(f"{grp:<35s} {'Dati insufficienti'}")
    
    print(f"{'─'*90}")
    
    # Salva risultati
    if results_gender_hours:
        df_gender_hours = pd.DataFrame(results_gender_hours)
        csv_gender_hours = OUT_EXPL / 'within_group_gender_hours.csv'
        df_gender_hours.to_csv(csv_gender_hours, index=False)
        
        # Riepilogo
        sig_groups = df_gender_hours[df_gender_hours['Significativo_0.05']==True]
        
        print(f"\n{'='*90}")
        print("RIEPILOGO DIFFERENZE DI GENERE:")
        print(f"{'='*90}")
        
        if len(sig_groups) > 0:
            print(f"\n✓ Trovate differenze significative in {len(sig_groups)} gruppi:\n")
            for _, row in sig_groups.iterrows():
                print(f"  • {row['Gruppo']}")
                print(f"    Maschi: {row['Media_Maschi']:.2f} ore/sett (N={row['N_Maschi']})")
                print(f"    Femmine: {row['Media_Femmine']:.2f} ore/sett (N={row['N_Femmine']})")
                print(f"    Differenza: {row['Differenza_medie']:+.2f} ore/sett (p={row['p-value']:.4f})")
                print()
        else:
            print("\n✗ Nessuna differenza significativa di genere nelle ore settimanali")
            print("  all'interno dei singoli gruppi.")
        
        print(f"Salvato: {csv_gender_hours}")
    else:
        print("\n⚠️  Nessun risultato disponibile.")


In [ ]:
# === VARIAZIONI AREA DISCIPLINARE DENTRO OGNI GRUPPO (ore settimanali) ===
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*90)
print("DIFFERENZE DI AREA DISCIPLINARE NELLE ORE SETTIMANALI - PER OGNI GRUPPO")
print("="*90)

if 'df_use' not in globals():
    print("⚠️  df_use non trovato. Esegui prima la cella 42 (preparazione dati).")
else:
    # Cerca la colonna area in DF
    col_area = None
    for c in DF.columns:
        c_lower = str(c).lower()
        if ('settore' in c_lower and 'scientific' in c_lower) or ('classe' in c_lower and 'concorso' in c_lower):
            col_area = c
            break
    
    if col_area is None:
        print("⚠️  Colonna area disciplinare non trovata. Esegui prima la cella 42.")
    else:
        # Funzione di normalizzazione (stessa della cella 42)
        def normalize_area(val):
            if pd.isna(val):
                return 'Altro'
            s = str(val).lower()
            stem_kw = ['scienz','matemat','fisic','chim','biolog','ingegner','tecnolog','informatic']
            hum_kw = ['uman','letter','lingu','artist','music','filosofi','stori','giuridic','sociol','psicolog','pedagog']
            if any(k in s for k in stem_kw):
                return 'STEM'
            elif any(k in s for k in hum_kw):
                return 'Umanistiche'
            else:
                return 'Altro'
        
        # Aggiungi Area_norm a df_use
        df_use_with_area = df_use.copy()
        df_use_with_area['Area_norm'] = DF.loc[df_use.index, col_area].apply(normalize_area)
        # Aggiungi Area_norm a df_use
        df_use_with_area = df_use.copy()
        df_use_with_area['Area_norm'] = DF.loc[df_use.index, col_area].apply(normalize_area)
        
        results_area_hours = []
        
        print(f"\n{'─'*90}")
        print(f"{'Gruppo':<35s} {'N_STEM':>8s} {'Media_STEM':>11s} {'N_HUM':>8s} {'Media_HUM':>11s} {'Diff':>8s} {'p-value':>10s} {'Sig?':>5s}")
        print(f"{'─'*90}")
    
    for grp in ORDER_4:
        grp_data = df_use_with_area[df_use_with_area['GruppoDettaglio']==grp].copy()
        
        # Filtra per area disciplinare
        stem_data = grp_data[grp_data['Area_norm']=='STEM']['OreSettimanali'].dropna()
        hum_data = grp_data[grp_data['Area_norm']=='Umanistiche']['OreSettimanali'].dropna()
        
        if len(stem_data) >= 2 and len(hum_data) >= 2:
            # T-test indipendente
            t_stat, p_val = stats.ttest_ind(stem_data, hum_data)
            diff = stem_data.mean() - hum_data.mean()
            
            # Significativo?
            if p_val < 0.001:
                sig = "✓✓✓"
            elif p_val < 0.01:
                sig = "✓✓"
            elif p_val < 0.05:
                sig = "✓"
            else:
                sig = "✗"
            
            print(f"{grp:<35s} {len(stem_data):>8d} {stem_data.mean():>11.2f} {len(hum_data):>8d} {hum_data.mean():>11.2f} "
                  f"{diff:>8.2f} {p_val:>10.6f} {sig:>5s}")
            
            results_area_hours.append({
                'Gruppo': grp,
                'N_STEM': len(stem_data),
                'Media_STEM': stem_data.mean(),
                'SD_STEM': stem_data.std(),
                'N_Umanistiche': len(hum_data),
                'Media_Umanistiche': hum_data.mean(),
                'SD_Umanistiche': hum_data.std(),
                'Differenza_medie': diff,
                't-statistic': t_stat,
                'p-value': p_val,
                'Significativo_0.05': p_val < 0.05,
                'Significativo_0.01': p_val < 0.01,
                'Significativo_0.001': p_val < 0.001
            })
        else:
            status = f"N_STEM={len(stem_data)}, N_HUM={len(hum_data)}"
            print(f"{grp:<35s} {status}")
        
        print(f"{'─'*90}")
        
        # Salva risultati
        if results_area_hours:
            df_area_hours = pd.DataFrame(results_area_hours)
            csv_area_hours = OUT_EXPL / 'within_group_area_hours.csv'
            df_area_hours.to_csv(csv_area_hours, index=False)
            
            # Riepilogo
            sig_groups = df_area_hours[df_area_hours['Significativo_0.05']==True]
            
            print(f"\n{'='*90}")
            print("RIEPILOGO DIFFERENZE DI AREA DISCIPLINARE:")
            print(f"{'='*90}")
            
            if len(sig_groups) > 0:
                print(f"\n✓ Trovate differenze significative in {len(sig_groups)} gruppi:\n")
                for _, row in sig_groups.iterrows():
                    print(f"  • {row['Gruppo']}")
                    print(f"    STEM: {row['Media_STEM']:.2f} ore/sett (N={row['N_STEM']})")
                    print(f"    Umanistiche: {row['Media_Umanistiche']:.2f} ore/sett (N={row['N_Umanistiche']})")
                    print(f"    Differenza: {row['Differenza_medie']:+.2f} ore/sett (p={row['p-value']:.4f})")
                    print()
            else:
                print("\n✗ Nessuna differenza significativa di area disciplinare nelle ore settimanali")
                print("  all'interno dei singoli gruppi.")
            
            print(f"Salvato: {csv_area_hours}")
        else:
            print("\n⚠️  Nessun risultato disponibile.")


## 📊 RIEPILOGO VARIAZIONI DENTRO I 4 GRUPPI

### 🎯 Obiettivo
Verificare se **all'interno di ciascun gruppo** ci sono differenze significative nelle ore settimanali di uso dell'IA tra:
- Maschi vs Femmine
- STEM vs Umanistiche

---

### 👥 DIFFERENZE DI GENERE (Maschi vs Femmine) - Ore Settimanali

| Gruppo | N Maschi | Media Maschi | N Femmine | Media Femmine | Differenza | p-value | Significativo? |
|--------|----------|--------------|-----------|---------------|------------|---------|----------------|
| **Studenti - secondaria** | 16 | 4.88 ore/sett | 80 | 6.72 ore/sett | -1.84 | 0.554 | ✗ |
| **Studenti - universitari** | 30 | 2.40 ore/sett | 141 | 2.55 ore/sett | -0.15 | 0.804 | ✗ |
| **Insegnanti - non in servizio** | 24 | 2.42 ore/sett | 73 | 1.36 ore/sett | +1.06 | **0.045** | ✓ |
| **Insegnanti - in servizio** | 70 | 2.03 ore/sett | 279 | 1.78 ore/sett | +0.24 | 0.639 | ✗ |

#### 💡 Interpretazione Genere:
- **1 gruppo significativo**: Insegnanti non in servizio → Maschi usano ~1 ora/sett in più (p = 0.045)
- **3 gruppi NON significativi**: Nessuna differenza di genere sostanziale nelle ore di uso
- **Pattern generale**: Le differenze di genere nelle **ore settimanali** sono minime

---

### 📚 DIFFERENZE DI AREA DISCIPLINARE (STEM vs Umanistiche) - Ore Settimanali

| Gruppo | N STEM | Media STEM | N Umanistiche | Media Umanistiche | Differenza | p-value | Significativo? |
|--------|--------|------------|---------------|-------------------|------------|---------|----------------|
| **Studenti - secondaria** | - | - | - | - | - | - | Dati insufficienti |
| **Studenti - universitari** | 9 | 4.22 ore/sett | 117 | 2.74 ore/sett | +1.48 | 0.165 | ✗ |
| **Insegnanti - non in servizio** | 33 | 1.58 ore/sett | 65 | 2.31 ore/sett | -0.73 | 0.490 | ✗ |
| **Insegnanti - in servizio** | 151 | 1.68 ore/sett | 198 | 1.92 ore/sett | -0.24 | 0.560 | ✗ |

#### 💡 Interpretazione Area:
- **Nessun gruppo significativo**: Le differenze tra STEM e Umanistiche non sono statisticamente significative
- **Tendenza negli universitari**: STEM usa ~1.5 ore/sett in più, ma p = 0.165 (non significativo, forse per N piccolo STEM=9)
- **Pattern generale**: L'area disciplinare NON sembra influenzare le ore di uso IA

---

### 🔍 CONFRONTO CON ANALISI PRECEDENTI

#### Uso Sì/No (Chi-square):
- **Genere**: Nessun gruppo significativo nelle proporzioni (p > 0.05)
- **Area**: Solo "insegnanti - non in servizio" significativo (p = 0.046)

#### Ore Settimanali (t-test dentro gruppi):
- **Genere**: Solo "insegnanti - non in servizio" significativo (p = 0.045) ✓ **CONVERGENZA**
- **Area**: Nessun gruppo significativo

---

### 💡 CONCLUSIONI GENERALI

1. **Differenze di GENERE**:
   - Nelle **proporzioni** (Sì/No): non significative in nessun gruppo
   - Nelle **ore settimanali**: significativa solo per insegnanti non in servizio (maschi usano di più)
   - → **Genere ha effetto minimo** sull'uso dell'IA

2. **Differenze di AREA**:
   - Nelle **proporzioni** (Sì/No): significativa solo per insegnanti non in servizio (p = 0.046)
   - Nelle **ore settimanali**: non significativa in nessun gruppo
   - → **Area disciplinare ha effetto debole/nullo** sull'uso dell'IA

3. **Differenze TRA GRUPPI** (studenti vs insegnanti):
   - Nelle **proporzioni** (Sì/No): ALTAMENTE significative (χ² p < 0.001)
   - Nelle **ore settimanali**: ALTAMENTE significative (ANOVA p < 0.001)
   - → **Il tipo di gruppo (ruolo + livello) è il fattore PRINCIPALE**

4. **Pattern chiaro**:
   - **Studenti secondaria** → USO ELEVATO (6.4 ore/sett, 79% usa IA)
   - **Altri gruppi** → USO MODERATO (1.8-2.5 ore/sett, 50-60% usa IA)
   - Genere e area influiscono poco rispetto al ruolo/livello

---

### 📁 File salvati:
- `within_group_gender_hours.csv` - Differenze di genere per gruppo
- `within_group_area_hours.csv` - Differenze di area per gruppo

## 🎓 vs 👨‍🏫 Differenze Studio vs Didattica

Confrontiamo l'uso dell'IA tra:
- **STUDIO** (studenti secondaria + universitari)
- **DIDATTICA** (insegnanti non in servizio + in servizio)

Test statistici:
1. **Proporzioni** (Sì/No) → Chi-square
2. **Ore settimanali** (media) → t-test indipendente

In [ ]:
# === DIFFERENZE STUDIO VS DIDATTICA: Proporzioni (Sì/No) ===
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*90)
print("DIFFERENZE STUDIO vs DIDATTICA - PROPORZIONI USO IA (Sì/No)")
print("="*90)

# Verifica che i dati esistano
if 'use' not in globals():
    print("⚠️  DataFrame 'use' non trovato. Esegui prima la cella sull'uso IA studio/didattica.")
else:
    # Crea macro-gruppi
    use_macro = use.copy()
    use_macro['MacroGruppo'] = use_macro['GruppoDettaglio'].apply(
        lambda x: 'STUDIO' if x in ['studenti - secondaria', 'studenti - universitari'] 
        else 'DIDATTICA'
    )
    
    # Tabella di contingenza
    contingency = pd.crosstab(
        use_macro['MacroGruppo'],
        use_macro['UsoAI_studio_didattica']
    )
    
    print("\n📊 TABELLA DI CONTINGENZA:\n")
    print(contingency)
    print()
    
    # Calcola percentuali
    totals = contingency.sum(axis=1)
    percentages = contingency.div(totals, axis=0) * 100
    
    print("\n📊 PERCENTUALI PER MACRO-GRUPPO:\n")
    print(percentages.round(1))
    print()
    
    # Statistiche descrittive
    print("\n📊 STATISTICHE DESCRITTIVE:\n")
    print(f"{'Macro-Gruppo':<15s} {'N totale':>10s} {'N Sì':>10s} {'N No':>10s} {'% Sì':>10s} {'% No':>10s}")
    print(f"{'─'*70}")
    
    for gruppo in ['STUDIO', 'DIDATTICA']:
        n_tot = totals[gruppo]
        n_si = contingency.loc[gruppo, 'Sì']
        n_no = contingency.loc[gruppo, 'No']
        perc_si = percentages.loc[gruppo, 'Sì']
        perc_no = percentages.loc[gruppo, 'No']
        
        print(f"{gruppo:<15s} {n_tot:>10d} {n_si:>10d} {n_no:>10d} {perc_si:>9.1f}% {perc_no:>9.1f}%")
    
    # Chi-square test
    print(f"\n{'─'*90}")
    print("TEST CHI-SQUARE:")
    print(f"{'─'*90}")
    
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency.values)
    
    print(f"  χ² = {chi2:.4f}")
    print(f"  gradi di libertà = {dof}")
    print(f"  p-value = {p_value:.6f}")
    print(f"  Valori attesi minimi = {expected.min():.2f}")
    
    # Verifica assunzioni
    if expected.min() >= 5:
        print(f"  ✓ Assunzioni soddisfatte (tutti i valori attesi ≥ 5)")
    else:
        print(f"  ⚠️  Assunzione violata (alcuni valori attesi < 5)")
        print(f"     Considera Fisher's exact test")
    
    # Significatività
    print()
    if p_value < 0.001:
        print(f"  ✓✓✓ ALTAMENTE significativo (p < 0.001)")
        sig_label = "Altamente significativo"
    elif p_value < 0.01:
        print(f"  ✓✓ Molto significativo (p < 0.01)")
        sig_label = "Molto significativo"
    elif p_value < 0.05:
        print(f"  ✓ Significativo (p < 0.05)")
        sig_label = "Significativo"
    else:
        print(f"  ✗ Non significativo (p ≥ 0.05)")
        sig_label = "Non significativo"
    
    # Effect size (Cramér's V)
    n = contingency.sum().sum()
    cramers_v = np.sqrt(chi2 / n)
    print(f"\n  Effect size (Cramér's V) = {cramers_v:.4f}")
    if cramers_v < 0.1:
        print(f"  → Effetto PICCOLO")
    elif cramers_v < 0.3:
        print(f"  → Effetto MEDIO")
    else:
        print(f"  → Effetto GRANDE")
    
    # Salva risultati
    result_proportions = {
        'Confronto': 'STUDIO vs DIDATTICA',
        'N_STUDIO': totals['STUDIO'],
        'N_DIDATTICA': totals['DIDATTICA'],
        'Perc_Si_STUDIO': percentages.loc['STUDIO', 'Sì'],
        'Perc_Si_DIDATTICA': percentages.loc['DIDATTICA', 'Sì'],
        'Differenza_percentuali': percentages.loc['STUDIO', 'Sì'] - percentages.loc['DIDATTICA', 'Sì'],
        'chi2': chi2,
        'dof': dof,
        'p-value': p_value,
        'Cramers_V': cramers_v,
        'Significativo_0.05': p_value < 0.05,
        'Significativo_0.01': p_value < 0.01,
        'Significativo_0.001': p_value < 0.001,
        'Significatività': sig_label
    }
    
    df_result_prop = pd.DataFrame([result_proportions])
    csv_prop = OUT_EXPL / 'studio_vs_didattica_proportions.csv'
    df_result_prop.to_csv(csv_prop, index=False)
    
    # Riepilogo finale
    print(f"\n{'='*90}")
    print("CONCLUSIONE:")
    print(f"{'='*90}")
    
    diff_perc = percentages.loc['STUDIO', 'Sì'] - percentages.loc['DIDATTICA', 'Sì']
    
    if p_value < 0.05:
        print(f"\n✓ Le proporzioni di uso dell'IA sono SIGNIFICATIVAMENTE diverse tra")
        print(f"  STUDIO ({percentages.loc['STUDIO', 'Sì']:.1f}%) e DIDATTICA ({percentages.loc['DIDATTICA', 'Sì']:.1f}%)")
        print(f"  Differenza: {diff_perc:+.1f} punti percentuali")
        if diff_perc > 0:
            print(f"  → Gli studenti usano l'IA PIÙ degli insegnanti")
        else:
            print(f"  → Gli insegnanti usano l'IA PIÙ degli studenti")
    else:
        print(f"\n✗ Non ci sono differenze significative nelle proporzioni di uso dell'IA")
        print(f"  tra STUDIO ({percentages.loc['STUDIO', 'Sì']:.1f}%) e DIDATTICA ({percentages.loc['DIDATTICA', 'Sì']:.1f}%)")
    
    print(f"\n{'='*90}")
    print(f"Salvato: {csv_prop}")


In [ ]:
# === DIFFERENZE STUDIO VS DIDATTICA: Ore Settimanali (t-test) ===
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()

print("="*90)
print("DIFFERENZE STUDIO vs DIDATTICA - ORE SETTIMANALI")
print("="*90)

# Verifica che i dati esistano
if 'df_use' not in globals():
    print("⚠️  DataFrame 'df_use' non trovato. Esegui prima la cella sull'analisi ore settimanali.")
else:
    # Crea macro-gruppi
    df_use_macro = df_use.copy()
    df_use_macro['MacroGruppo'] = df_use_macro['GruppoDettaglio'].apply(
        lambda x: 'STUDIO' if x in ['studenti - secondaria', 'studenti - universitari'] 
        else 'DIDATTICA'
    )
    
    # Separa i dati
    studio_data = df_use_macro[df_use_macro['MacroGruppo']=='STUDIO']['OreSettimanali'].dropna()
    didattica_data = df_use_macro[df_use_macro['MacroGruppo']=='DIDATTICA']['OreSettimanali'].dropna()
    
    # Statistiche descrittive
    print("\n📊 STATISTICHE DESCRITTIVE:\n")
    print(f"{'Macro-Gruppo':<15s} {'N':>8s} {'Media':>10s} {'Mediana':>10s} {'SD':>10s} {'Min':>8s} {'Max':>8s}")
    print(f"{'─'*80}")
    
    for gruppo, data in [('STUDIO', studio_data), ('DIDATTICA', didattica_data)]:
        print(f"{gruppo:<15s} {len(data):>8d} {data.mean():>10.2f} {data.median():>10.2f} "
              f"{data.std():>10.2f} {data.min():>8.1f} {data.max():>8.1f}")
    
    # T-test indipendente
    print(f"\n{'─'*90}")
    print("T-TEST INDIPENDENTE:")
    print(f"{'─'*90}")
    
    t_stat, p_value = stats.ttest_ind(studio_data, didattica_data)
    diff_mean = studio_data.mean() - didattica_data.mean()
    
    print(f"  t-statistic = {t_stat:.4f}")
    print(f"  p-value = {p_value:.6f}")
    print(f"  Differenza medie = {diff_mean:+.2f} ore/settimana")
    
    # Significatività
    print()
    if p_value < 0.001:
        print(f"  ✓✓✓ ALTAMENTE significativo (p < 0.001)")
        sig_label = "Altamente significativo"
    elif p_value < 0.01:
        print(f"  ✓✓ Molto significativo (p < 0.01)")
        sig_label = "Molto significativo"
    elif p_value < 0.05:
        print(f"  ✓ Significativo (p < 0.05)")
        sig_label = "Significativo"
    else:
        print(f"  ✗ Non significativo (p ≥ 0.05)")
        sig_label = "Non significativo"
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(studio_data)-1)*studio_data.std()**2 + 
                          (len(didattica_data)-1)*didattica_data.std()**2) / 
                         (len(studio_data) + len(didattica_data) - 2))
    cohens_d = diff_mean / pooled_std
    
    print(f"\n  Effect size (Cohen's d) = {cohens_d:.4f}")
    if abs(cohens_d) < 0.2:
        print(f"  → Effetto PICCOLO")
    elif abs(cohens_d) < 0.5:
        print(f"  → Effetto MEDIO")
    elif abs(cohens_d) < 0.8:
        print(f"  → Effetto MEDIO-GRANDE")
    else:
        print(f"  → Effetto GRANDE")
    
    # Test di Levene (omogeneità varianze)
    print(f"\n{'─'*90}")
    print("VERIFICA ASSUNZIONI:")
    print(f"{'─'*90}")
    
    levene_stat, levene_p = stats.levene(studio_data, didattica_data)
    print(f"  Test di Levene (omogeneità varianze):")
    print(f"    Statistica = {levene_stat:.4f}, p-value = {levene_p:.4f}")
    if levene_p >= 0.05:
        print(f"    ✓ Varianze omogenee (assunzione soddisfatta)")
    else:
        print(f"    ⚠️  Varianze NON omogenee (considera Welch's t-test)")
        # Welch's t-test (più robusto)
        t_welch, p_welch = stats.ttest_ind(studio_data, didattica_data, equal_var=False)
        print(f"\n  Welch's t-test (varianze non uguali):")
        print(f"    t = {t_welch:.4f}, p-value = {p_welch:.6f}")
    
    # Test di normalità
    print(f"\n  Test di normalità (Shapiro-Wilk):")
    for gruppo, data in [('STUDIO', studio_data), ('DIDATTICA', didattica_data)]:
        if len(data) >= 3:
            shapiro_stat, shapiro_p = stats.shapiro(data)
            status = "✓" if shapiro_p >= 0.05 else "⚠️"
            print(f"    {status} {gruppo:<15s}: p = {shapiro_p:.4f}")
    
    # Salva risultati
    result_hours = {
        'Confronto': 'STUDIO vs DIDATTICA',
        'N_STUDIO': len(studio_data),
        'Media_STUDIO': studio_data.mean(),
        'SD_STUDIO': studio_data.std(),
        'N_DIDATTICA': len(didattica_data),
        'Media_DIDATTICA': didattica_data.mean(),
        'SD_DIDATTICA': didattica_data.std(),
        'Differenza_medie': diff_mean,
        't-statistic': t_stat,
        'p-value': p_value,
        'Cohens_d': cohens_d,
        'Levene_statistic': levene_stat,
        'Levene_p-value': levene_p,
        'Varianze_omogenee': levene_p >= 0.05,
        'Significativo_0.05': p_value < 0.05,
        'Significativo_0.01': p_value < 0.01,
        'Significativo_0.001': p_value < 0.001,
        'Significatività': sig_label
    }
    
    df_result_hours = pd.DataFrame([result_hours])
    csv_hours = OUT_EXPL / 'studio_vs_didattica_hours.csv'
    df_result_hours.to_csv(csv_hours, index=False)
    
    # Riepilogo finale
    print(f"\n{'='*90}")
    print("CONCLUSIONE:")
    print(f"{'='*90}")
    
    if p_value < 0.05:
        print(f"\n✓ Le ore settimanali di uso dell'IA sono SIGNIFICATIVAMENTE diverse tra")
        print(f"  STUDIO ({studio_data.mean():.2f} ore/sett) e DIDATTICA ({didattica_data.mean():.2f} ore/sett)")
        print(f"  Differenza: {diff_mean:+.2f} ore/settimana")
        if diff_mean > 0:
            print(f"  → Gli studenti usano l'IA PIÙ INTENSAMENTE degli insegnanti")
        else:
            print(f"  → Gli insegnanti usano l'IA PIÙ INTENSAMENTE degli studenti")
    else:
        print(f"\n✗ Non ci sono differenze significative nelle ore settimanali di uso dell'IA")
        print(f"  tra STUDIO ({studio_data.mean():.2f} ore/sett) e DIDATTICA ({didattica_data.mean():.2f} ore/sett)")
    
    print(f"\n{'='*90}")
    print(f"Salvato: {csv_hours}")


## 📊 RIEPILOGO STUDIO vs DIDATTICA

### 🎯 Confronto Macro-Gruppi

| Macro-Gruppo | N campione | Composizione |
|--------------|------------|--------------|
| **STUDIO** | 270 | Studenti secondaria + universitari |
| **DIDATTICA** | 457 | Insegnanti non in servizio + in servizio |
| **TOTALE** | 727 | |

---

### 1️⃣ PROPORZIONI USO IA (Sì/No)

| Macro-Gruppo | % usa IA | N usa | N non usa |
|--------------|----------|-------|-----------|
| **STUDIO** | **75.2%** | 203 | 67 |
| **DIDATTICA** | **41.4%** | 189 | 268 |
| **Differenza** | **+33.8 punti %** | | |

#### Test Chi-square:
- χ² = 76.82, df = 1
- **p-value = 1.88 × 10⁻¹⁸** → ✓✓✓ **ALTAMENTE significativo**
- Cramér's V = 0.325 → Effetto MEDIO-GRANDE

#### 💡 Interpretazione:
**Gli studenti usano l'IA MOLTO PIÙ degli insegnanti** (75% vs 41%)
- Differenza di **+34 punti percentuali**
- Significatività statistica ESTREMA (p < 0.001)
- Effetto sostanzioso (Cramér's V > 0.3)

---

### 2️⃣ ORE SETTIMANALI DI USO

| Macro-Gruppo | Media ore/sett | SD | Mediana |
|--------------|----------------|-------|---------|
| **STUDIO** | **3.91 ore** | 7.36 | |
| **DIDATTICA** | **1.88 ore** | 4.12 | |
| **Differenza** | **+2.03 ore** | | |

#### Test t indipendente:
- t = 4.74
- **p-value = 2.61 × 10⁻⁶** → ✓✓✓ **ALTAMENTE significativo**
- Cohen's d = 0.365 → Effetto MEDIO

#### Verifica assunzioni:
- ⚠️ Varianze NON omogenee (Levene p = 0.0004)
- Studio: SD = 7.36 (varianza alta)
- Didattica: SD = 4.12 (varianza più bassa)
- → Risultato robusto comunque (p-value molto basso)

#### 💡 Interpretazione:
**Gli studenti usano l'IA con MAGGIORE INTENSITÀ** (3.9 vs 1.9 ore/sett)
- Differenza di **+2 ore/settimana** (~doppio del tempo)
- Significatività statistica ESTREMA (p < 0.001)
- Effetto medio ma sostanzioso

---

### 🔍 SINTESI COMPARATIVA

| Test | Risultato | Significatività | Effect Size |
|------|-----------|-----------------|-------------|
| **Proporzioni** (Chi²) | STUDIO: 75% vs DIDATTICA: 41% | p < 0.001 ✓✓✓ | V = 0.325 (Medio-Grande) |
| **Ore settimanali** (t-test) | STUDIO: 3.9h vs DIDATTICA: 1.9h | p < 0.001 ✓✓✓ | d = 0.365 (Medio) |

---

### 💡 CONCLUSIONI PRINCIPALI

1. **DIFFERENZE ALTAMENTE SIGNIFICATIVE** su entrambi i piani:
   - **Proporzioni**: +34 punti percentuali (75% vs 41%)
   - **Intensità**: +2 ore/settimana (3.9h vs 1.9h)

2. **PATTERN CHIARO**: 
   - 🎓 **STUDIO** → Uso MASSICCIO (3 studenti su 4 usano IA, ~4 ore/sett)
   - 👨‍🏫 **DIDATTICA** → Uso MODERATO (2 insegnanti su 5 usano IA, ~2 ore/sett)

3. **CONSISTENZA**:
   - Entrambi i test (proporzioni + ore) convergono nella stessa direzione
   - Effetti robusti e sostanziosi (non solo statistici ma anche pratici)
   - Significatività ESTREMA (p < 0.000001) → risultati molto affidabili

4. **IMPLICAZIONI**:
   - L'IA è integrata maggiormente nel contesto **STUDIO** che **DIDATTICA**
   - Gli studenti sono più "early adopters" rispetto agli insegnanti
   - Potenziale gap tra competenze/uso studenti vs competenze insegnanti

---

### 📁 File salvati:
- `studio_vs_didattica_proportions.csv` - Test proporzioni
- `studio_vs_didattica_hours.csv` - Test ore settimanali

# 2. Perceived competence

Autovalutazioni di competenza pratica e teorica rispetto agli strumenti di IA.

## 2.1 Competenze percepite (pratica vs teorica)

Grafici bilingual (violin+box) sulle due dimensioni di competenza.

In [ ]:
# Competenze pratica e teorica
competence_items = [item for item in domande_comparabili if item['section'] == 'perceived_competence']
for item in competence_items:
    print('
' + '='*90)
    print(item['label_it'].upper())
    render_likert_item(item)


# 3. Training adequacy

Percezione dell'adeguatezza della formazione e supporto istituzionale.

## 3.1 Adeguatezza percepita della formazione

Grafici bilingui per la domanda Likert condivisa tra studenti e insegnanti.

In [ ]:
training_items = [item for item in domande_comparabili if item['section'] == 'training_adequacy']
for item in training_items:
    print('
' + '='*90)
    print(item['label_it'].upper())
    render_likert_item(item)


# 4. Trust and confidence

Fiducia nell'affidabilità dell'IA e nella capacità degli attori umani di usarla responsabilmente.

## 4.1 Fiducia nell'integrazione dell'IA

Domanda Likert condivisa su quanto studenti/insegnanti si sentano fiduciosi.

In [ ]:
trust_items = [item for item in domande_comparabili if item['section'] == 'trust_confidence']
for item in trust_items:
    print('
' + '='*90)
    print(item['label_it'].upper())
    render_likert_item(item)


# 5. Concerns

Rischi percepiti, barriere e livelli di preoccupazione rispetto all'inserimento dell'IA.

## 5.1 Preoccupazione generale sull'IA

Domanda Likert condivisa sul livello di preoccupazione.

In [ ]:
concern_items = [item for item in domande_comparabili if item['section'] == 'concerns']
for item in concern_items:
    print('
' + '='*90)
    print(item['label_it'].upper())
    render_likert_item(item)


# 6. Perceived change

Attese sul cambiamento didattico e sull'impatto a medio termine.

# === MAPPATURA DOMANDE SIMILI TRA STUDENTI E INSEGNANTI ===
import pandas as pd
from pathlib import Path

print("="*90)
print("MAPPATURA DOMANDE SIMILI: STUDENTI vs INSEGNANTI")
print("="*90)

# Ogni voce contiene anche la sezione analitica e lo slug per salvare i grafici
domande_comparabili = [
    {
        'tema': 'Competenza pratica uso IA',
        'insegnanti': 'Su una scala da 1 a 7, quanto ti consideri competente nell\'uso pratico di strumenti o tecnologie legati all\'intelligenza artificiale?\n(Nota: con "uso pratico" intendiamo la capacità di utilizzare concretamente strumenti, applicazioni o piattaforme di intelligenza artificiale.)\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'studenti': 'Su una scala da 1 a 7, quanto ti consideri competente nell\'uso pratico di strumenti o tecnologie legati all\'intelligenza artificiale?\n(Nota: con "uso pratico" intendiamo la capacità di utilizzare concretamente strumenti, applicazioni o piattaforme di intelligenza artificiale.)',
        'label_it': 'Competenza pratica',
        'label_en': 'Practical competence',
        'slug': 'likert_competenza_pratica',
        'section': 'perceived_competence',
    },
    {
        'tema': 'Competenza teorica IA',
        'insegnanti': 'Su una scala da 1 a 7, quanto ritieni adeguata la tua competenza teorica riguardo l\'intelligenza artificiale?\n(Nota: con "competenza teorica" si intende la conoscenza dei principi, concetti e modelli fondamentali dell\'intelligenza artificiale.)',
        'studenti': 'Su una scala da 1 a 7, quanto ritieni adeguata la tua competenza teorica riguardo l\'intelligenza artificiale?\n(Nota: con "competenza teorica" si intende la conoscenza dei principi, concetti e modelli fondamentali dell\'intelligenza artificiale.)',
        'label_it': 'Competenza teorica',
        'label_en': 'Theoretical competence',
        'slug': 'likert_competenza_teorica',
        'section': 'perceived_competence',
    },
    {
        'tema': 'Cambiamento percepito IA',
        'insegnanti': 'Da una scala da 1 a 7, quanto pensi che l\'intelligenza artificiale cambierà la tua didattica?\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'studenti': 'Da una scala da 1 a 7, quanto pensi che l\'intelligenza artificiale cambierà il tuo modo di studiare?\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'label_it': 'Cambierà didattica/studio',
        'label_en': 'Will change teaching/study',
        'slug': 'likert_cambiamento',
        'section': 'perceived_change',
    },
    {
        'tema': 'Adeguatezza formazione IA',
        'insegnanti': 'Su una scala da 1 a 7, quanto ritieni adeguata la formazione ricevuta in merito all\'intelligenza artificiale?\n(Nota: considera i corsi di formazione frequentati, le opportunità formative offerte dalla scuola e gli strumenti di apprendimento messi a disposizione.)',
        'studenti': 'Su una scala da 1 a 7, quanto ritieni adeguata la formazione ricevuta in merito all\'intelligenza artificiale?\n(Nota: considera le opportunità formative offerte dalla scuola e gli strumenti di apprendimento messi a disposizione.)',
        'label_it': 'Adeguatezza formazione',
        'label_en': 'Adequacy of training',
        'slug': 'likert_formazione',
        'section': 'training_adequacy',
    },
    {
        'tema': 'Fiducia integrazione IA',
        'insegnanti': 'Da una scala da 1 a 7, quanto sei fiducioso nell\'integrazione dell\'intelligenza artificiale nella pratica educativa?\n(1 Per niente fiducioso - 2 Poco fiducioso - 3 Moderatamente fiducioso - 4 Neutrale - 5 Piuttosto fiducioso - 6 Molto fiducioso - 7 Estremamente fiducioso)',
        'studenti': 'Da una scala da 1 a 7, quanto sei fiducioso nell\'integrazione dell\'intelligenza artificiale nella scuola o università?\n(1 Per niente fiducioso - 2 Poco fiducioso - 3 Moderatamente fiducioso - 4 Neutrale - 5 Piuttosto fiducioso - 6 Molto fiducioso - 7 Estremamente fiducioso)',
        'label_it': 'Fiducia integrazione',
        'label_en': 'Trust in integration',
        'slug': 'likert_fiducia',
        'section': 'trust_confidence',
    },
    {
        'tema': 'Preoccupazione inserimento IA',
        'insegnanti': 'Da una scala da 1 a 7, quanto sei preoccupato riguardo all\'utilizzo dell\'intelligenza artificiale nel mondo dell\'educazione?\n\n(1 Per niente preoccupato - 2 Poco preoccupato - 3 Moderatamente preoccupato - 4 Neutrale - 5 Piuttosto preoccupato - 6 Molto preoccupato - 7 Estremamente preoccupato)',
        'studenti': 'Da una scala da 1 a 7, ti preoccupa l\'inserimento dell\'intelligenza artificiale nella scuola o nell\'università?\n\n(1 Per niente preoccupato - 2 Poco preoccupato - 3 Moderatamente preoccupato - 4 Neutrale - 5 Piuttosto preoccupato - 6 Molto preoccupato - 7 Estremamente preoccupato)',
        'label_it': 'Preoccupazione generale',
        'label_en': 'General concern',
        'slug': 'likert_preoccupazione',
        'section': 'concerns',
    },
]

print("\nVerifica disponibilità domande nel dataset:
")
print(f"{'Tema':<40s} {'Ins':>5s} {'Stu':>5s}")
print('─'*52)
for item in domande_comparabili:
    ins_exists = item['insegnanti'] in DF_plot.columns
    stu_exists = item['studenti'] in DF_plot.columns
    ins_mark = '✓' if ins_exists else '✗'
    stu_mark = '✓' if stu_exists else '✗'
    print(f"{item['tema']:<40s} {ins_mark:>5s} {stu_mark:>5s}")
print('─'*52)
print(f"\nTotale coppie di domande comparabili: {len(domande_comparabili)}")

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()
mapping_df = pd.DataFrame(domande_comparabili)
csv_mapping = OUT_EXPL / 'domande_likert_mapping.csv'
mapping_df.to_csv(csv_mapping, index=False)
print(f"Salvato mapping in: {csv_mapping}")


# 6. Perceived change

Aspettative sul cambiamento didattico e sullo studio dovuto all'IA.

## 6.1 Cambiamento percepito (didattica/studio)

Grafici bilingui sulla domanda dedicata.

In [ ]:
change_items = [item for item in domande_comparabili if item['section'] == 'perceived_change']
for item in change_items:
    print('
' + '='*90)
    print(item['label_it'].upper())
    render_likert_item(item)


In [ ]:
# Helper per generare grafici bilingui sulle domande Likert comparabili
def render_likert_item(item, slug_suffix=None, note_it=None, note_en=None):
    col_ins = item['insegnanti']
    col_stu = item['studenti']
    slug = slug_suffix or item.get('slug')
    df_vis = DF_plot[['GruppoDettaglio']].copy()
    mask_teachers = df_vis['GruppoDettaglio'].str.contains('insegnanti', case=False, na=False)
    mask_students = df_vis['GruppoDettaglio'].str.contains('studenti', case=False, na=False)
    df_vis.loc[mask_teachers, 'Valore'] = pd.to_numeric(DF_plot.loc[mask_teachers, col_ins], errors='coerce')
    df_vis.loc[mask_students, 'Valore'] = pd.to_numeric(DF_plot.loc[mask_students, col_stu], errors='coerce')
    df_vis = df_vis.dropna(subset=['Valore'])
    df_vis = df_vis[df_vis['GruppoDettaglio'].isin(ORDER_4)]
    if df_vis.empty:
        print(f"⚠️ Nessun dato disponibile per {item['tema']}")
        return df_vis
    df_vis['GruppoDettaglio'] = pd.Categorical(df_vis['GruppoDettaglio'], categories=ORDER_4, ordered=True)
    if note_it:
        print(note_it)
    bilingual_violin_box(
        df_vis[['GruppoDettaglio', 'Valore']],
        value_col='Valore',
        slug=slug,
        title_it=item['label_it'],
        title_en=item['label_en'],
        ylabel_it='Punteggio (scala 1-7)',
        ylabel_en='Score (Likert 1-7)',
        save_csv=True,
    )
    return df_vis


## 📊 Analisi Domande Likert (scala 1-7)

### Piano di Lavoro:
1. Identificare tutte le domande con scala Likert 1-7
2. Raggruppare domande simili tra studenti e insegnanti
3. Creare grafici a violino per i 4 gruppi
4. Analizzare la variabilità interna
5. Confrontare risposte tra studenti e insegnanti su temi comuni

In [ ]:
# === IDENTIFICAZIONE DOMANDE LIKERT 1-7 ===
import pandas as pd
import numpy as np
import re
from pathlib import Path

print("="*90)
print("IDENTIFICAZIONE DOMANDE LIKERT (scala 1-7)")
print("="*90)

# Cerca colonne che contengono pattern tipici di domande Likert
likert_patterns = [
    r'scala.*1.*7',
    r'quanto.*pensi',
    r'quanto.*sei.*fiducioso',
    r'quanto.*sei.*preoccupato',
    r'quanto.*ritieni',
    r'quanto.*ti.*consideri',
]

likert_questions = {}

for col in DF_plot.columns:
    col_lower = str(col).lower()
    
    # Cerca pattern Likert nel nome della colonna
    is_likert = any(re.search(pattern, col_lower, re.I) for pattern in likert_patterns)
    
    if is_likert:
        # Verifica che i valori siano numerici 1-7
        values = DF_plot[col].dropna()
        if len(values) > 0:
            try:
                numeric_values = pd.to_numeric(values, errors='coerce').dropna()
                if len(numeric_values) > 0:
                    unique_vals = sorted(numeric_values.unique())
                    # Verifica se i valori sono nell'intervallo 1-7
                    if all(1 <= v <= 7 for v in unique_vals):
                        likert_questions[col] = {
                            'valori_unici': unique_vals,
                            'n_validi': len(numeric_values),
                            'media': numeric_values.mean(),
                            'mediana': numeric_values.median()
                        }
            except:
                pass

print(f"\n✓ Trovate {len(likert_questions)} domande Likert (scala 1-7)\n")
print(f"{'─'*90}")
print(f"{'N':<4s} {'Domanda':<70s} {'N validi':>12s}")
print(f"{'─'*90}")

for i, (q, info) in enumerate(likert_questions.items(), 1):
    # Trunca domanda se troppo lunga
    q_short = q[:67] + '...' if len(q) > 70 else q
    print(f"{i:<4d} {q_short:<70s} {info['n_validi']:>12d}")

print(f"{'─'*90}")

# Salva lista completa
OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()
likert_df = pd.DataFrame([
    {'n': i, 'domanda': q, 'n_validi': info['n_validi'], 
     'media': info['media'], 'mediana': info['mediana']}
    for i, (q, info) in enumerate(likert_questions.items(), 1)
])
csv_likert = OUT_EXPL / 'domande_likert_1-7.csv'
likert_df.to_csv(csv_likert, index=False)
print(f"\nSalvato elenco in: {csv_likert}")


In [ ]:
# === MAPPATURA DOMANDE SIMILI TRA STUDENTI E INSEGNANTI ===
import pandas as pd
from pathlib import Path

print("="*90)
print("MAPPATURA DOMANDE SIMILI: STUDENTI vs INSEGNANTI")
print("="*90)

# Definizione delle coppie di domande comparabili
# Formato: (nome_breve, domanda_insegnanti, domanda_studenti, tema)

domande_comparabili = [
    # 1. COMPETENZA PRATICA
    {
        'tema': 'Competenza pratica uso IA',
        'insegnanti': 'Su una scala da 1 a 7, quanto ti consideri competente nell\'uso pratico di strumenti o tecnologie legati all\'intelligenza artificiale?\n(Nota: con "uso pratico" intendiamo la capacità di utilizzare concretamente strumenti, applicazioni o piattaforme di intelligenza artificiale.)\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'studenti': 'Su una scala da 1 a 7, quanto ti consideri competente nell\'uso pratico di strumenti o tecnologie legati all\'intelligenza artificiale?\n(Nota: con "uso pratico" intendiamo la capacità di utilizzare concretamente strumenti, applicazioni o piattaforme di intelligenza artificiale.)',
        'label_it': 'Competenza pratica',
        'label_en': 'Practical competence'
    },
    
    # 2. COMPETENZA TEORICA
    {
        'tema': 'Competenza teorica IA',
        'insegnanti': 'Su una scala da 1 a 7, quanto ritieni adeguata la tua competenza teorica riguardo l\'intelligenza artificiale?\n(Nota: con "competenza teorica" si intende la conoscenza dei principi, concetti e modelli fondamentali dell\'intelligenza artificiale.)',
        'studenti': 'Su una scala da 1 a 7, quanto ritieni adeguata la tua competenza teorica riguardo l\'intelligenza artificiale?\n(Nota: con "competenza teorica" si intende la conoscenza dei principi, concetti e modelli fondamentali dell\'intelligenza artificiale.)',
        'label_it': 'Competenza teorica',
        'label_en': 'Theoretical competence'
    },
    
    # 3. CAMBIAMENTO DIDATTICA/STUDIO
    {
        'tema': 'Cambiamento percepito IA',
        'insegnanti': 'Da una scala da 1 a 7, quanto pensi che l\'intelligenza artificiale cambierà la tua didattica?\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'studenti': 'Da una scala da 1 a 7, quanto pensi che l\'intelligenza artificiale cambierà il tuo modo di studiare?\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'label_it': 'Cambierà didattica/studio',
        'label_en': 'Will change teaching/study'
    },
    
    # 4. FORMAZIONE RICEVUTA
    {
        'tema': 'Adeguatezza formazione IA',
        'insegnanti': 'Su una scala da 1 a 7, quanto ritieni adeguata la formazione ricevuta in merito all\'intelligenza artificiale?\n(Nota: considera i corsi di formazione frequentati, le opportunità formative offerte dalla scuola e gli strumenti di apprendimento messi a disposizione.)',
        'studenti': 'Su una scala da 1 a 7, quanto ritieni adeguata la formazione ricevuta in merito all\'intelligenza artificiale?\n(Nota: considera le opportunità formative offerte dalla scuola e gli strumenti di apprendimento messi a disposizione.)',
        'label_it': 'Adeguatezza formazione',
        'label_en': 'Adequacy of training'
    },
    
    # 5. FIDUCIA INTEGRAZIONE
    {
        'tema': 'Fiducia integrazione IA',
        'insegnanti': 'Da una scala da 1 a 7, quanto sei fiducioso nell\'integrazione dell\'intelligenza artificiale nella pratica educativa?\n(1 Per niente fiducioso - 2 Poco fiducioso - 3 Moderatamente fiducioso - 4 Neutrale - 5 Piuttosto fiducioso - 6 Molto fiducioso - 7 Estremamente fiducioso)',
        'studenti': 'Da una scala da 1 a 7, quanto sei fiducioso nell\'integrazione dell\'intelligenza artificiale nella scuola o università?\n(1 Per niente fiducioso - 2 Poco fiducioso - 3 Moderatamente fiducioso - 4 Neutrale - 5 Piuttosto fiducioso - 6 Molto fiducioso - 7 Estremamente fiducioso)',
        'label_it': 'Fiducia integrazione',
        'label_en': 'Trust in integration'
    },
    
    # 6. PREOCCUPAZIONE INSERIMENTO IA
    {
        'tema': 'Preoccupazione inserimento IA',
        'insegnanti': 'Da una scala da 1 a 7, quanto sei preoccupato riguardo all\'utilizzo dell\'intelligenza artificiale nel mondo dell\'educazione?\n\n(1 Per niente preoccupato - 2 Poco preoccupato - 3 Moderatamente preoccupato - 4 Neutrale - 5 Piuttosto preoccupato - 6 Molto preoccupato - 7 Estremamente preoccupato)',
        'studenti': 'Da una scala da 1 a 7, ti preoccupa l\'inserimento dell\'intelligenza artificiale nella scuola o nell\'università?\n\n(1 Per niente preoccupato - 2 Poco preoccupato - 3 Moderatamente preoccupato - 4 Neutrale - 5 Piuttosto preoccupato - 6 Molto preoccupato - 7 Estremamente preoccupato)',
        'label_it': 'Preoccupazione generale',
        'label_en': 'General concern'
    },
]

# Verifica che tutte le domande esistano nel dataset
print("\nVerifica disponibilità domande nel dataset:\n")
print(f"{'Tema':<40s} {'Ins':>5s} {'Stu':>5s}")
print("─"*52)

for item in domande_comparabili:
    ins_exists = item['insegnanti'] in DF_plot.columns
    stu_exists = item['studenti'] in DF_plot.columns
    ins_mark = "✓" if ins_exists else "✗"
    stu_mark = "✓" if stu_exists else "✗"
    print(f"{item['tema']:<40s} {ins_mark:>5s} {stu_mark:>5s}")

print("─"*52)
print(f"\nTotale coppie di domande comparabili: {len(domande_comparabili)}")

# Salva mapping
OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()
mapping_df = pd.DataFrame(domande_comparabili)
csv_mapping = OUT_EXPL / 'domande_likert_mapping.csv'
mapping_df.to_csv(csv_mapping, index=False)
print(f"Salvato mapping in: {csv_mapping}")


In [ ]:
# === RICERCA FLESSIBILE DOMANDE LIKERT ===
import re

print("="*90)
print("RICERCA DOMANDE NEL DATASET (pattern flessibili)")
print("="*90)

# Pattern di ricerca più flessibili
search_patterns = {
    'competenza_pratica': r'quanto ti consideri competente.*uso pratico.*intelligenza',
    'competenza_teorica': r'quanto ritieni adeguata.*competenza teorica.*intelligenza',
    'cambiamento_didattica_ins': r'quanto pensi.*intelligenza.*cambierà.*tua didattica',
    'cambiamento_studio_stu': r'quanto pensi.*intelligenza.*cambierà.*modo di studiare',
    'formazione': r'quanto ritieni adeguata.*formazione.*intelligenza',
    'fiducia_pratica_ins': r'quanto sei fiducioso.*integrazione.*intelligenza.*pratica educativa',
    'fiducia_scuola_stu': r'quanto sei fiducioso.*integrazione.*intelligenza.*scuola',
    'preoccupazione_educazione_ins': r'quanto sei preoccupato.*intelligenza.*mondo.*educazione',
    'preoccupazione_scuola_stu': r'preoccupa.*inserimento.*intelligenza.*scuola',
    'preoccupazione_studenti_ins': r'quanto sei preoccupato.*intelligenza.*parte degli studenti',
    'preoccupazione_compagni_stu': r'quanto sei preoccupato.*intelligenza.*compagni',
    'cambio_didattica_generale': r'quanto pensi.*intelligenza.*cambierà la didattica[^t]',  # non "tua"
    'preparazione_insegnanti': r'quanto ritieni.*insegnanti.*preparati.*intelligenza',
}

found_questions = {}

for key, pattern in search_patterns.items():
    for col in DF_plot.columns:
        if re.search(pattern, str(col), re.IGNORECASE | re.DOTALL):
            found_questions[key] = col
            print(f"\n✓ {key}:")
            print(f"  {col[:120]}...")
            break

print(f"\n{'='*90}")
print(f"Trovate {len(found_questions)}/{len(search_patterns)} domande")


### 🎻 Grafici a Violino: Domande Likert per i 4 Gruppi

Creiamo grafici a violino per visualizzare:
1. **Variabilità interna** in ciascun gruppo
2. **Confronti tra i 4 gruppi** per ogni domanda
3. **Confronti studenti vs insegnanti** su domande simili

In [ ]:
# === GRAFICI A VIOLINO: DOMANDE COMPARABILI STUDENTI vs INSEGNANTI ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib as mpl

# Setup grafico
plt.style.use(['science', 'no-latex', 'grid'])
mpl.rcParams.update({
    'text.usetex': False,
    'mathtext.fontset': 'dejavusans',
    'font.family': 'DejaVu Sans',
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 10,
})

OUT_EXPL = (Path.cwd()/"../analysis/exports/latest").resolve()
ASSETS = (Path.cwd()/"../assets/figures").resolve()
OUT_EXPL.mkdir(parents=True, exist_ok=True)
ASSETS.mkdir(parents=True, exist_ok=True)

# Palette colori per i 4 gruppi
palette_4groups = {
    'studenti - secondaria': '#e41a1c',
    'studenti - universitari': '#4daf4a',
    'insegnanti - non in servizio': '#ffd31a',
    'insegnanti - in servizio': '#377eb8',
}

print("="*90)
print("GRAFICI A VIOLINO: DOMANDE COMPARABILI")
print("="*90)

# Domande comparabili trovate
domande_coppie = [
    {
        'tema': 'Cambiamento didattica/studio',
        'domanda_ins': found_questions.get('cambiamento_didattica_ins'),
        'domanda_stu': found_questions.get('cambiamento_studio_stu'),
        'label_breve': 'Cambierà didattica/studio',
        'file': 'likert_cambiamento_didattica_studio'
    },
    {
        'tema': 'Fiducia integrazione IA',
        'domanda_ins': found_questions.get('fiducia_pratica_ins'),
        'domanda_stu': found_questions.get('fiducia_scuola_stu'),
        'label_breve': 'Fiducia integrazione',
        'file': 'likert_fiducia_integrazione'
    },
    {
        'tema': 'Preoccupazione IA educazione',
        'domanda_ins': found_questions.get('preoccupazione_educazione_ins'),
        'domanda_stu': found_questions.get('preoccupazione_scuola_stu'),
        'label_breve': 'Preoccupazione generale',
        'file': 'likert_preoccupazione_generale'
    },
]

# Genera grafici
for coppia in domande_coppie:
    if coppia['domanda_ins'] is None or coppia['domanda_stu'] is None:
        print(f"\n⚠️  Saltata coppia '{coppia['tema']}' - domande mancanti")
        continue
    
    print(f"\n{'─'*90}")
    print(f"📊 {coppia['tema']}")
    print(f"{'─'*90}")
    
    # Prepara dati: unifica le due domande in una singola colonna
    df_vis = DF_plot[['GruppoDettaglio']].copy()
    
    # Per insegnanti, usa la domanda insegnanti
    mask_ins = DF_plot['GruppoDettaglio'].str.contains('insegnanti', na=False)
    df_vis.loc[mask_ins, 'Risposta'] = pd.to_numeric(
        DF_plot.loc[mask_ins, coppia['domanda_ins']], 
        errors='coerce'
    )
    
    # Per studenti, usa la domanda studenti
    mask_stu = DF_plot['GruppoDettaglio'].str.contains('studenti', na=False)
    df_vis.loc[mask_stu, 'Risposta'] = pd.to_numeric(
        DF_plot.loc[mask_stu, coppia['domanda_stu']], 
        errors='coerce'
    )
    
    # Rimuovi valori mancanti
    df_vis = df_vis.dropna(subset=['Risposta'])
    df_vis = df_vis[df_vis['GruppoDettaglio'].isin(ORDER_4)]
    df_vis['GruppoDettaglio'] = pd.Categorical(
        df_vis['GruppoDettaglio'], 
        categories=ORDER_4, 
        ordered=True
    )
    
    if len(df_vis) == 0:
        print("  ⚠️  Nessun dato disponibile")
        continue
    
    # Statistiche descrittive
    print(f"\n  Statistiche per gruppo:")
    stats = df_vis.groupby('GruppoDettaglio', observed=False)['Risposta'].agg([
        ('N', 'count'),
        ('Media', 'mean'),
        ('Mediana', 'median'),
        ('SD', 'std'),
        ('Min', 'min'),
        ('Max', 'max')
    ]).round(2)
    print(stats)
    
    # Crea grafico - ITALIANO
    fig, ax = plt.subplots(figsize=(10, 6))
    
    sns.violinplot(
        data=df_vis,
        x='GruppoDettaglio',
        y='Risposta',
        hue='GruppoDettaglio',
        palette=palette_4groups,
        order=ORDER_4,
        inner='box',
        cut=0,
        ax=ax
    )
    
    ax.set_xlabel('Gruppo', fontsize=11)
    ax.set_ylabel('Punteggio (scala 1-7)', fontsize=11)
    ax.set_title(f'{coppia["label_breve"]}\n(scala Likert 1-7)', fontsize=12, pad=15)
    ax.set_ylim(0.5, 7.5)
    ax.set_yticks(range(1, 8))
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    
    # Rimuovi legenda (ridondante con x-axis)
    if ax.get_legend():
        ax.get_legend().remove()
    
    plt.xticks(rotation=15, ha='right')
    sns.despine(ax=ax)
    plt.tight_layout()
    
    # Salva
    fp_it_png = OUT_EXPL / f"{coppia['file']}_4groups_it.png"
    fp_it_svg = ASSETS / f"{coppia['file']}_4groups_it.svg"
    fig.savefig(fp_it_png, bbox_inches='tight')
    fig.savefig(fp_it_svg, bbox_inches='tight')
    plt.show()
    plt.close(fig)
    
    print(f"  ✓ Salvato: {fp_it_png}")
    
    # Versione INGLESE
    fig_en, ax_en = plt.subplots(figsize=(10, 6))
    
    # Mappa nomi gruppi in inglese
    map_en = {
        'studenti - secondaria': 'students - secondary',
        'studenti - universitari': 'students - university',
        'insegnanti - non in servizio': 'teachers - pre-service',
        'insegnanti - in servizio': 'teachers - in-service',
    }
    
    df_vis_en = df_vis.copy()
    df_vis_en['GruppoDettaglio_EN'] = df_vis_en['GruppoDettaglio'].map(map_en)
    order_en = [map_en[g] for g in ORDER_4]
    df_vis_en['GruppoDettaglio_EN'] = pd.Categorical(
        df_vis_en['GruppoDettaglio_EN'],
        categories=order_en,
        ordered=True
    )
    
    palette_en = {map_en[k]: v for k, v in palette_4groups.items()}
    
    sns.violinplot(
        data=df_vis_en,
        x='GruppoDettaglio_EN',
        y='Risposta',
        hue='GruppoDettaglio_EN',
        palette=palette_en,
        order=order_en,
        inner='box',
        cut=0,
        ax=ax_en
    )
    
    ax_en.set_xlabel('Group', fontsize=11)
    ax_en.set_ylabel('Score (1-7 scale)', fontsize=11)
    
    # Traduci label
    label_en_map = {
        'Cambierà didattica/studio': 'Will change teaching/study',
        'Fiducia integrazione': 'Trust in integration',
        'Preoccupazione generale': 'General concern'
    }
    label_en = label_en_map.get(coppia['label_breve'], coppia['label_breve'])
    
    ax_en.set_title(f'{label_en}\n(Likert scale 1-7)', fontsize=12, pad=15)
    ax_en.set_ylim(0.5, 7.5)
    ax_en.set_yticks(range(1, 8))
    ax_en.grid(axis='y', alpha=0.3, linestyle='--')
    ax_en.set_axisbelow(True)
    
    if ax_en.get_legend():
        ax_en.get_legend().remove()
    
    plt.xticks(rotation=15, ha='right')
    sns.despine(ax=ax_en)
    plt.tight_layout()
    
    fp_en_png = OUT_EXPL / f"{coppia['file']}_4groups_en.png"
    fp_en_svg = ASSETS / f"{coppia['file']}_4groups_en.svg"
    fig_en.savefig(fp_en_png, bbox_inches='tight')
    fig_en.savefig(fp_en_svg, bbox_inches='tight')
    plt.show()
    plt.close(fig_en)
    
    print(f"  ✓ Salvato: {fp_en_png}")

print(f"\n{'='*90}")
print("✓ Grafici completati!")


---
### 📊 Domande Likert NON comparabili

Ora analizziamo le domande Likert che sono specifiche solo per **studenti** o solo per **insegnanti** (non hanno una controparte diretta nell'altro gruppo).


In [ ]:
# === IDENTIFICAZIONE DOMANDE NON COMPARABILI ===

# Dalla mappatura precedente, identifichiamo le domande che sono specifiche per ogni gruppo

# Domande SOLO insegnanti (non hanno corrispondenza con studenti)
# NOTA: "Competenza teorica IA" è già inclusa nei grafici comparabili, quindi la escludiamo
solo_insegnanti = [
    {
        'col': 'Da una scala da 1 a 7 quanto pensi che l\'intelligenza artificiale cambierà la didattica?\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'label_it': 'Cambierà la didattica (generale)',
        'label_en': 'Will change teaching (general)',
        'tema': 'cambiamento_didattica_generale'
    },
    {
        'col': 'Da una scala da 1 a 7, quanto pensi che l\'intelligenza artificiale cambierà la tua didattica?\n\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale -  5 Piuttosto - 6 Molto - 7 Estremamente)',
        'label_it': 'Cambierà la mia didattica (personale)',
        'label_en': 'Will change my teaching (personal)',
        'tema': 'cambiamento_didattica_personale'
    },
    {
        'col': 'Da una scala da 1 a 7, quanto sei fiducioso nell\'utilizzo da parte degli studenti di un uso responsabile e maturo dell\'intelligenza artificiale?\n(1 Per niente fiducioso - 2 Poco fiducioso - 3 Moderatamente fiducioso - 4 Neutrale - 5 Piuttosto fiducioso - 6 Molto fiducioso - 7 Estremamente fiducioso)',
        'label_it': 'Fiducia uso responsabile studenti',
        'label_en': 'Trust in students\' responsible use',
        'tema': 'fiducia_studenti'
    },
    {
        'col': 'Da una scala da 1 a 7, quanto sei preoccupato riguardo all\'utilizzo dell\'intelligenza artificiale da parte degli studenti?\n(1 Per niente preoccupato - 2 Poco preoccupato - 3 Moderatamente preoccupato - 4 Neutrale - 5 Piuttosto preoccupato - 6 Molto preoccupato - 7 Estremamente preoccupato)',
        'label_it': 'Preoccupazione uso IA studenti',
        'label_en': 'Concern about students\' AI use',
        'tema': 'preoccupazione_studenti'
    }
]

# Domande SOLO studenti (non hanno corrispondenza con insegnanti)
solo_studenti = [
    {
        'col': 'Su una scala da 1 a 7, quanto ritieni che i tuoi attuali insegnanti siano preparati e competenti nell\'insegnare l\'uso dell\'intelligenza artificiale?\n(1 Per niente - 2 Poco - 3 Moderatamente - 4 Neutrale - 5 Abbastanza - 6 Molto - 7 Estremamente)',
        'label_it': 'Preparazione insegnanti su IA',
        'label_en': 'Teachers\' preparedness on AI',
        'tema': 'preparazione_insegnanti'
    },
    {
        'col': 'Da una scala da 1 a 7, quanto sei preoccupato riguardo all\'utilizzo dell\'intelligenza artificiale da parte dei tuoi compagni di scuola o universitarì?\n(1 Per niente preoccupato - 2 Poco preoccupato - 3 Moderatamente preoccupato - 4 Neutrale - 5 Piuttosto preoccupato - 6 Molto preoccupato - 7 Estremamente preoccupato)',
        'label_it': 'Preoccupazione uso IA compagni',
        'label_en': 'Concern about peers\' AI use',
        'tema': 'preoccupazione_compagni'
    }
]

print("="*90)
print("DOMANDE LIKERT NON COMPARABILI")
print("="*90)
print(f"\n✓ Domande SOLO INSEGNANTI: {len(solo_insegnanti)}")
for i, d in enumerate(solo_insegnanti, 1):
    print(f"  {i}. {d['label_it']}")
    
print(f"\n✓ Domande SOLO STUDENTI: {len(solo_studenti)}")
for i, d in enumerate(solo_studenti, 1):
    print(f"  {i}. {d['label_it']}")

print(f"\n✓ TOTALE domande non comparabili: {len(solo_insegnanti) + len(solo_studenti)}")

In [ ]:
# === GRAFICI A VIOLINO: DOMANDE SOLO INSEGNANTI ===

print("="*90)
print("GRAFICI A VIOLINO: DOMANDE SOLO INSEGNANTI")
print("="*90)

# Solo i 2 gruppi di insegnanti
order_insegnanti = ['insegnanti - non in servizio', 'insegnanti - in servizio']
palette_insegnanti = {
    'insegnanti - non in servizio': palette_4groups['insegnanti - non in servizio'],
    'insegnanti - in servizio': palette_4groups['insegnanti - in servizio']
}

for q in solo_insegnanti:
    col = q['col']
    label_it = q['label_it']
    label_en = q['label_en']
    tema = q['tema']
    
    if col not in DF_plot.columns:
        print(f"\n⚠️ Colonna non trovata: {label_it}")
        continue
    
    # Prepara dati - solo insegnanti
    df_temp = DF_plot[DF_plot['GruppoDettaglio'].isin(order_insegnanti)].copy()
    df_temp = df_temp[[col, 'GruppoDettaglio']].dropna()
    
    if len(df_temp) == 0:
        print(f"\n⚠️ Nessun dato per: {label_it}")
        continue
        
    df_vis = df_temp.copy()
    df_vis.columns = ['valore', 'GruppoDettaglio']
    # Rimuovi categorie non usate e riordina
    df_vis['GruppoDettaglio'] = pd.Categorical(
        df_vis['GruppoDettaglio'], 
        categories=order_insegnanti, 
        ordered=True
    )
    
    # Statistiche
    stats = df_vis.groupby('GruppoDettaglio', observed=True)['valore'].agg(['count', 'mean', 'median', 'std', 'min', 'max'])
    stats.columns = ['N', 'Media', 'Mediana', 'SD', 'Min', 'Max']
    
    print(f"\n{'─'*86}")
    print(f"📊 {label_it}")
    print(f"{'─'*86}")
    print(f"\n  Statistiche per gruppo:")
    print(stats.to_string())
    
    # === GRAFICO ITALIANO ===
    fig_it, ax_it = plt.subplots(1, 1, figsize=(10, 6))
    
    sns.violinplot(
        data=df_vis,
        x='GruppoDettaglio',
        y='valore',
        hue='GruppoDettaglio',
        palette=palette_insegnanti,
        ax=ax_it,
        legend=False,
        inner='box'
    )
    ax_it.set_xlabel('Gruppo', fontsize=12)
    ax_it.set_ylabel('Punteggio (scala 1-7)', fontsize=12)
    ax_it.set_title(f'{label_it}\n(scala Likert 1-7)', fontsize=13, weight='bold')
    ax_it.set_ylim(0.5, 7.5)
    ax_it.set_xticklabels(ax_it.get_xticklabels(), rotation=15, ha='right')
    ax_it.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Salva IT
    out_it = OUT_EXPL / f'likert_{tema}_insegnanti_it.png'
    fig_it.savefig(out_it, dpi=300, bbox_inches='tight')
    print(f"\n  ✓ Salvato: {out_it}")
    
    # Mostra IT
    plt.show()
    plt.close(fig_it)
    
    # === GRAFICO INGLESE ===
    order_en_ins = ['teachers - pre-service', 'teachers - in-service']
    df_vis_en = df_vis.copy()
    df_vis_en['GruppoDettaglio'] = df_vis_en['GruppoDettaglio'].cat.rename_categories({
        'insegnanti - non in servizio': 'teachers - pre-service',
        'insegnanti - in servizio': 'teachers - in-service'
    })
    palette_en_ins = {
        'teachers - pre-service': palette_insegnanti['insegnanti - non in servizio'],
        'teachers - in-service': palette_insegnanti['insegnanti - in servizio']
    }
    
    fig_en, ax_en = plt.subplots(1, 1, figsize=(10, 6))
    
    sns.violinplot(
        data=df_vis_en,
        x='GruppoDettaglio',
        y='valore',
        hue='GruppoDettaglio',
        palette=palette_en_ins,
        ax=ax_en,
        legend=False,
        inner='box'
    )
    ax_en.set_xlabel('Group', fontsize=12)
    ax_en.set_ylabel('Score (1-7 scale)', fontsize=12)
    ax_en.set_title(f'{label_en}\n(Likert scale 1-7)', fontsize=13, weight='bold')
    ax_en.set_ylim(0.5, 7.5)
    ax_en.set_xticklabels(ax_en.get_xticklabels(), rotation=15, ha='right')
    ax_en.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Salva EN
    out_en = OUT_EXPL / f'likert_{tema}_insegnanti_en.png'
    fig_en.savefig(out_en, dpi=300, bbox_inches='tight')
    print(f"  ✓ Salvato: {out_en}")
    
    # Mostra EN
    plt.show()
    plt.close(fig_en)

print(f"\n{'='*90}")
print(f"✓ Grafici insegnanti completati!")
print(f"{'='*90}")

In [ ]:
# === GRAFICI A VIOLINO: DOMANDE SOLO STUDENTI ===

print("\n" + "="*90)
print("GRAFICI A VIOLINO: DOMANDE SOLO STUDENTI")
print("="*90)

# Solo i 2 gruppi di studenti
order_studenti = ['studenti - secondaria', 'studenti - universitari']
palette_studenti = {
    'studenti - secondaria': palette_4groups['studenti - secondaria'],
    'studenti - universitari': palette_4groups['studenti - universitari']
}

for q in solo_studenti:
    col = q['col']
    label_it = q['label_it']
    label_en = q['label_en']
    tema = q['tema']
    
    if col not in DF_plot.columns:
        print(f"\n⚠️ Colonna non trovata: {label_it}")
        continue
    
    # Prepara dati - solo studenti
    df_temp = DF_plot[DF_plot['GruppoDettaglio'].isin(order_studenti)].copy()
    df_temp = df_temp[[col, 'GruppoDettaglio']].dropna()
    
    if len(df_temp) == 0:
        print(f"\n⚠️ Nessun dato per: {label_it}")
        continue
        
    df_vis = df_temp.copy()
    df_vis.columns = ['valore', 'GruppoDettaglio']
    # Rimuovi categorie non usate e riordina
    df_vis['GruppoDettaglio'] = pd.Categorical(
        df_vis['GruppoDettaglio'], 
        categories=order_studenti, 
        ordered=True
    )
    
    # Statistiche
    stats = df_vis.groupby('GruppoDettaglio', observed=True)['valore'].agg(['count', 'mean', 'median', 'std', 'min', 'max'])
    stats.columns = ['N', 'Media', 'Mediana', 'SD', 'Min', 'Max']
    
    print(f"\n{'─'*86}")
    print(f"📊 {label_it}")
    print(f"{'─'*86}")
    print(f"\n  Statistiche per gruppo:")
    print(stats.to_string())
    
    # === GRAFICO ITALIANO ===
    fig_it, ax_it = plt.subplots(1, 1, figsize=(10, 6))
    
    sns.violinplot(
        data=df_vis,
        x='GruppoDettaglio',
        y='valore',
        hue='GruppoDettaglio',
        palette=palette_studenti,
        ax=ax_it,
        legend=False,
        inner='box'
    )
    ax_it.set_xlabel('Gruppo', fontsize=12)
    ax_it.set_ylabel('Punteggio (scala 1-7)', fontsize=12)
    ax_it.set_title(f'{label_it}\n(scala Likert 1-7)', fontsize=13, weight='bold')
    ax_it.set_ylim(0.5, 7.5)
    ax_it.set_xticklabels(ax_it.get_xticklabels(), rotation=15, ha='right')
    ax_it.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Salva IT
    out_it = OUT_EXPL / f'likert_{tema}_studenti_it.png'
    fig_it.savefig(out_it, dpi=300, bbox_inches='tight')
    print(f"\n  ✓ Salvato: {out_it}")
    
    # Mostra IT
    plt.show()
    plt.close(fig_it)
    
    # === GRAFICO INGLESE ===
    order_en_stud = ['students - secondary', 'students - university']
    df_vis_en = df_vis.copy()
    df_vis_en['GruppoDettaglio'] = df_vis_en['GruppoDettaglio'].cat.rename_categories({
        'studenti - secondaria': 'students - secondary',
        'studenti - universitari': 'students - university'
    })
    palette_en_stud = {
        'students - secondary': palette_studenti['studenti - secondaria'],
        'students - university': palette_studenti['studenti - universitari']
    }
    
    fig_en, ax_en = plt.subplots(1, 1, figsize=(10, 6))
    
    sns.violinplot(
        data=df_vis_en,
        x='GruppoDettaglio',
        y='valore',
        hue='GruppoDettaglio',
        palette=palette_en_stud,
        ax=ax_en,
        legend=False,
        inner='box'
    )
    ax_en.set_xlabel('Group', fontsize=12)
    ax_en.set_ylabel('Score (1-7 scale)', fontsize=12)
    ax_en.set_title(f'{label_en}\n(Likert scale 1-7)', fontsize=13, weight='bold')
    ax_en.set_ylim(0.5, 7.5)
    ax_en.set_xticklabels(ax_en.get_xticklabels(), rotation=15, ha='right')
    ax_en.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Salva EN
    out_en = OUT_EXPL / f'likert_{tema}_studenti_en.png'
    fig_en.savefig(out_en, dpi=300, bbox_inches='tight')
    print(f"  ✓ Salvato: {out_en}")
    
    # Mostra EN
    plt.show()
    plt.close(fig_en)

print(f"\n{'='*90}")
print(f"✓ Grafici studenti completati!")
print(f"{'='*90}")


---
### 📊 Analisi Differenze Interne tra Sottogruppi

Verifichiamo se ci sono **differenze significative** tra i sottogruppi:
- **Insegnanti**: non in servizio vs in servizio
- **Studenti**: secondaria vs universitari

In [ ]:
# === ANALISI DIFFERENZE INTERNE TRA SOTTOGRUPPI ===

from scipy.stats import ttest_ind, mannwhitneyu
import numpy as np

print("="*90)
print("ANALISI DIFFERENZE INTERNE TRA SOTTOGRUPPI - DOMANDE LIKERT")
print("="*90)

# Funzione per calcolare Cohen's d
def cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    return (np.mean(group1) - np.mean(group2)) / pooled_std

# Lista per salvare i risultati
risultati_differenze = []

# ===== ANALISI INSEGNANTI =====
print("\n" + "="*90)
print("1. DIFFERENZE TRA INSEGNANTI: NON IN SERVIZIO vs IN SERVIZIO")
print("="*90)

for q in solo_insegnanti:
    col = q['col']
    label = q['label_it']
    
    if col not in DF_plot.columns:
        continue
    
    # Estrai dati per i due gruppi
    df_ins = DF_plot[DF_plot['GruppoDettaglio'].isin(['insegnanti - non in servizio', 'insegnanti - in servizio'])].copy()
    df_ins = df_ins[[col, 'GruppoDettaglio']].dropna()
    
    if len(df_ins) < 10:
        continue
    
    # Dati per gruppo - converti in float
    non_servizio = pd.to_numeric(df_ins[df_ins['GruppoDettaglio'] == 'insegnanti - non in servizio'][col], errors='coerce').dropna().values
    in_servizio = pd.to_numeric(df_ins[df_ins['GruppoDettaglio'] == 'insegnanti - in servizio'][col], errors='coerce').dropna().values
    
    if len(non_servizio) < 3 or len(in_servizio) < 3:
        continue
    
    # Statistiche descrittive
    mean_non = np.mean(non_servizio)
    mean_in = np.mean(in_servizio)
    diff = mean_non - mean_in
    
    # T-test
    t_stat, p_val = ttest_ind(non_servizio, in_servizio)
    
    # Cohen's d
    d = cohens_d(non_servizio, in_servizio)
    
    # Significatività
    sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
    
    print(f"\n{'─'*86}")
    print(f"📊 {label}")
    print(f"{'─'*86}")
    print(f"  Non in servizio: M = {mean_non:.2f} (N = {len(non_servizio)})")
    print(f"  In servizio:     M = {mean_in:.2f} (N = {len(in_servizio)})")
    print(f"  Differenza:      Δ = {diff:+.2f}")
    print(f"  T-test:          t = {t_stat:.2f}, p = {p_val:.4f} {sig}")
    print(f"  Effect size:     Cohen's d = {d:.3f}")
    
    # Interpreta effect size
    if abs(d) < 0.2:
        effect_label = "trascurabile"
    elif abs(d) < 0.5:
        effect_label = "piccolo"
    elif abs(d) < 0.8:
        effect_label = "medio"
    else:
        effect_label = "grande"
    
    print(f"  Interpretazione: Effetto {effect_label}")
    
    # Salva risultati
    risultati_differenze.append({
        'Gruppo': 'Insegnanti',
        'Domanda': label,
        'M_gruppo1': mean_non,
        'M_gruppo2': mean_in,
        'Differenza': diff,
        't': t_stat,
        'p': p_val,
        'sig': sig,
        'Cohen_d': d,
        'Effetto': effect_label
    })

# ===== ANALISI STUDENTI =====
print("\n\n" + "="*90)
print("2. DIFFERENZE TRA STUDENTI: SECONDARIA vs UNIVERSITARI")
print("="*90)

for q in solo_studenti:
    col = q['col']
    label = q['label_it']
    
    if col not in DF_plot.columns:
        continue
    
    # Estrai dati per i due gruppi
    df_stu = DF_plot[DF_plot['GruppoDettaglio'].isin(['studenti - secondaria', 'studenti - universitari'])].copy()
    df_stu = df_stu[[col, 'GruppoDettaglio']].dropna()
    
    if len(df_stu) < 10:
        continue
    
    # Dati per gruppo - converti in float
    secondaria = pd.to_numeric(df_stu[df_stu['GruppoDettaglio'] == 'studenti - secondaria'][col], errors='coerce').dropna().values
    universitari = pd.to_numeric(df_stu[df_stu['GruppoDettaglio'] == 'studenti - universitari'][col], errors='coerce').dropna().values
    
    if len(secondaria) < 3 or len(universitari) < 3:
        continue
    
    # Statistiche descrittive
    mean_sec = np.mean(secondaria)
    mean_uni = np.mean(universitari)
    diff = mean_sec - mean_uni
    
    # T-test
    t_stat, p_val = ttest_ind(secondaria, universitari)
    
    # Cohen's d
    d = cohens_d(secondaria, universitari)
    
    # Significatività
    sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
    
    print(f"\n{'─'*86}")
    print(f"📊 {label}")
    print(f"{'─'*86}")
    print(f"  Secondaria:      M = {mean_sec:.2f} (N = {len(secondaria)})")
    print(f"  Universitari:    M = {mean_uni:.2f} (N = {len(universitari)})")
    print(f"  Differenza:      Δ = {diff:+.2f}")
    print(f"  T-test:          t = {t_stat:.2f}, p = {p_val:.4f} {sig}")
    print(f"  Effect size:     Cohen's d = {d:.3f}")
    
    # Interpreta effect size
    if abs(d) < 0.2:
        effect_label = "trascurabile"
    elif abs(d) < 0.5:
        effect_label = "piccolo"
    elif abs(d) < 0.8:
        effect_label = "medio"
    else:
        effect_label = "grande"
    
    print(f"  Interpretazione: Effetto {effect_label}")
    
    # Salva risultati
    risultati_differenze.append({
        'Gruppo': 'Studenti',
        'Domanda': label,
        'M_gruppo1': mean_sec,
        'M_gruppo2': mean_uni,
        'Differenza': diff,
        't': t_stat,
        'p': p_val,
        'sig': sig,
        'Cohen_d': d,
        'Effetto': effect_label
    })

# ===== RIEPILOGO =====
print("\n\n" + "="*90)
print("📊 RIEPILOGO DIFFERENZE SIGNIFICATIVE")
print("="*90)

df_diff = pd.DataFrame(risultati_differenze)
df_sig = df_diff[df_diff['p'] < 0.05].sort_values('p')

if len(df_sig) > 0:
    print(f"\n✓ Trovate {len(df_sig)} differenze significative (p < 0.05):\n")
    for idx, row in df_sig.iterrows():
        print(f"  • {row['Gruppo']} - {row['Domanda']}")
        print(f"    Δ = {row['Differenza']:+.2f}, p = {row['p']:.4f} {row['sig']}, d = {row['Cohen_d']:.3f} ({row['Effetto']})")
else:
    print("\n✗ Nessuna differenza significativa trovata (tutte p > 0.05)")

# Salva risultati
csv_out = OUT_EXPL / 'likert_differenze_interne_sottogruppi.csv'
df_diff.to_csv(csv_out, index=False)
print(f"\n✓ Risultati salvati: {csv_out}")

print("\n" + "="*90)

## 📊 Sintesi Analisi Differenze Interne

### Risultati Principali

#### ✅ **INSEGNANTI** (non in servizio vs in servizio)
**Nessuna differenza significativa** tra i due sottogruppi:
- **Cambierà la didattica generale**: Δ = +0.07, p = 0.634 (ns)
- **Cambierà la mia didattica**: Δ = +0.34, p = 0.051 (marginale, quasi significativo!)
- **Fiducia uso responsabile studenti**: Δ = +0.29, p = 0.105 (ns)
- **Preoccupazione uso IA studenti**: Δ = -0.09, p = 0.608 (ns)

**Interpretazione**: Gli insegnanti non in servizio e in servizio hanno **atteggiamenti molto simili** verso l'IA. L'unica differenza borderline è sul "Cambierà la mia didattica" dove i non in servizio sono leggermente più ottimisti (5.02 vs 4.68, effetto piccolo d=0.22).

---

#### ✅ **STUDENTI** (secondaria vs universitari)
**Due differenze significative**:
1. **Preparazione insegnanti su IA**: ⭐⭐
   - Secondaria: M = 3.14
   - Universitari: M = 3.70
   - Δ = -0.57, p = **0.009**, d = -0.34 (effetto piccolo)
   - **Gli universitari giudicano i loro insegnanti leggermente più preparati**

2. **Preoccupazione uso IA compagni**: ⭐⭐
   - Secondaria: M = 2.91
   - Universitari: M = 3.53
   - Δ = -0.63, p = **0.005**, d = -0.36 (effetto piccolo)
   - **Gli universitari sono più preoccupati per l'uso IA dei compagni**

---

### 🎯 Conclusioni

**Hai ragione nella tua osservazione!** Tuttavia:
- **Insegnanti**: Non ci sono differenze significative, anche se c'è una tendenza borderline (p=0.051) sul "Cambierà la mia didattica"
- **Studenti**: Ci sono differenze significative ma con effetti piccoli (d ≈ 0.3-0.4)

**Il risultato più importante**: I **sottogruppi sono relativamente omogenei** all'interno di ciascuna categoria principale (insegnanti/studenti). Le differenze maggiori rimangono quelle **tra** le categorie principali (studenti vs insegnanti), non **dentro** le categorie.

---
### 🔍 Approfondimento: Fiducia e Preoccupazione negli Insegnanti

Focus su due domande chiave:
1. **Fiducia uso responsabile studenti**: Gli insegnanti hanno fiducia che gli studenti usino l'IA responsabilmente?
2. **Preoccupazione uso IA studenti**: Gli insegnanti sono preoccupati per l'uso IA degli studenti?

Verifichiamo se ci sono differenze tra insegnanti **non in servizio** vs **in servizio**.

In [ ]:
# === ANALISI DETTAGLIATA: FIDUCIA E PREOCCUPAZIONE INSEGNANTI ===

from scipy.stats import ttest_ind, mannwhitneyu, levene
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("="*90)
print("ANALISI DETTAGLIATA: FIDUCIA E PREOCCUPAZIONE NEGLI INSEGNANTI")
print("="*90)

# Trova le due domande specifiche
domanda_fiducia = None
domanda_preoccupazione = None

for q in solo_insegnanti:
    if 'Fiducia uso responsabile studenti' in q['label_it']:
        domanda_fiducia = q
    if 'Preoccupazione uso IA studenti' in q['label_it']:
        domanda_preoccupazione = q

# === ANALISI 1: FIDUCIA USO RESPONSABILE ===
if domanda_fiducia:
    col = domanda_fiducia['col']
    
    print("\n" + "="*90)
    print("1️⃣  FIDUCIA NELL'USO RESPONSABILE DA PARTE DEGLI STUDENTI")
    print("="*90)
    print("\nDomanda: 'Quanto sei fiducioso nell'utilizzo da parte degli studenti")
    print("         di un uso responsabile e maturo dell'intelligenza artificiale?'")
    print("         (1 = Per niente fiducioso ... 7 = Estremamente fiducioso)")
    
    # Estrai dati
    df_ins = DF_plot[DF_plot['GruppoDettaglio'].isin(['insegnanti - non in servizio', 'insegnanti - in servizio'])].copy()
    df_ins = df_ins[[col, 'GruppoDettaglio']].dropna()
    df_ins[col] = pd.to_numeric(df_ins[col], errors='coerce')
    df_ins = df_ins.dropna()
    
    # Dati per gruppo
    non_servizio = df_ins[df_ins['GruppoDettaglio'] == 'insegnanti - non in servizio'][col].values
    in_servizio = df_ins[df_ins['GruppoDettaglio'] == 'insegnanti - in servizio'][col].values
    
    # Statistiche descrittive complete
    print(f"\n{'─'*86}")
    print("📊 STATISTICHE DESCRITTIVE")
    print(f"{'─'*86}")
    
    print(f"\n  Insegnanti NON IN SERVIZIO (N = {len(non_servizio)}):")
    print(f"    Media:    {np.mean(non_servizio):.2f}")
    print(f"    Mediana:  {np.median(non_servizio):.2f}")
    print(f"    Dev.Std:  {np.std(non_servizio, ddof=1):.2f}")
    print(f"    Min-Max:  {np.min(non_servizio):.0f} - {np.max(non_servizio):.0f}")
    
    print(f"\n  Insegnanti IN SERVIZIO (N = {len(in_servizio)}):")
    print(f"    Media:    {np.mean(in_servizio):.2f}")
    print(f"    Mediana:  {np.median(in_servizio):.2f}")
    print(f"    Dev.Std:  {np.std(in_servizio, ddof=1):.2f}")
    print(f"    Min-Max:  {np.min(in_servizio):.0f} - {np.max(in_servizio):.0f}")
    
    # Differenza
    diff = np.mean(non_servizio) - np.mean(in_servizio)
    print(f"\n  Differenza (non servizio - in servizio): {diff:+.2f}")
    
    # Test di Levene per omogeneità delle varianze
    lev_stat, lev_p = levene(non_servizio, in_servizio)
    print(f"\n{'─'*86}")
    print("📊 TEST DI OMOGENEITÀ VARIANZE (Levene)")
    print(f"{'─'*86}")
    print(f"  Statistica: {lev_stat:.3f}")
    print(f"  p-value:    {lev_p:.4f}")
    print(f"  Risultato:  {'Varianze omogenee' if lev_p > 0.05 else 'Varianze NON omogenee'}")
    
    # T-test
    t_stat, p_val = ttest_ind(non_servizio, in_servizio)
    
    # Cohen's d
    n1, n2 = len(non_servizio), len(in_servizio)
    var1, var2 = np.var(non_servizio, ddof=1), np.var(in_servizio, ddof=1)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    d = diff / pooled_std
    
    print(f"\n{'─'*86}")
    print("📊 TEST T DI STUDENT")
    print(f"{'─'*86}")
    print(f"  t-statistic: {t_stat:.3f}")
    print(f"  p-value:     {p_val:.4f}")
    
    if p_val < 0.001:
        sig = "*** ALTAMENTE SIGNIFICATIVO"
    elif p_val < 0.01:
        sig = "** MOLTO SIGNIFICATIVO"
    elif p_val < 0.05:
        sig = "* SIGNIFICATIVO"
    elif p_val < 0.10:
        sig = "(·) MARGINALMENTE SIGNIFICATIVO"
    else:
        sig = "NON SIGNIFICATIVO"
    
    print(f"  Significatività: {sig}")
    print(f"\n  Cohen's d:   {d:.3f}")
    
    if abs(d) < 0.2:
        effect_label = "TRASCURABILE"
    elif abs(d) < 0.5:
        effect_label = "PICCOLO"
    elif abs(d) < 0.8:
        effect_label = "MEDIO"
    else:
        effect_label = "GRANDE"
    
    print(f"  Effect size: {effect_label}")
    
    print(f"\n{'─'*86}")
    print("💡 INTERPRETAZIONE")
    print(f"{'─'*86}")
    if p_val >= 0.05:
        print("  ❌ NON ci sono differenze statisticamente significative tra i due gruppi.")
        print(f"     Entrambi i gruppi hanno un livello SIMILE di fiducia negli studenti.")
        print(f"     (Fiducia media: ~{np.mean(df_ins[col]):.2f} su scala 1-7, quindi BASSA)")
    else:
        print(f"  ✅ CI SONO differenze statisticamente significative (p = {p_val:.4f})!")
        if diff > 0:
            print(f"     Gli insegnanti NON in servizio hanno PIÙ fiducia (+{diff:.2f} punti)")
        else:
            print(f"     Gli insegnanti IN servizio hanno PIÙ fiducia (+{abs(diff):.2f} punti)")

# === ANALISI 2: PREOCCUPAZIONE USO IA ===
if domanda_preoccupazione:
    col = domanda_preoccupazione['col']
    
    print("\n\n" + "="*90)
    print("2️⃣  PREOCCUPAZIONE SULL'USO IA DA PARTE DEGLI STUDENTI")
    print("="*90)
    print("\nDomanda: 'Quanto sei preoccupato riguardo all'utilizzo dell'intelligenza")
    print("         artificiale da parte degli studenti?'")
    print("         (1 = Per niente preoccupato ... 7 = Estremamente preoccupato)")
    
    # Estrai dati
    df_ins = DF_plot[DF_plot['GruppoDettaglio'].isin(['insegnanti - non in servizio', 'insegnanti - in servizio'])].copy()
    df_ins = df_ins[[col, 'GruppoDettaglio']].dropna()
    df_ins[col] = pd.to_numeric(df_ins[col], errors='coerce')
    df_ins = df_ins.dropna()
    
    # Dati per gruppo
    non_servizio = df_ins[df_ins['GruppoDettaglio'] == 'insegnanti - non in servizio'][col].values
    in_servizio = df_ins[df_ins['GruppoDettaglio'] == 'insegnanti - in servizio'][col].values
    
    # Statistiche descrittive complete
    print(f"\n{'─'*86}")
    print("📊 STATISTICHE DESCRITTIVE")
    print(f"{'─'*86}")
    
    print(f"\n  Insegnanti NON IN SERVIZIO (N = {len(non_servizio)}):")
    print(f"    Media:    {np.mean(non_servizio):.2f}")
    print(f"    Mediana:  {np.median(non_servizio):.2f}")
    print(f"    Dev.Std:  {np.std(non_servizio, ddof=1):.2f}")
    print(f"    Min-Max:  {np.min(non_servizio):.0f} - {np.max(non_servizio):.0f}")
    
    print(f"\n  Insegnanti IN SERVIZIO (N = {len(in_servizio)}):")
    print(f"    Media:    {np.mean(in_servizio):.2f}")
    print(f"    Mediana:  {np.median(in_servizio):.2f}")
    print(f"    Dev.Std:  {np.std(in_servizio, ddof=1):.2f}")
    print(f"    Min-Max:  {np.min(in_servizio):.0f} - {np.max(in_servizio):.0f}")
    
    # Differenza
    diff = np.mean(non_servizio) - np.mean(in_servizio)
    print(f"\n  Differenza (non servizio - in servizio): {diff:+.2f}")
    
    # Test di Levene
    lev_stat, lev_p = levene(non_servizio, in_servizio)
    print(f"\n{'─'*86}")
    print("📊 TEST DI OMOGENEITÀ VARIANZE (Levene)")
    print(f"{'─'*86}")
    print(f"  Statistica: {lev_stat:.3f}")
    print(f"  p-value:    {lev_p:.4f}")
    print(f"  Risultato:  {'Varianze omogenee' if lev_p > 0.05 else 'Varianze NON omogenee'}")
    
    # T-test
    t_stat, p_val = ttest_ind(non_servizio, in_servizio)
    
    # Cohen's d
    n1, n2 = len(non_servizio), len(in_servizio)
    var1, var2 = np.var(non_servizio, ddof=1), np.var(in_servizio, ddof=1)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    d = diff / pooled_std
    
    print(f"\n{'─'*86}")
    print("📊 TEST T DI STUDENT")
    print(f"{'─'*86}")
    print(f"  t-statistic: {t_stat:.3f}")
    print(f"  p-value:     {p_val:.4f}")
    
    if p_val < 0.001:
        sig = "*** ALTAMENTE SIGNIFICATIVO"
    elif p_val < 0.01:
        sig = "** MOLTO SIGNIFICATIVO"
    elif p_val < 0.05:
        sig = "* SIGNIFICATIVO"
    elif p_val < 0.10:
        sig = "(·) MARGINALMENTE SIGNIFICATIVO"
    else:
        sig = "NON SIGNIFICATIVO"
    
    print(f"  Significatività: {sig}")
    print(f"\n  Cohen's d:   {d:.3f}")
    
    if abs(d) < 0.2:
        effect_label = "TRASCURABILE"
    elif abs(d) < 0.5:
        effect_label = "PICCOLO"
    elif abs(d) < 0.8:
        effect_label = "MEDIO"
    else:
        effect_label = "GRANDE"
    
    print(f"  Effect size: {effect_label}")
    
    print(f"\n{'─'*86}")
    print("💡 INTERPRETAZIONE")
    print(f"{'─'*86}")
    if p_val >= 0.05:
        print("  ❌ NON ci sono differenze statisticamente significative tra i due gruppi.")
        print(f"     Entrambi i gruppi hanno un livello SIMILE di preoccupazione.")
        print(f"     (Preoccupazione media: ~{np.mean(df_ins[col]):.2f} su scala 1-7, quindi MODERATA)")
    else:
        print(f"  ✅ CI SONO differenze statisticamente significative (p = {p_val:.4f})!")
        if diff > 0:
            print(f"     Gli insegnanti NON in servizio sono PIÙ preoccupati (+{diff:.2f} punti)")
        else:
            print(f"     Gli insegnanti IN servizio sono PIÙ preoccupati (+{abs(diff):.2f} punti)")

# === RIEPILOGO FINALE ===
print("\n\n" + "="*90)
print("🎯 RIEPILOGO FINALE")
print("="*90)
print("\n📌 Le due domande sono legate:")
print("   • Bassa FIDUCIA nell'uso responsabile")
print("   • Alta PREOCCUPAZIONE sull'uso IA")
print("\n📌 Risultato:")
print("   ❌ NON ci sono differenze significative tra insegnanti in servizio e non")
print("   ✅ Entrambi i gruppi condividono:")
print("      - BASSA fiducia negli studenti (~3.2-3.5 su 7)")
print("      - ALTA preoccupazione (~4.6-4.7 su 7)")
print("\n💡 Conclusione: È un atteggiamento TRASVERSALE al mondo degli insegnanti,")
print("   indipendentemente dal fatto che siano già in servizio o meno.")
print("="*90)

In [ ]:
# === GRAFICI A VIOLINO: DOMANDE SOLO STUDENTI ===

print("\n" + "="*90)
print("GRAFICI A VIOLINO: DOMANDE SOLO STUDENTI")
print("="*90)

# Solo i 2 gruppi di studenti
order_studenti = ['studenti - secondaria', 'studenti - universitari']
palette_studenti = {
    'studenti - secondaria': palette_4groups['studenti - secondaria'],
    'studenti - universitari': palette_4groups['studenti - universitari']
}

for q in solo_studenti:
    col = q['col']
    label_it = q['label_it']
    label_en = q['label_en']
    tema = q['tema']
    
    if col not in DF_plot.columns:
        print(f"\n⚠️ Colonna non trovata: {label_it}")
        continue
    
    # Prepara dati - solo studenti
    df_temp = DF_plot[DF_plot['GruppoDettaglio'].isin(order_studenti)].copy()
    df_temp = df_temp[[col, 'GruppoDettaglio']].dropna()
    
    if len(df_temp) == 0:
        print(f"\n⚠️ Nessun dato per: {label_it}")
        continue
        
    df_vis = df_temp.copy()
    df_vis.columns = ['valore', 'GruppoDettaglio']
    # Rimuovi categorie non usate e riordina
    df_vis['GruppoDettaglio'] = pd.Categorical(
        df_vis['GruppoDettaglio'], 
        categories=order_studenti, 
        ordered=True
    )
    
    # Statistiche
    stats = df_vis.groupby('GruppoDettaglio', observed=True)['valore'].agg(['count', 'mean', 'median', 'std', 'min', 'max'])
    stats.columns = ['N', 'Media', 'Mediana', 'SD', 'Min', 'Max']
    
    print(f"\n{'─'*86}")
    print(f"📊 {label_it}")
    print(f"{'─'*86}")
    print(f"\n  Statistiche per gruppo:")
    print(stats.to_string())
    
    # Crea grafico doppio (IT + EN)
    fig, (ax_it, ax_en) = plt.subplots(1, 2, figsize=(14, 5))
    
    # ITALIANO
    sns.violinplot(
        data=df_vis,
        x='GruppoDettaglio',
        y='valore',
        hue='GruppoDettaglio',
        palette=palette_studenti,
        ax=ax_it,
        legend=False,
        inner='box'
    )
    ax_it.set_xlabel('Gruppo', fontsize=11)
    ax_it.set_ylabel('Punteggio (scala 1-7)', fontsize=11)
    ax_it.set_title(f'{label_it}\n(scala Likert 1-7)', fontsize=12, weight='bold')
    ax_it.set_ylim(0.5, 7.5)
    ax_it.set_xticklabels(ax_it.get_xticklabels(), rotation=15, ha='right')
    ax_it.grid(axis='y', alpha=0.3)
    
    # INGLESE
    order_en_stu = ['students - secondary', 'students - university']
    df_vis_en = df_vis.copy()
    df_vis_en['GruppoDettaglio'] = df_vis_en['GruppoDettaglio'].cat.rename_categories({
        'studenti - secondaria': 'students - secondary',
        'studenti - universitari': 'students - university'
    })
    palette_en_stu = {
        'students - secondary': palette_studenti['studenti - secondaria'],
        'students - university': palette_studenti['studenti - universitari']
    }
    
    sns.violinplot(
        data=df_vis_en,
        x='GruppoDettaglio',
        y='valore',
        hue='GruppoDettaglio',
        palette=palette_en_stu,
        ax=ax_en,
        legend=False,
        inner='box'
    )
    ax_en.set_xlabel('Group', fontsize=11)
    ax_en.set_ylabel('Score (1-7 scale)', fontsize=11)
    ax_en.set_title(f'{label_en}\n(Likert scale 1-7)', fontsize=12, weight='bold')
    ax_en.set_ylim(0.5, 7.5)
    ax_en.set_xticklabels(ax_en.get_xticklabels(), rotation=15, ha='right')
    ax_en.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Salva
    out_it = OUT_EXPL / f'likert_{tema}_studenti_it.png'
    out_en = OUT_EXPL / f'likert_{tema}_studenti_en.png'
    
    fig.savefig(out_it, dpi=300, bbox_inches='tight')
    print(f"\n  ✓ Salvato: {out_it}")
    
    fig.savefig(out_en, dpi=300, bbox_inches='tight')
    print(f"  ✓ Salvato: {out_en}")
    
plt.show()
    plt.close(fig)

print(f"\n{'='*90}")
print(f"✓ Grafici studenti completati!")
print(f"{'='*90}")

---
## 🔍 Confronto Preoccupazione: Insegnanti vs Studenti (4 gruppi)

Analizziamo la **preoccupazione generale** sull'inserimento dell'IA nell'educazione confrontando:
- I 4 gruppi principali (studenti-secondaria, studenti-universitari, insegnanti-non servizio, insegnanti-in servizio)
- Differenze tra studenti e insegnanti
- Differenze interne ai sottogruppi

In [ ]:
# === ANALISI PREOCCUPAZIONE GENERALE: CONFRONTO TRA TUTTI I GRUPPI ===

from scipy.stats import f_oneway, kruskal, ttest_ind
import numpy as np
import pandas as pd

print("="*90)
print("ANALISI PREOCCUPAZIONE GENERALE SULL'IA NELL'EDUCAZIONE")
print("="*90)

# Trova la domanda sulla preoccupazione generale
# Cerca direttamente nel dizionario found_questions
col_preoccupazione_ins = found_questions.get('preoccupazione_educazione_ins')
col_preoccupazione_stu = found_questions.get('preoccupazione_scuola_stu')

if col_preoccupazione_ins and col_preoccupazione_stu:
    print(f"\n✓ Domanda insegnanti: {col_preoccupazione_ins}")
    print(f"✓ Domanda studenti: {col_preoccupazione_stu}")
    
    # Prepara i dati per tutti i 4 gruppi
    df_analisi = DF_plot[['GruppoDettaglio', col_preoccupazione_ins, col_preoccupazione_stu]].copy()
    
    # Crea una colonna unificata con la preoccupazione
    df_analisi['Preoccupazione'] = np.nan
    
    # Per gli insegnanti, usa la domanda insegnanti
    mask_ins = df_analisi['GruppoDettaglio'].str.contains('insegnanti', case=False, na=False)
    df_analisi.loc[mask_ins, 'Preoccupazione'] = pd.to_numeric(
        df_analisi.loc[mask_ins, col_preoccupazione_ins], errors='coerce'
    )
    
    # Per gli studenti, usa la domanda studenti
    mask_stu = df_analisi['GruppoDettaglio'].str.contains('studenti', case=False, na=False)
    df_analisi.loc[mask_stu, 'Preoccupazione'] = pd.to_numeric(
        df_analisi.loc[mask_stu, col_preoccupazione_stu], errors='coerce'
    )
    
    # Rimuovi i valori mancanti
    df_analisi = df_analisi[['GruppoDettaglio', 'Preoccupazione']].dropna()
    
    print(f"\n📊 Campione totale: N = {len(df_analisi)}")
    
    # ===== 1. STATISTICHE DESCRITTIVE PER TUTTI I 4 GRUPPI =====
    print("\n" + "="*90)
    print("1. STATISTICHE DESCRITTIVE PER GRUPPO")
    print("="*90)
    
    stats_preoccupazione = []
    
    for gruppo in ORDER_4:
        dati = df_analisi[df_analisi['GruppoDettaglio'] == gruppo]['Preoccupazione'].values
        
        if len(dati) > 0:
            stats_preoccupazione.append({
                'Gruppo': gruppo,
                'N': len(dati),
                'Media': np.mean(dati),
                'Mediana': np.median(dati),
                'SD': np.std(dati, ddof=1),
                'Min': np.min(dati),
                'Max': np.max(dati)
            })
            
            print(f"\n{gruppo}:")
            print(f"  N = {len(dati)}")
            print(f"  Media = {np.mean(dati):.2f}")
            print(f"  Mediana = {np.median(dati):.0f}")
            print(f"  SD = {np.std(dati, ddof=1):.2f}")
            print(f"  Range = {np.min(dati):.0f} - {np.max(dati):.0f}")
    
    df_stats_preocc = pd.DataFrame(stats_preoccupazione)
    
    # ===== 2. ANOVA TRA I 4 GRUPPI =====
    print("\n" + "="*90)
    print("2. ANOVA: CONFRONTO TRA TUTTI I 4 GRUPPI")
    print("="*90)
    
    # Estrai i dati per gruppo
    gruppo_dati = []
    for gruppo in ORDER_4:
        dati = df_analisi[df_analisi['GruppoDettaglio'] == gruppo]['Preoccupazione'].values
        gruppo_dati.append(dati)
    
    # ANOVA
    f_stat, p_anova = f_oneway(*gruppo_dati)
    
    print(f"\nF-statistic = {f_stat:.3f}")
    print(f"p-value = {p_anova:.6f}")
    
    if p_anova < 0.001:
        sig_anova = "*** (p < 0.001)"
    elif p_anova < 0.01:
        sig_anova = "** (p < 0.01)"
    elif p_anova < 0.05:
        sig_anova = "* (p < 0.05)"
    else:
        sig_anova = "ns (p ≥ 0.05)"
    
    print(f"Significatività: {sig_anova}")
    
    if p_anova < 0.05:
        print("\n✓ Ci sono DIFFERENZE SIGNIFICATIVE tra i 4 gruppi")
    else:
        print("\n✗ NON ci sono differenze significative tra i 4 gruppi")
    
    # ===== 3. CONFRONTO MACRO: STUDENTI vs INSEGNANTI =====
    print("\n" + "="*90)
    print("3. CONFRONTO MACRO: TUTTI GLI STUDENTI vs TUTTI GLI INSEGNANTI")
    print("="*90)
    
    # Dati studenti (secondaria + universitari)
    studenti_data = df_analisi[df_analisi['GruppoDettaglio'].str.contains('studenti', case=False, na=False)]['Preoccupazione'].values
    
    # Dati insegnanti (non in servizio + in servizio)
    insegnanti_data = df_analisi[df_analisi['GruppoDettaglio'].str.contains('insegnanti', case=False, na=False)]['Preoccupazione'].values
    
    print(f"\nStudenti: N = {len(studenti_data)}, M = {np.mean(studenti_data):.2f}, SD = {np.std(studenti_data, ddof=1):.2f}")
    print(f"Insegnanti: N = {len(insegnanti_data)}, M = {np.mean(insegnanti_data):.2f}, SD = {np.std(insegnanti_data, ddof=1):.2f}")
    
    diff_macro = np.mean(studenti_data) - np.mean(insegnanti_data)
    print(f"\nDifferenza: Δ = {diff_macro:+.2f}")
    
    # T-test
    t_macro, p_macro = ttest_ind(studenti_data, insegnanti_data)
    
    # Cohen's d
    def cohens_d(group1, group2):
        n1, n2 = len(group1), len(group2)
        var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
        pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
        return (np.mean(group1) - np.mean(group2)) / pooled_std
    
    d_macro = cohens_d(studenti_data, insegnanti_data)
    
    print(f"\nT-test: t = {t_macro:.3f}, p = {p_macro:.6f}")
    print(f"Cohen's d = {d_macro:.3f}")
    
    if abs(d_macro) < 0.2:
        effect_macro = "trascurabile"
    elif abs(d_macro) < 0.5:
        effect_macro = "piccolo"
    elif abs(d_macro) < 0.8:
        effect_macro = "medio"
    else:
        effect_macro = "grande"
    
    print(f"Effect size: {effect_macro}")
    
    if p_macro < 0.001:
        sig_macro = "***"
        print("\n✓ Differenza ALTAMENTE SIGNIFICATIVA tra studenti e insegnanti")
    elif p_macro < 0.01:
        sig_macro = "**"
        print("\n✓ Differenza MOLTO SIGNIFICATIVA tra studenti e insegnanti")
    elif p_macro < 0.05:
        sig_macro = "*"
        print("\n✓ Differenza SIGNIFICATIVA tra studenti e insegnanti")
    else:
        sig_macro = "ns"
        print("\n✗ NON ci sono differenze significative tra studenti e insegnanti")
    
    # ===== 4. POST-HOC: CONFRONTI A COPPIE =====
    print("\n" + "="*90)
    print("4. POST-HOC: CONFRONTI A COPPIE TRA I 4 GRUPPI")
    print("="*90)
    
    posthoc_preocc = []
    
    for i, grp1 in enumerate(ORDER_4):
        for j, grp2 in enumerate(ORDER_4):
            if i < j:  # Evita confronti duplicati
                data1 = df_analisi[df_analisi['GruppoDettaglio'] == grp1]['Preoccupazione'].values
                data2 = df_analisi[df_analisi['GruppoDettaglio'] == grp2]['Preoccupazione'].values
                
                if len(data1) >= 3 and len(data2) >= 3:
                    mean1 = np.mean(data1)
                    mean2 = np.mean(data2)
                    diff = mean1 - mean2
                    
                    t, p = ttest_ind(data1, data2)
                    d = cohens_d(data1, data2)
                    
                    if p < 0.001:
                        sig = "***"
                    elif p < 0.01:
                        sig = "**"
                    elif p < 0.05:
                        sig = "*"
                    else:
                        sig = "ns"
                    
                    if abs(d) < 0.2:
                        effect = "trascurabile"
                    elif abs(d) < 0.5:
                        effect = "piccolo"
                    elif abs(d) < 0.8:
                        effect = "medio"
                    else:
                        effect = "grande"
                    
                    posthoc_preocc.append({
                        'Gruppo_1': grp1,
                        'Gruppo_2': grp2,
                        'M1': mean1,
                        'M2': mean2,
                        'Differenza': diff,
                        't': t,
                        'p': p,
                        'sig': sig,
                        'Cohen_d': d,
                        'Effetto': effect
                    })
                    
                    print(f"\n{grp1} vs {grp2}:")
                    print(f"  M1 = {mean1:.2f}, M2 = {mean2:.2f}, Δ = {diff:+.2f}")
                    print(f"  t = {t:.3f}, p = {p:.6f} {sig}")
                    print(f"  Cohen's d = {d:.3f} ({effect})")
    
    df_posthoc_preocc = pd.DataFrame(posthoc_preocc)
    
    # ===== 5. SINTESI E INTERPRETAZIONE =====
    print("\n" + "="*90)
    print("5. SINTESI E INTERPRETAZIONE")
    print("="*90)
    
    print("\n📊 MEDIE PER GRUPPO (scala 1-7):")
    for _, row in df_stats_preocc.iterrows():
        print(f"  {row['Gruppo']}: M = {row['Media']:.2f} (SD = {row['SD']:.2f})")
    
    print(f"\n🔍 ANOVA: F = {f_stat:.3f}, p = {p_anova:.6f} {sig_anova}")
    
    print(f"\n📈 CONFRONTO MACRO:")
    print(f"  Studenti:    M = {np.mean(studenti_data):.2f}")
    print(f"  Insegnanti:  M = {np.mean(insegnanti_data):.2f}")
    print(f"  Differenza:  Δ = {diff_macro:+.2f} {sig_macro}")
    print(f"  Effect size: Cohen's d = {d_macro:.3f} ({effect_macro})")
    
    # Trova i confronti più significativi
    if len(df_posthoc_preocc) > 0:
        sig_comparisons = df_posthoc_preocc[df_posthoc_preocc['sig'] != 'ns'].sort_values('p')
        
        if len(sig_comparisons) > 0:
            print("\n🎯 CONFRONTI SIGNIFICATIVI (post-hoc):")
            for _, row in sig_comparisons.iterrows():
                print(f"  {row['Gruppo_1']} vs {row['Gruppo_2']}: Δ = {row['Differenza']:+.2f} {row['sig']}, d = {row['Cohen_d']:.3f}")
        else:
            print("\n✗ Nessun confronto post-hoc significativo")
    
    # Interpretazione finale
    print("\n" + "="*90)
    print("💡 INTERPRETAZIONE FINALE")
    print("="*90)
    
    if p_anova < 0.05:
        print("\n✓ Ci sono differenze significative nella preoccupazione tra i gruppi.")
        
        # Chi è più preoccupato?
        gruppo_max = df_stats_preocc.loc[df_stats_preocc['Media'].idxmax(), 'Gruppo']
        media_max = df_stats_preocc['Media'].max()
        gruppo_min = df_stats_preocc.loc[df_stats_preocc['Media'].idxmin(), 'Gruppo']
        media_min = df_stats_preocc['Media'].min()
        
        print(f"\n  PIÙ PREOCCUPATI: {gruppo_max} (M = {media_max:.2f})")
        print(f"  MENO PREOCCUPATI: {gruppo_min} (M = {media_min:.2f})")
        
        if p_macro < 0.05:
            if diff_macro > 0:
                print(f"\n  → Gli STUDENTI sono significativamente PIÙ preoccupati degli insegnanti")
            else:
                print(f"\n  → Gli INSEGNANTI sono significativamente PIÙ preoccupati degli studenti")
        else:
            print(f"\n  → NON ci sono differenze significative tra studenti e insegnanti nel complesso")
            print(f"    (le differenze sono principalmente tra sottogruppi)")
    else:
        print("\n✗ NON ci sono differenze significative nella preoccupazione tra i gruppi.")
        print(f"  Tutti i gruppi mostrano un livello di preoccupazione simile (~{df_stats_preocc['Media'].mean():.1f}/7)")
    
    # Salva risultati
    csv_stats = OUT_EXPL / 'preoccupazione_stats_4gruppi.csv'
    df_stats_preocc.to_csv(csv_stats, index=False)
    print(f"\n✓ Statistiche salvate: {csv_stats}")
    
    if len(df_posthoc_preocc) > 0:
        csv_posthoc = OUT_EXPL / 'preoccupazione_posthoc_4gruppi.csv'
        df_posthoc_preocc.to_csv(csv_posthoc, index=False)
        print(f"✓ Post-hoc salvato: {csv_posthoc}")

else:
    print("\n⚠️ Domanda sulla preoccupazione generale non trovata!")

print("\n" + "="*90)

In [ ]:
# === VISUALIZZAZIONE: PREOCCUPAZIONE PER GRUPPO ===

import matplotlib.pyplot as plt
import seaborn as sns

print("="*90)
print("VISUALIZZAZIONE: VIOLIN PLOT PREOCCUPAZIONE GENERALE")
print("="*90)

if col_preoccupazione_ins and col_preoccupazione_stu:
    # Prepara i dati - filtra solo i 4 gruppi principali
    df_vis_preocc = df_analisi[df_analisi['GruppoDettaglio'].isin(ORDER_4)][['GruppoDettaglio', 'Preoccupazione']].copy()
    
    # === GRAFICO ITALIANO ===
    fig_it, ax_it = plt.subplots(1, 1, figsize=(12, 7))
    
    # Create color list based on order
    colors_ordered = [palette_4groups[g] for g in ORDER_4]
    
    sns.violinplot(
        data=df_vis_preocc,
        x='GruppoDettaglio',
        y='Preoccupazione',
        order=ORDER_4,
        palette=colors_ordered,
        ax=ax_it,
        inner='box'
    )
    
    ax_it.set_xlabel('Gruppo', fontsize=13, fontweight='bold')
    ax_it.set_ylabel('Preoccupazione (scala 1-7)', fontsize=13, fontweight='bold')
    ax_it.set_title('Preoccupazione generale sull\'inserimento dell\'IA nell\'educazione', 
                    fontsize=15, fontweight='bold', pad=20)
    
    # Aggiungi linee di riferimento
    ax_it.axhline(y=4, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    ax_it.text(3.5, 4.1, 'Neutrale (4)', fontsize=9, color='gray', ha='right')
    
    # Ruota le etichette
    ax_it.set_xticklabels(ax_it.get_xticklabels(), rotation=15, ha='right')
    ax_it.grid(axis='y', alpha=0.3, linestyle=':')
    
    plt.tight_layout()
    
    # Salva
    out_it_png = OUT_EXPL / 'preoccupazione_generale_4gruppi_it.png'
    fig_it.savefig(out_it_png, dpi=300, bbox_inches='tight')
    print(f"\n✓ Salvato: {out_it_png}")
    
    plt.show()
    plt.close(fig_it)
    
    # === GRAFICO INGLESE ===
    fig_en, ax_en = plt.subplots(1, 1, figsize=(12, 7))
    
    # Map dei nomi in inglese
    df_vis_preocc_en = df_vis_preocc.copy()
    map_en = {
        'studenti - secondaria': 'students - secondary',
        'studenti - universitari': 'students - university',
        'insegnanti - non in servizio': 'teachers - pre-service',
        'insegnanti - in servizio': 'teachers - in-service'
    }
    df_vis_preocc_en['Group'] = df_vis_preocc_en['GruppoDettaglio'].map(map_en)
    order_en = [map_en[g] for g in ORDER_4]
    palette_en = {map_en[k]: v for k, v in palette_4groups.items()}
    
    # Create color list based on order
    colors_ordered_en = [palette_en[g] for g in order_en]
    
    sns.violinplot(
        data=df_vis_preocc_en,
        x='Group',
        y='Preoccupazione',
        order=order_en,
        palette=colors_ordered_en,
        ax=ax_en,
        inner='box'
    )
    
    ax_en.set_xlabel('Group', fontsize=13, fontweight='bold')
    ax_en.set_ylabel('Concern (scale 1-7)', fontsize=13, fontweight='bold')
    ax_en.set_title('General concern about AI integration in education', 
                    fontsize=15, fontweight='bold', pad=20)
    
    # Aggiungi linee di riferimento
    ax_en.axhline(y=4, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    ax_en.text(3.5, 4.1, 'Neutral (4)', fontsize=9, color='gray', ha='right')
    
    # Ruota le etichette
    ax_en.set_xticklabels(ax_en.get_xticklabels(), rotation=15, ha='right')
    ax_en.grid(axis='y', alpha=0.3, linestyle=':')
    
    plt.tight_layout()
    
    # Salva
    out_en_png = OUT_EXPL / 'preoccupazione_generale_4gruppi_en.png'
    fig_en.savefig(out_en_png, dpi=300, bbox_inches='tight')
    print(f"✓ Salvato: {out_en_png}")
    
    plt.show()
    plt.close(fig_en)
    
    print("\n✓ Grafici completati")

print("\n" + "="*90)

---
## 📋 Sintesi Finale: Differenze nella Preoccupazione tra Insegnanti e Studenti

### 🎯 Risultati Principali

**1. DIFFERENZE ALTAMENTE SIGNIFICATIVE TRA I 4 GRUPPI**
- ANOVA: F = 19.723, p < 0.001 ***
- Ci sono differenze molto significative nella preoccupazione generale sull'IA tra i 4 gruppi

**2. CONFRONTO MACRO: INSEGNANTI vs STUDENTI**
- **Studenti**: M = 3.31/7 (bassa preoccupazione)
- **Insegnanti**: M = 4.24/7 (preoccupazione moderata)
- **Differenza**: Δ = -0.93 punti
- **Significatività**: p < 0.001 *** (altamente significativa)
- **Effect size**: Cohen's d = -0.583 (effetto **medio**)

**3. DETTAGLIO PER GRUPPO**
- **Studenti secondaria**: M = 3.16 (i MENO preoccupati)
- **Studenti universitari**: M = 3.39
- **Insegnanti non in servizio**: M = 4.30 (i PIÙ preoccupati)
- **Insegnanti in servizio**: M = 4.22

**4. CONFRONTI POST-HOC SIGNIFICATIVI**

Tutti i confronti **studenti vs insegnanti** sono altamente significativi (p < 0.001):
- Studenti secondaria vs Insegnanti in servizio: Δ = -1.06, d = -0.662 (medio)
- Studenti secondaria vs Insegnanti non servizio: Δ = -1.15, d = -0.731 (medio)
- Studenti universitari vs Insegnanti in servizio: Δ = -0.84, d = -0.517 (medio)
- Studenti universitari vs Insegnanti non servizio: Δ = -0.92, d = -0.575 (medio)

I confronti **all'interno della stessa categoria** NON sono significativi:
- Studenti secondaria vs universitari: p = 0.285 (ns)
- Insegnanti non servizio vs in servizio: p = 0.641 (ns)

### 💡 Interpretazione

1. **Gli INSEGNANTI sono significativamente PIÙ PREOCCUPATI degli STUDENTI** riguardo l'inserimento dell'IA nell'educazione

2. **La differenza è TRASVERSALE**: non dipende dal sottogruppo
   - Tra studenti (secondaria vs universitari): nessuna differenza significativa
   - Tra insegnanti (pre-service vs in-service): nessuna differenza significativa
   
3. **La differenza è di RUOLO**, non di esperienza:
   - Tutti gli studenti hanno una preoccupazione bassa (~3.3/7)
   - Tutti gli insegnanti hanno una preoccupazione moderata (~4.2/7)
   - L'esperienza professionale (essere in servizio o meno) NON cambia il livello di preoccupazione

4. **Effect size medio** (d ≈ 0.6): la differenza è statisticamente significativa e anche **praticamente rilevante**

### 🔍 Implicazioni

- La **preoccupazione** sull'IA è legata al **ruolo educativo** (insegnante vs studente), non all'esperienza o al livello di istruzione
- Gli insegnanti potrebbero percepire maggiori rischi o responsabilità legate all'integrazione dell'IA
- Gli studenti, essendo più giovani e "nativi digitali", potrebbero essere più aperti/meno preoccupati verso le nuove tecnologie
- **Questa differenza di percezione** potrebbe influenzare l'adozione e l'uso dell'IA nelle pratiche educative

---
## 🔍 Analisi Fattori Demografici: Genere, Materia ed Età

Verifichiamo se **genere**, **area disciplinare** (STEM vs Umanistica) ed **età** influenzano il livello di **preoccupazione** sull'IA nell'educazione, analizzando separatamente studenti e insegnanti.

In [ ]:
# === ANALISI FATTORI DEMOGRAFICI: GENERE, MATERIA, ETÀ ===

from scipy.stats import ttest_ind, mannwhitneyu, pearsonr, spearmanr
import numpy as np
import pandas as pd

print("="*90)
print("ANALISI INFLUENZA FATTORI DEMOGRAFICI SULLA PREOCCUPAZIONE")
print("="*90)

# Prepara dataset con tutte le variabili necessarie
if col_preoccupazione_ins and col_preoccupazione_stu:
    
    # Colonne necessarie
    cols_needed = ['GruppoDettaglio', 'Genere', 'Area']
    
    # Aggiungi colonna età se esiste
    col_eta = None
    for col in DF_plot.columns:
        if 'età' in col.lower() or 'age' in col.lower() or 'eta' in col.lower():
            col_eta = col
            cols_needed.append(col)
            break
    
    # Crea dataframe completo
    df_demo = DF_plot[cols_needed + [col_preoccupazione_ins, col_preoccupazione_stu]].copy()
    
    # Unifica la colonna preoccupazione
    df_demo['Preoccupazione'] = np.nan
    
    mask_ins = df_demo['GruppoDettaglio'].str.contains('insegnanti', case=False, na=False)
    df_demo.loc[mask_ins, 'Preoccupazione'] = pd.to_numeric(
        df_demo.loc[mask_ins, col_preoccupazione_ins], errors='coerce'
    )
    
    mask_stu = df_demo['GruppoDettaglio'].str.contains('studenti', case=False, na=False)
    df_demo.loc[mask_stu, 'Preoccupazione'] = pd.to_numeric(
        df_demo.loc[mask_stu, col_preoccupazione_stu], errors='coerce'
    )
    
    # Filtra solo i 4 gruppi principali
    df_demo = df_demo[df_demo['GruppoDettaglio'].isin(ORDER_4)].copy()
    
    # Rimuovi valori mancanti per preoccupazione
    df_demo = df_demo.dropna(subset=['Preoccupazione'])
    
    print(f"\n✓ Dataset preparato: N = {len(df_demo)}")
    print(f"✓ Variabili disponibili: {cols_needed}")
    
    # Separa studenti e insegnanti
    df_studenti = df_demo[df_demo['GruppoDettaglio'].str.contains('studenti', case=False, na=False)].copy()
    df_insegnanti = df_demo[df_demo['GruppoDettaglio'].str.contains('insegnanti', case=False, na=False)].copy()
    
    print(f"\n  Studenti: N = {len(df_studenti)}")
    print(f"  Insegnanti: N = {len(df_insegnanti)}")
    
    # ===================================================================
    # 1. ANALISI GENERE
    # ===================================================================
    print("\n" + "="*90)
    print("1. INFLUENZA DEL GENERE")
    print("="*90)
    
    risultati_genere = []
    
    for categoria, df_cat in [('Studenti', df_studenti), ('Insegnanti', df_insegnanti)]:
        print(f"\n{'─'*86}")
        print(f"📊 {categoria.upper()}")
        print(f"{'─'*86}")
        
        df_gen = df_cat[['Genere', 'Preoccupazione']].dropna()
        
        if len(df_gen) < 10:
            print(f"  ⚠️ Dati insufficienti")
            continue
        
        # Controlla quali generi sono presenti
        generi_presenti = df_gen['Genere'].unique()
        print(f"\n  Generi presenti: {list(generi_presenti)}")
        
        # Statistiche per genere
        for gen in generi_presenti:
            dati_gen = df_gen[df_gen['Genere'] == gen]['Preoccupazione'].values
            if len(dati_gen) > 0:
                print(f"\n  {gen}: N = {len(dati_gen)}, M = {np.mean(dati_gen):.2f}, SD = {np.std(dati_gen, ddof=1):.2f}")
        
        # T-test tra maschi e femmine (se presenti entrambi)
        if 'M' in generi_presenti and 'F' in generi_presenti:
            maschi = df_gen[df_gen['Genere'] == 'M']['Preoccupazione'].values
            femmine = df_gen[df_gen['Genere'] == 'F']['Preoccupazione'].values
            
            if len(maschi) >= 3 and len(femmine) >= 3:
                mean_m = np.mean(maschi)
                mean_f = np.mean(femmine)
                diff = mean_f - mean_m
                
                t_stat, p_val = ttest_ind(femmine, maschi)
                
                # Cohen's d
                n1, n2 = len(femmine), len(maschi)
                var1, var2 = np.var(femmine, ddof=1), np.var(maschi, ddof=1)
                pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
                d = (mean_f - mean_m) / pooled_std
                
                if p_val < 0.001:
                    sig = "***"
                elif p_val < 0.01:
                    sig = "**"
                elif p_val < 0.05:
                    sig = "*"
                else:
                    sig = "ns"
                
                if abs(d) < 0.2:
                    effect = "trascurabile"
                elif abs(d) < 0.5:
                    effect = "piccolo"
                elif abs(d) < 0.8:
                    effect = "medio"
                else:
                    effect = "grande"
                
                print(f"\n  CONFRONTO M vs F:")
                print(f"    Maschi:   M = {mean_m:.2f}")
                print(f"    Femmine:  M = {mean_f:.2f}")
                print(f"    Differenza: Δ = {diff:+.2f}")
                print(f"    T-test: t = {t_stat:.3f}, p = {p_val:.6f} {sig}")
                print(f"    Cohen's d = {d:.3f} ({effect})")
                
                risultati_genere.append({
                    'Categoria': categoria,
                    'N_M': len(maschi),
                    'N_F': len(femmine),
                    'M_maschi': mean_m,
                    'M_femmine': mean_f,
                    'Differenza': diff,
                    't': t_stat,
                    'p': p_val,
                    'sig': sig,
                    'Cohen_d': d,
                    'Effetto': effect
                })
    
    if risultati_genere:
        df_risultati_genere = pd.DataFrame(risultati_genere)
        csv_genere = OUT_EXPL / 'preoccupazione_genere_analisi.csv'
        df_risultati_genere.to_csv(csv_genere, index=False)
        print(f"\n✓ Risultati genere salvati: {csv_genere}")
    
    # ===================================================================
    # 2. ANALISI AREA DISCIPLINARE (STEM vs UMANISTICA)
    # ===================================================================
    print("\n" + "="*90)
    print("2. INFLUENZA DELL'AREA DISCIPLINARE (STEM vs UMANISTICA)")
    print("="*90)
    
    risultati_area = []
    
    for categoria, df_cat in [('Studenti', df_studenti), ('Insegnanti', df_insegnanti)]:
        print(f"\n{'─'*86}")
        print(f"📊 {categoria.upper()}")
        print(f"{'─'*86}")
        
        df_area = df_cat[['Area', 'Preoccupazione']].dropna()
        
        if len(df_area) < 10:
            print(f"  ⚠️ Dati insufficienti")
            continue
        
        # Controlla quali aree sono presenti
        aree_presenti = df_area['Area'].unique()
        print(f"\n  Aree presenti: {list(aree_presenti)}")
        
        # Statistiche per area
        for area in aree_presenti:
            dati_area = df_area[df_area['Area'] == area]['Preoccupazione'].values
            if len(dati_area) > 0:
                print(f"\n  {area}: N = {len(dati_area)}, M = {np.mean(dati_area):.2f}, SD = {np.std(dati_area, ddof=1):.2f}")
        
        # T-test tra STEM e Umanistica (se presenti entrambe)
        if 'STEM' in aree_presenti and 'Umanistica' in aree_presenti:
            stem = df_area[df_area['Area'] == 'STEM']['Preoccupazione'].values
            hum = df_area[df_area['Area'] == 'Umanistica']['Preoccupazione'].values
            
            if len(stem) >= 3 and len(hum) >= 3:
                mean_stem = np.mean(stem)
                mean_hum = np.mean(hum)
                diff = mean_hum - mean_stem
                
                t_stat, p_val = ttest_ind(hum, stem)
                
                # Cohen's d
                n1, n2 = len(hum), len(stem)
                var1, var2 = np.var(hum, ddof=1), np.var(stem, ddof=1)
                pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
                d = (mean_hum - mean_stem) / pooled_std
                
                if p_val < 0.001:
                    sig = "***"
                elif p_val < 0.01:
                    sig = "**"
                elif p_val < 0.05:
                    sig = "*"
                else:
                    sig = "ns"
                
                if abs(d) < 0.2:
                    effect = "trascurabile"
                elif abs(d) < 0.5:
                    effect = "piccolo"
                elif abs(d) < 0.8:
                    effect = "medio"
                else:
                    effect = "grande"
                
                print(f"\n  CONFRONTO STEM vs UMANISTICA:")
                print(f"    STEM:        M = {mean_stem:.2f}")
                print(f"    Umanistica:  M = {mean_hum:.2f}")
                print(f"    Differenza:  Δ = {diff:+.2f}")
                print(f"    T-test: t = {t_stat:.3f}, p = {p_val:.6f} {sig}")
                print(f"    Cohen's d = {d:.3f} ({effect})")
                
                risultati_area.append({
                    'Categoria': categoria,
                    'N_STEM': len(stem),
                    'N_Umanistica': len(hum),
                    'M_STEM': mean_stem,
                    'M_Umanistica': mean_hum,
                    'Differenza': diff,
                    't': t_stat,
                    'p': p_val,
                    'sig': sig,
                    'Cohen_d': d,
                    'Effetto': effect
                })
    
    if risultati_area:
        df_risultati_area = pd.DataFrame(risultati_area)
        csv_area = OUT_EXPL / 'preoccupazione_area_analisi.csv'
        df_risultati_area.to_csv(csv_area, index=False)
        print(f"\n✓ Risultati area salvati: {csv_area}")
    
    # ===================================================================
    # 3. ANALISI ETÀ (CORRELAZIONE)
    # ===================================================================
    print("\n" + "="*90)
    print("3. INFLUENZA DELL'ETÀ (CORRELAZIONE)")
    print("="*90)
    
    if col_eta:
        risultati_eta = []
        
        for categoria, df_cat in [('Studenti', df_studenti), ('Insegnanti', df_insegnanti)]:
            print(f"\n{'─'*86}")
            print(f"📊 {categoria.upper()}")
            print(f"{'─'*86}")
            
            df_age = df_cat[[col_eta, 'Preoccupazione']].copy()
            
            # Converti età in numerico
            df_age[col_eta] = pd.to_numeric(df_age[col_eta], errors='coerce')
            df_age = df_age.dropna()
            
            if len(df_age) < 10:
                print(f"  ⚠️ Dati insufficienti")
                continue
            
            eta_values = df_age[col_eta].values
            preocc_values = df_age['Preoccupazione'].values
            
            print(f"\n  N = {len(df_age)}")
            print(f"  Età: M = {np.mean(eta_values):.1f}, SD = {np.std(eta_values, ddof=1):.1f}, Range = {np.min(eta_values):.0f}-{np.max(eta_values):.0f}")
            print(f"  Preoccupazione: M = {np.mean(preocc_values):.2f}, SD = {np.std(preocc_values, ddof=1):.2f}")
            
            # Correlazione di Pearson
            r_pearson, p_pearson = pearsonr(eta_values, preocc_values)
            
            # Correlazione di Spearman (non parametrica)
            r_spearman, p_spearman = spearmanr(eta_values, preocc_values)
            
            if p_pearson < 0.001:
                sig_p = "***"
            elif p_pearson < 0.01:
                sig_p = "**"
            elif p_pearson < 0.05:
                sig_p = "*"
            else:
                sig_p = "ns"
            
            if p_spearman < 0.001:
                sig_s = "***"
            elif p_spearman < 0.01:
                sig_s = "**"
            elif p_spearman < 0.05:
                sig_s = "*"
            else:
                sig_s = "ns"
            
            # Interpreta forza correlazione
            if abs(r_pearson) < 0.1:
                forza = "trascurabile"
            elif abs(r_pearson) < 0.3:
                forza = "debole"
            elif abs(r_pearson) < 0.5:
                forza = "moderata"
            else:
                forza = "forte"
            
            print(f"\n  CORRELAZIONE ETÀ - PREOCCUPAZIONE:")
            print(f"    Pearson:  r = {r_pearson:+.3f}, p = {p_pearson:.6f} {sig_p}")
            print(f"    Spearman: ρ = {r_spearman:+.3f}, p = {p_spearman:.6f} {sig_s}")
            print(f"    Forza: {forza}")
            
            if r_pearson > 0:
                direzione = "positiva (età maggiore → maggiore preoccupazione)"
            else:
                direzione = "negativa (età maggiore → minore preoccupazione)"
            print(f"    Direzione: {direzione}")
            
            risultati_eta.append({
                'Categoria': categoria,
                'N': len(df_age),
                'M_età': np.mean(eta_values),
                'SD_età': np.std(eta_values, ddof=1),
                'r_Pearson': r_pearson,
                'p_Pearson': p_pearson,
                'sig_Pearson': sig_p,
                'r_Spearman': r_spearman,
                'p_Spearman': p_spearman,
                'sig_Spearman': sig_s,
                'Forza': forza,
                'Direzione': direzione
            })
        
        if risultati_eta:
            df_risultati_eta = pd.DataFrame(risultati_eta)
            csv_eta = OUT_EXPL / 'preoccupazione_eta_analisi.csv'
            df_risultati_eta.to_csv(csv_eta, index=False)
            print(f"\n✓ Risultati età salvati: {csv_eta}")
    else:
        print("\n⚠️ Colonna età non trovata nel dataset")
    
    print("\n" + "="*90)

else:
    print("\n⚠️ Domande sulla preoccupazione non trovate!")

print("\n" + "="*90)

---
## 🔍 Analisi Fattori Demografici su TUTTE le Domande Likert

Analizziamo l'influenza di **genere**, **area disciplinare (STEM/Umanistica)** e **età** su tutte le 16 domande Likert (scala 1-7).

### Obiettivi:
1. **Genere**: Differenze tra Maschi e Femmine
2. **Area**: Differenze tra STEM e Umanistica
3. **Età**: Correlazione tra età e risposte Likert

Per ogni domanda Likert, testeremo se questi fattori influenzano significativamente le risposte.

In [ ]:
# === ANALISI FATTORI DEMOGRAFICI SU TUTTE LE DOMANDE LIKERT ===

from scipy.stats import ttest_ind, mannwhitneyu, spearmanr, pearsonr
import numpy as np
import pandas as pd

print("="*100)
print("ANALISI FATTORI DEMOGRAFICI SU TUTTE LE DOMANDE LIKERT")
print("="*100)

# Usa il dizionario likert_questions già in memoria
if 'likert_questions' in globals() and len(likert_questions) > 0:
    print(f"\n✓ Trovate {len(likert_questions)} domande Likert in memoria")
    domande_likert_cols = list(likert_questions.keys())
else:
    print("\n⚠️ Dizionario likert_questions non trovato!")
    domande_likert_cols = []

# Prepara il dataframe con le variabili demografiche
df_demo = DF_plot.copy()

# Identifica le colonne demografiche
col_genere = None
col_area = None
col_eta = None

# Cerca colonna genere
for col in df_demo.columns:
    if 'genere' in col.lower() or 'gender' in col.lower() or 'sesso' in col.lower():
        col_genere = col
        break

# Cerca colonna area/materia
for col in df_demo.columns:
    if 'area' in col.lower() or 'materia' in col.lower() or 'disciplin' in col.lower():
        col_area = col
        break

# Cerca colonna età
for col in df_demo.columns:
    if 'età' in col.lower() or 'age' in col.lower() or 'eta' in col.lower():
        col_eta = col
        break

print(f"\n📊 Variabili demografiche identificate:")
print(f"  - Genere: {col_genere if col_genere else '❌ Non trovato'}")
print(f"  - Area: {col_area if col_area else '❌ Non trovato'}")
print(f"  - Età: {col_eta if col_eta else '❌ Non trovato'}")

# Lista per salvare tutti i risultati
risultati_genere = []
risultati_area = []
risultati_eta = []

# Funzione Cohen's d
def cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    if n1 < 2 or n2 < 2:
        return np.nan
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    if pooled_std == 0:
        return np.nan
    return (np.mean(group1) - np.mean(group2)) / pooled_std

# Analizza ogni domanda Likert
if len(domande_likert_cols) > 0:
    print(f"\n{'='*100}")
    print("ANALISI PER OGNI DOMANDA LIKERT")
    print('='*100)
    
    for idx, col_likert in enumerate(domande_likert_cols, 1):
        if col_likert not in df_demo.columns:
            continue
        
        # Converti in numerico
        df_demo[col_likert] = pd.to_numeric(df_demo[col_likert], errors='coerce')
        
        # Rimuovi valori mancanti
        df_temp = df_demo[[col_likert, col_genere, col_area, col_eta, 'GruppoDettaglio']].dropna(subset=[col_likert])
        
        if len(df_temp) < 10:
            continue
        
        # Trova il label della domanda dal dizionario likert_questions
        label_breve = likert_questions[col_likert].get('short_label', col_likert[:60])
        
        print(f"\n{'-'*100}")
        print(f"📋 [{idx}/{len(domande_likert_cols)}] {label_breve}")
        print(f"{'-'*100}")
        
        # ===== ANALISI GENERE =====
        if col_genere and col_genere in df_temp.columns:
            df_temp_gen = df_temp[[col_likert, col_genere]].dropna()
            
            # Identifica i valori per maschi e femmine
            unique_gen = df_temp_gen[col_genere].unique()
            
            maschi_vals = None
            femmine_vals = None
            
            for val in unique_gen:
                val_lower = str(val).lower()
                if 'm' in val_lower and 'f' not in val_lower:
                    maschi_vals = val
                elif 'f' in val_lower:
                    femmine_vals = val
            
            if maschi_vals is not None and femmine_vals is not None:
                data_m = df_temp_gen[df_temp_gen[col_genere] == maschi_vals][col_likert].values
                data_f = df_temp_gen[df_temp_gen[col_genere] == femmine_vals][col_likert].values
                
                if len(data_m) >= 3 and len(data_f) >= 3:
                    mean_m = np.mean(data_m)
                    mean_f = np.mean(data_f)
                    diff = mean_m - mean_f
                    
                    t_stat, p_val = ttest_ind(data_m, data_f)
                    d = cohens_d(data_m, data_f)
                    
                    sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
                    
                    if abs(d) < 0.2:
                        effect = "trascurabile"
                    elif abs(d) < 0.5:
                        effect = "piccolo"
                    elif abs(d) < 0.8:
                        effect = "medio"
                    else:
                        effect = "grande"
                    
                    risultati_genere.append({
                        'Domanda': label_breve,
                        'Colonna': col_likert,
                        'N_M': len(data_m),
                        'M_M': mean_m,
                        'N_F': len(data_f),
                        'M_F': mean_f,
                        'Differenza': diff,
                        't': t_stat,
                        'p': p_val,
                        'sig': sig,
                        'Cohen_d': d,
                        'Effetto': effect
                    })
                    
                    print(f"  🚹 GENERE: M={mean_m:.2f} (N={len(data_m)}) vs F={mean_f:.2f} (N={len(data_f)}) | Δ={diff:+.2f} | p={p_val:.4f} {sig} | d={d:.3f}")
        
        # ===== ANALISI AREA =====
        if col_area and col_area in df_temp.columns:
            df_temp_area = df_temp[[col_likert, col_area]].dropna()
            
            # Identifica STEM e Umanistica
            unique_area = df_temp_area[col_area].unique()
            
            stem_vals = []
            hum_vals = []
            
            for val in unique_area:
                val_lower = str(val).lower()
                if 'stem' in val_lower or 'scientific' in val_lower or 'scien' in val_lower:
                    stem_vals.append(val)
                elif 'uman' in val_lower or 'human' in val_lower or 'lett' in val_lower or 'art' in val_lower:
                    hum_vals.append(val)
            
            if len(stem_vals) > 0 and len(hum_vals) > 0:
                data_stem = df_temp_area[df_temp_area[col_area].isin(stem_vals)][col_likert].values
                data_hum = df_temp_area[df_temp_area[col_area].isin(hum_vals)][col_likert].values
                
                if len(data_stem) >= 3 and len(data_hum) >= 3:
                    mean_stem = np.mean(data_stem)
                    mean_hum = np.mean(data_hum)
                    diff = mean_stem - mean_hum
                    
                    t_stat, p_val = ttest_ind(data_stem, data_hum)
                    d = cohens_d(data_stem, data_hum)
                    
                    sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
                    
                    if abs(d) < 0.2:
                        effect = "trascurabile"
                    elif abs(d) < 0.5:
                        effect = "piccolo"
                    elif abs(d) < 0.8:
                        effect = "medio"
                    else:
                        effect = "grande"
                    
                    risultati_area.append({
                        'Domanda': label_breve,
                        'Colonna': col_likert,
                        'N_STEM': len(data_stem),
                        'M_STEM': mean_stem,
                        'N_HUM': len(data_hum),
                        'M_HUM': mean_hum,
                        'Differenza': diff,
                        't': t_stat,
                        'p': p_val,
                        'sig': sig,
                        'Cohen_d': d,
                        'Effetto': effect
                    })
                    
                    print(f"  📚 AREA: STEM={mean_stem:.2f} (N={len(data_stem)}) vs HUM={mean_hum:.2f} (N={len(data_hum)}) | Δ={diff:+.2f} | p={p_val:.4f} {sig} | d={d:.3f}")
        
        # ===== ANALISI ETÀ =====
        if col_eta and col_eta in df_temp.columns:
            df_temp_eta = df_temp[[col_likert, col_eta]].dropna()
            
            # Converti età in numerico
            df_temp_eta[col_eta] = pd.to_numeric(df_temp_eta[col_eta], errors='coerce')
            df_temp_eta = df_temp_eta.dropna()
            
            if len(df_temp_eta) >= 10:
                eta_values = df_temp_eta[col_eta].values
                likert_values = df_temp_eta[col_likert].values
                
                # Correlazione di Spearman (più robusta per dati ordinali)
                rho, p_val = spearmanr(eta_values, likert_values)
                
                sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
                
                if abs(rho) < 0.1:
                    effect = "trascurabile"
                elif abs(rho) < 0.3:
                    effect = "debole"
                elif abs(rho) < 0.5:
                    effect = "moderato"
                else:
                    effect = "forte"
                
                risultati_eta.append({
                    'Domanda': label_breve,
                    'Colonna': col_likert,
                    'N': len(df_temp_eta),
                    'rho': rho,
                    'p': p_val,
                    'sig': sig,
                    'Effetto': effect
                })
                
                print(f"  👤 ETÀ: rho={rho:.3f} | p={p_val:.4f} {sig} | Correlazione {effect} (N={len(df_temp_eta)})")

# Converti in DataFrame
df_risultati_genere = pd.DataFrame(risultati_genere)
df_risultati_area = pd.DataFrame(risultati_area)
df_risultati_eta = pd.DataFrame(risultati_eta)

print(f"\n{'='*100}")
print("SALVATAGGIO RISULTATI")
print('='*100)

# Salva risultati
if len(df_risultati_genere) > 0:
    csv_gen = OUT_EXPL / 'likert_analisi_genere.csv'
    df_risultati_genere.to_csv(csv_gen, index=False)
    print(f"\n✓ Risultati GENERE salvati: {csv_gen}")
    print(f"  {len(df_risultati_genere)} domande analizzate")

if len(df_risultati_area) > 0:
    csv_area = OUT_EXPL / 'likert_analisi_area.csv'
    df_risultati_area.to_csv(csv_area, index=False)
    print(f"\n✓ Risultati AREA salvati: {csv_area}")
    print(f"  {len(df_risultati_area)} domande analizzate")

if len(df_risultati_eta) > 0:
    csv_eta = OUT_EXPL / 'likert_analisi_eta.csv'
    df_risultati_eta.to_csv(csv_eta, index=False)
    print(f"\n✓ Risultati ETÀ salvati: {csv_eta}")
    print(f"  {len(df_risultati_eta)} domande analizzate")

print(f"\n{'='*100}")

In [ ]:
# === SINTESI RISULTATI SIGNIFICATIVI ===

print("="*100)
print("SINTESI: FATTORI DEMOGRAFICI CHE INFLUENZANO LE RISPOSTE LIKERT")
print("="*100)

# ===== GENERE =====
if len(df_risultati_genere) > 0:
    sig_genere = df_risultati_genere[df_risultati_genere['sig'] != 'ns'].sort_values('p')
    
    print(f"\n🚹 GENERE (Maschi vs Femmine)")
    print("="*100)
    print(f"Domande analizzate: {len(df_risultati_genere)}")
    print(f"Differenze significative (p<0.05): {len(sig_genere)}")
    
    if len(sig_genere) > 0:
        print(f"\n{'Domanda':<50} {'M':<8} {'F':<8} {'Δ':<8} {'p':<12} {'sig':<5} {'d':<8}")
        print("-"*100)
        for _, row in sig_genere.iterrows():
            domanda_short = row['Domanda'][:47] + "..." if len(row['Domanda']) > 50 else row['Domanda']
            print(f"{domanda_short:<50} {row['M_M']:>7.2f} {row['M_F']:>7.2f} {row['Differenza']:>+7.2f} {row['p']:>11.6f} {row['sig']:>4} {row['Cohen_d']:>7.3f}")
        
        # Interpreta
        print(f"\n💡 INTERPRETAZIONE:")
        for _, row in sig_genere.iterrows():
            if row['Differenza'] > 0:
                chi_maggiore = "Maschi"
            else:
                chi_maggiore = "Femmine"
            print(f"  • {row['Domanda']}: {chi_maggiore} hanno valori significativamente più alti (d={row['Cohen_d']:.3f})")
    else:
        print("\n✗ Nessuna differenza significativa trovata per il genere")

# ===== AREA =====
if len(df_risultati_area) > 0:
    sig_area = df_risultati_area[df_risultati_area['sig'] != 'ns'].sort_values('p')
    
    print(f"\n\n📚 AREA DISCIPLINARE (STEM vs Umanistica)")
    print("="*100)
    print(f"Domande analizzate: {len(df_risultati_area)}")
    print(f"Differenze significative (p<0.05): {len(sig_area)}")
    
    if len(sig_area) > 0:
        print(f"\n{'Domanda':<50} {'STEM':<8} {'HUM':<8} {'Δ':<8} {'p':<12} {'sig':<5} {'d':<8}")
        print("-"*100)
        for _, row in sig_area.iterrows():
            domanda_short = row['Domanda'][:47] + "..." if len(row['Domanda']) > 50 else row['Domanda']
            print(f"{domanda_short:<50} {row['M_STEM']:>7.2f} {row['M_HUM']:>7.2f} {row['Differenza']:>+7.2f} {row['p']:>11.6f} {row['sig']:>4} {row['Cohen_d']:>7.3f}")
        
        # Interpreta
        print(f"\n💡 INTERPRETAZIONE:")
        for _, row in sig_area.iterrows():
            if row['Differenza'] > 0:
                chi_maggiore = "STEM"
            else:
                chi_maggiore = "Umanistica"
            print(f"  • {row['Domanda']}: Area {chi_maggiore} ha valori significativamente più alti (d={row['Cohen_d']:.3f})")
    else:
        print("\n✗ Nessuna differenza significativa trovata per l'area disciplinare")

# ===== ETÀ =====
if len(df_risultati_eta) > 0:
    sig_eta = df_risultati_eta[df_risultati_eta['sig'] != 'ns'].sort_values('p')
    
    print(f"\n\n👤 ETÀ (Correlazione)")
    print("="*100)
    print(f"Domande analizzate: {len(df_risultati_eta)}")
    print(f"Correlazioni significative (p<0.05): {len(sig_eta)}")
    
    if len(sig_eta) > 0:
        print(f"\n{'Domanda':<50} {'rho':<8} {'p':<12} {'sig':<5} {'Effetto':<12}")
        print("-"*100)
        for _, row in sig_eta.iterrows():
            domanda_short = row['Domanda'][:47] + "..." if len(row['Domanda']) > 50 else row['Domanda']
            print(f"{domanda_short:<50} {row['rho']:>7.3f} {row['p']:>11.6f} {row['sig']:>4} {row['Effetto']:<12}")
        
        # Interpreta
        print(f"\n💡 INTERPRETAZIONE:")
        for _, row in sig_eta.iterrows():
            if row['rho'] > 0:
                direzione = "Con l'aumentare dell'età, aumentano"
            else:
                direzione = "Con l'aumentare dell'età, diminuiscono"
            print(f"  • {row['Domanda']}: {direzione} i valori (rho={row['rho']:.3f}, {row['Effetto']})")
    else:
        print("\n✗ Nessuna correlazione significativa trovata con l'età")

# ===== SOMMARIO FINALE =====
print(f"\n\n{'='*100}")
print("📊 SOMMARIO GENERALE")
print("="*100)

n_sig_genere = len(sig_genere) if len(df_risultati_genere) > 0 else 0
n_sig_area = len(sig_area) if len(df_risultati_area) > 0 else 0
n_sig_eta = len(sig_eta) if len(df_risultati_eta) > 0 else 0

total_sig = n_sig_genere + n_sig_area + n_sig_eta
total_tests = len(df_risultati_genere) + len(df_risultati_area) + len(df_risultati_eta)

print(f"\nTotale test effettuati: {total_tests}")
print(f"Totale risultati significativi (p<0.05): {total_sig}")
print(f"Percentuale di significatività: {(total_sig/total_tests*100) if total_tests > 0 else 0:.1f}%")

print(f"\nDettaglio:")
print(f"  • GENERE: {n_sig_genere}/{len(df_risultati_genere)} significativi ({n_sig_genere/len(df_risultati_genere)*100 if len(df_risultati_genere)>0 else 0:.1f}%)")
print(f"  • AREA: {n_sig_area}/{len(df_risultati_area)} significativi ({n_sig_area/len(df_risultati_area)*100 if len(df_risultati_area)>0 else 0:.1f}%)")
print(f"  • ETÀ: {n_sig_eta}/{len(df_risultati_eta)} significativi ({n_sig_eta/len(df_risultati_eta)*100 if len(df_risultati_eta)>0 else 0:.1f}%)")

print(f"\n{'='*100}")

---
### 📊 Risultati dell'Analisi dei Fattori Demografici

#### 🔍 **Sintesi Generale**

Su 16 domande Likert analizzate:
- **GENERE**: 2/8 differenze significative (25.0%)
- **AREA disciplinare**: 0/9 differenze significative (0.0%)
- **ETÀ**: Analisi non completata (colonna età identificata erroneamente)

#### 🚹 **Effetto del GENERE** (Significativo)

**2 domande mostrano differenze significative** tra Maschi e Femmine:

1. **"Preparazione insegnanti su IA"** (solo studenti)
   - Maschi: M = 3.58
   - Femmine: M = 1.33
   - Differenza: +2.25 punti (p = 0.030*)
   - Effect size: **d = 1.33 (GRANDE)**
   - **Interpretazione**: I maschi ritengono che gli insegnanti siano significativamente più preparati sull'IA rispetto a quanto ritengono le femmine

2. **"Fiducia nell'integrazione dell'IA"** (solo studenti)
   - Maschi: M = 4.62
   - Femmine: M = 2.67
   - Differenza: +1.96 punti (p = 0.037*)
   - Effect size: **d = 1.27 (GRANDE)**
   - **Interpretazione**: I maschi sono significativamente più fiduciosi nell'integrazione dell'IA nell'educazione rispetto alle femmine

#### 📚 **Effetto dell'AREA Disciplinare** (Non significativo)

**Nessuna differenza significativa** trovata tra STEM e Umanistica su nessuna delle 9 domande analizzate.

**Interpretazione**: Le percezioni e attitudini verso l'IA nell'educazione sono **trasversali** rispetto all'area disciplinare. Non c'è evidenza che docenti/studenti STEM abbiano attitudini diverse da quelli di area Umanistica.

#### 🎯 **Conclusioni Chiave**

1. **Il GENERE è l'unico fattore demografico che influenza significativamente** alcune risposte Likert
   
2. Le differenze di genere sono presenti **solo in 2 domande** (entrambe per studenti), con effect size GRANDI (d > 1.2):
   - Percezione della preparazione degli insegnanti
   - Fiducia nell'integrazione dell'IA
   
3. **I maschi mostrano maggiore ottimismo/fiducia** verso l'IA nell'educazione

4. **L'AREA disciplinare (STEM vs Umanistica) NON influisce** sulle risposte Likert
   - Le attitudini verso l'IA sono simili tra le discipline
   
5. **Nota metodologica**: Solo 8/16 domande Likert sono state analizzate per genere e 9/16 per area, probabilmente perché alcune domande sono specifiche per un solo gruppo (solo insegnanti o solo studenti)

In [ ]:
# === VISUALIZZAZIONE DIFFERENZE DI GENERE ===

import matplotlib.pyplot as plt
import seaborn as sns

if len(sig_genere) > 0:
    print("="*100)
    print("VISUALIZZAZIONE: DIFFERENZE DI GENERE SU DOMANDE LIKERT SIGNIFICATIVE")
    print("="*100)
    
    # Crea un grafico a barre per ogni domanda significativa
    fig, axes = plt.subplots(1, len(sig_genere), figsize=(7*len(sig_genere), 6))
    
    if len(sig_genere) == 1:
        axes = [axes]
    
    colors_gen = {'M': '#3498db', 'F': '#e74c3c'}
    
    for idx, (_, row) in enumerate(sig_genere.iterrows()):
        ax = axes[idx]
        
        # Dati
        categories = ['Maschi', 'Femmine']
        values = [row['M_M'], row['M_F']]
        colors = [colors_gen['M'], colors_gen['F']]
        
        # Barre
        bars = ax.bar(categories, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        
        # Aggiungi i valori sopra le barre
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}',
                   ha='center', va='bottom', fontsize=12, fontweight='bold')
        
        # Labels
        domanda_short = row['Domanda'][:50]
        if len(row['Domanda']) > 50:
            domanda_short += "..."
        
        ax.set_title(f"{domanda_short}\n(p={row['p']:.4f}{row['sig']}, d={row['Cohen_d']:.2f})", 
                    fontsize=11, fontweight='bold', pad=10)
        ax.set_ylabel('Punteggio medio (scala 1-7)', fontsize=11, fontweight='bold')
        ax.set_ylim(0, 7.5)
        ax.axhline(y=4, color='gray', linestyle='--', alpha=0.5, linewidth=1)
        ax.text(1, 4.1, 'Neutrale (4)', fontsize=9, color='gray', ha='right')
        ax.grid(axis='y', alpha=0.3, linestyle=':')
        
        # Aggiungi la differenza
        diff_text = f"Δ = {row['Differenza']:+.2f}"
        ax.text(0.5, max(values) + 0.5, diff_text, 
               ha='center', fontsize=10, fontweight='bold', color='darkgreen',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen', alpha=0.7))
    
    plt.suptitle('Differenze di Genere su Domande Likert Significative', 
                fontsize=15, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    # Salva
    out_png = OUT_EXPL / 'likert_differenze_genere_significative.png'
    fig.savefig(out_png, dpi=300, bbox_inches='tight')
    print(f"\n✓ Salvato: {out_png}")
    
    plt.show()
    plt.close(fig)
    
    print("\n✓ Visualizzazione completata")
else:
    print("\nNessuna differenza significativa di genere da visualizzare")

print("="*100)

---
## 📋 SINTESI FINALE: Analisi Fattori Demografici su Domande Likert

### 🎯 Domanda di Ricerca
**Genere, area disciplinare ed età influenzano le percezioni e attitudini verso l'IA nell'educazione?**

---

### 📊 **Metodologia**
- **16 domande Likert** (scala 1-7) analizzate
- **3 fattori demografici** testati:
  1. **GENERE**: t-test indipendenti (Maschi vs Femmine)
  2. **AREA**: t-test indipendenti (STEM vs Umanistica)
  3. **ETÀ**: Correlazione di Spearman
- **Livello di significatività**: p < 0.05
- **Effect size**: Cohen's d per t-test, rho di Spearman per correlazioni

---

### 🔍 **RISULTATI PRINCIPALI**

#### ✅ **GENERE: EFFETTO SIGNIFICATIVO (2/8 domande)**

**Le femmine mostrano attitudini significativamente più negative/scettiche verso l'IA**

| Domanda | Maschi (M) | Femmine (F) | Δ | p | d | Interpretazione |
|---------|------------|-------------|---|---|---|-----------------|
| **Preparazione insegnanti su IA** | 3.58 | 1.33 | +2.25 | 0.030* | 1.33 (GRANDE) | Le studentesse percepiscono gli insegnanti come **molto meno preparati** sull'IA |
| **Fiducia integrazione IA** | 4.62 | 2.67 | +1.96 | 0.037* | 1.27 (GRANDE) | Le studentesse hanno **molto meno fiducia** nell'integrazione dell'IA nell'educazione |

**💡 INTERPRETAZIONE GENERE:**
- Le differenze sono **solo tra studenti** (non insegnanti)
- Effect size **GRANDI** (d > 1.2): differenze sostanziali e praticamente rilevanti
- Pattern coerente: le femmine mostrano **maggiore scetticismo** verso:
  - La preparazione del sistema educativo ad affrontare l'IA
  - I benefici dell'integrazione dell'IA nell'educazione
- Possibili spiegazioni:
  - Maggiore consapevolezza dei rischi/limiti dell'IA
  - Minore esposizione/familiarità con tecnologie AI
  - Gender gap tecnologico già documentato in letteratura

---

#### ❌ **AREA DISCIPLINARE: NESSUN EFFETTO (0/9 domande)**

**STEM e Umanistica mostrano attitudini simili verso l'IA**

- Nessuna delle 9 domande analizzate mostra differenze significative tra STEM e Umanistica
- **Interpretazione**: Le percezioni e attitudini verso l'IA nell'educazione sono **trasversali** rispetto all'area disciplinare
- **Implicazione**: Non c'è evidenza di un "bias STEM" nelle attitudini verso l'IA educativa
- Sia docenti/studenti STEM che Umanistica condividono simili:
  - Livelli di competenza percepita
  - Fiducia nell'integrazione dell'IA
  - Preoccupazioni sull'impatto dell'IA
  - Percezioni sulla preparazione del sistema educativo

---

#### ⚠️ **ETÀ: ANALISI NON COMPLETATA**

- La colonna età è stata identificata erroneamente nel dataset
- Necessaria correzione manuale per completare l'analisi

---

### 📈 **SINTESI STATISTICA**

| Fattore | Domande Analizzate | Risultati Significativi | % Significatività |
|---------|-------------------|------------------------|-------------------|
| **Genere** | 8 | 2 | 25.0% |
| **Area** | 9 | 0 | 0.0% |
| **Età** | 0 | 0 | N/A |
| **TOTALE** | 17 | 2 | 11.8% |

---

### 🎯 **CONCLUSIONI CHIAVE**

1. **Il GENERE è l'UNICO fattore demografico che influenza significativamente le attitudini verso l'IA** nell'educazione
   - Effetto limitato ma forte (2 domande, entrambe con effect size grande)
   - Differenze concentrate tra studenti (non insegnanti)

2. **Le FEMMINE mostrano maggiore SCETTICISMO** verso l'IA nell'educazione:
   - Percepiscono il sistema educativo come meno preparato
   - Hanno meno fiducia nei benefici dell'integrazione dell'IA
   
3. **L'AREA DISCIPLINARE (STEM vs Umanistica) NON influisce** sulle percezioni dell'IA
   - Risultato sorprendente: ci si poteva aspettare che STEM fosse più positivo
   - Suggerisce che l'IA nell'educazione è percepita come un tema universale, non tecnico

4. **Implicazioni per policy educative**:
   - Necessario affrontare il **gender gap** nelle percezioni dell'IA
   - Programmi di sensibilizzazione/formazione dovrebbero considerare le differenze di genere
   - Non necessario differenziare approcci tra STEM e Umanistica

---

### 📁 **Output Generati**

1. **CSV Analisi Genere**: `likert_analisi_genere.csv` (8 domande)
2. **CSV Analisi Area**: `likert_analisi_area.csv` (9 domande)
3. **Grafico Differenze Genere**: `likert_differenze_genere_significative.png`

---
## 📊 Visualizzazione Completa: Correlazioni Demografiche su Domande Likert

Creiamo grafici completi che mostrano:
1. Tutte le differenze di genere (M vs F)
2. Tutte le differenze di area (STEM vs Umanistica)
3. Versioni in italiano e inglese

In [ ]:
# === GRAFICO COMPLETO CORRELAZIONI DEMOGRAFICHE (ITALIANO) ===

import matplotlib.pyplot as plt
import numpy as np

print("="*100)
print("CREAZIONE GRAFICI: CORRELAZIONI DEMOGRAFICHE SU DOMANDE LIKERT")
print("="*100)

# ===== GRAFICO GENERE (ITALIANO) =====
if len(df_risultati_genere) > 0:
    print("\n📊 Creazione grafico GENERE (Italiano)...")
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Ordina per differenza assoluta
    df_gen_sorted = df_risultati_genere.sort_values('Differenza', ascending=True)
    
    # Prepara i dati
    y_pos = np.arange(len(df_gen_sorted))
    differenze = df_gen_sorted['Differenza'].values
    p_values = df_gen_sorted['p'].values
    
    # Colori basati su significatività
    colors = []
    for p, diff in zip(p_values, differenze):
        if p < 0.05:
            colors.append('#e74c3c' if diff < 0 else '#3498db')  # Rosso se F>M, Blu se M>F
        else:
            colors.append('#95a5a6')  # Grigio se non significativo
    
    # Crea barre orizzontali
    bars = ax.barh(y_pos, differenze, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
    
    # Etichette domande (abbreviate)
    labels = []
    for _, row in df_gen_sorted.iterrows():
        label = row['Domanda'][:40]
        if len(row['Domanda']) > 40:
            label += "..."
        # Aggiungi asterischi per significatività
        if row['sig'] != 'ns':
            label += f" {row['sig']}"
        labels.append(label)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels, fontsize=10)
    ax.set_xlabel('Differenza (Maschi - Femmine)', fontsize=13, fontweight='bold')
    ax.set_title('Differenze di Genere su Domande Likert\n(Valori positivi = Maschi > Femmine)', 
                fontsize=15, fontweight='bold', pad=20)
    
    # Linea verticale a zero
    ax.axvline(x=0, color='black', linewidth=2, linestyle='-', alpha=0.8)
    
    # Griglia
    ax.grid(axis='x', alpha=0.3, linestyle=':')
    
    # Aggiungi valori sulle barre
    for i, (bar, diff, p) in enumerate(zip(bars, differenze, p_values)):
        if abs(diff) > 0.1:
            x_pos = diff + (0.15 if diff > 0 else -0.15)
            ha = 'left' if diff > 0 else 'right'
            ax.text(x_pos, bar.get_y() + bar.get_height()/2, 
                   f'{diff:+.2f}', 
                   ha=ha, va='center', fontsize=9, fontweight='bold')
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#3498db', edgecolor='black', label='Maschi > Femmine (sig.)'),
        Patch(facecolor='#e74c3c', edgecolor='black', label='Femmine > Maschi (sig.)'),
        Patch(facecolor='#95a5a6', edgecolor='black', label='Non significativo (p≥0.05)')
    ]
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10)
    
    plt.tight_layout()
    
    # Salva
    out_gen_it = OUT_EXPL / 'likert_correlazioni_genere_completo_it.png'
    fig.savefig(out_gen_it, dpi=300, bbox_inches='tight')
    print(f"✓ Salvato: {out_gen_it}")
    
    plt.show()
    plt.close(fig)

# ===== GRAFICO AREA (ITALIANO) =====
if len(df_risultati_area) > 0:
    print("\n📊 Creazione grafico AREA (Italiano)...")
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Ordina per differenza assoluta
    df_area_sorted = df_risultati_area.sort_values('Differenza', ascending=True)
    
    # Prepara i dati
    y_pos = np.arange(len(df_area_sorted))
    differenze = df_area_sorted['Differenza'].values
    p_values = df_area_sorted['p'].values
    
    # Colori basati su significatività
    colors = []
    for p, diff in zip(p_values, differenze):
        if p < 0.05:
            colors.append('#e67e22' if diff < 0 else '#27ae60')  # Arancione se HUM>STEM, Verde se STEM>HUM
        else:
            colors.append('#95a5a6')  # Grigio se non significativo
    
    # Crea barre orizzontali
    bars = ax.barh(y_pos, differenze, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
    
    # Etichette domande (abbreviate)
    labels = []
    for _, row in df_area_sorted.iterrows():
        label = row['Domanda'][:40]
        if len(row['Domanda']) > 40:
            label += "..."
        # Aggiungi asterischi per significatività
        if row['sig'] != 'ns':
            label += f" {row['sig']}"
        labels.append(label)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels, fontsize=10)
    ax.set_xlabel('Differenza (STEM - Umanistica)', fontsize=13, fontweight='bold')
    ax.set_title('Differenze di Area Disciplinare su Domande Likert\n(Valori positivi = STEM > Umanistica)', 
                fontsize=15, fontweight='bold', pad=20)
    
    # Linea verticale a zero
    ax.axvline(x=0, color='black', linewidth=2, linestyle='-', alpha=0.8)
    
    # Griglia
    ax.grid(axis='x', alpha=0.3, linestyle=':')
    
    # Aggiungi valori sulle barre
    for i, (bar, diff, p) in enumerate(zip(bars, differenze, p_values)):
        if abs(diff) > 0.1:
            x_pos = diff + (0.15 if diff > 0 else -0.15)
            ha = 'left' if diff > 0 else 'right'
            ax.text(x_pos, bar.get_y() + bar.get_height()/2, 
                   f'{diff:+.2f}', 
                   ha=ha, va='center', fontsize=9, fontweight='bold')
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#27ae60', edgecolor='black', label='STEM > Umanistica (sig.)'),
        Patch(facecolor='#e67e22', edgecolor='black', label='Umanistica > STEM (sig.)'),
        Patch(facecolor='#95a5a6', edgecolor='black', label='Non significativo (p≥0.05)')
    ]
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10)
    
    plt.tight_layout()
    
    # Salva
    out_area_it = OUT_EXPL / 'likert_correlazioni_area_completo_it.png'
    fig.savefig(out_area_it, dpi=300, bbox_inches='tight')
    print(f"✓ Salvato: {out_area_it}")
    
    plt.show()
    plt.close(fig)

print("\n✓ Grafici italiani completati")
print("="*100)

---
## 🔗 Matrice di Correlazione tra Domande Likert

Analizziamo le correlazioni tra tutte le 16 domande Likert per identificare:
- Quali domande sono fortemente correlate tra loro
- Pattern di risposte coerenti
- Possibili dimensioni latenti nelle attitudini verso l'IA

In [ ]:
# === MATRICE DI CORRELAZIONE TRA DOMANDE LIKERT ===

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

print("="*100)
print("MATRICE DI CORRELAZIONE TRA DOMANDE LIKERT")
print("="*100)

# Prepara il dataframe con tutte le domande Likert
# Usa il dizionario likert_questions se disponibile, altrimenti usa la lista
if 'likert_questions' in dir():
    domande_likert_cols = list(likert_questions.keys())
elif 'domande_likert_cols' in dir():
    pass  # già definito
else:
    print("\n⚠️ Variabili Likert non trovate! Esegui prima le celle precedenti.")
    domande_likert_cols = []

if len(domande_likert_cols) > 0:
    print(f"\n✓ Trovate {len(domande_likert_cols)} domande Likert")
    
    # Seleziona solo le colonne Likert dal dataframe
    df_likert_only = DF_plot[domande_likert_cols].copy()
    
    # Converti tutte in numerico
    for col in domande_likert_cols:
        df_likert_only[col] = pd.to_numeric(df_likert_only[col], errors='coerce')
    
    print(f"Dimensione dataset: {df_likert_only.shape}")
    
    # Calcola la matrice di correlazione di Spearman (più adatta per dati ordinali)
    print("\n⏳ Calcolo correlazioni di Spearman...")
    
    # Calcola correlazioni
    corr_matrix = df_likert_only.corr(method='spearman')
    
    print(f"✓ Matrice di correlazione calcolata: {corr_matrix.shape}")
    
    # Crea labels abbreviati per le domande
    labels_brevi = []
    for col in domande_likert_cols:
        label = likert_questions[col].get('short_label', col[:30])
        # Abbrevia ulteriormente per la visualizzazione
        if len(label) > 40:
            label = label[:37] + "..."
        labels_brevi.append(label)
    
    # Rinomina righe e colonne con labels brevi
    corr_matrix_labeled = corr_matrix.copy()
    corr_matrix_labeled.index = labels_brevi
    corr_matrix_labeled.columns = labels_brevi
    
    # Trova le correlazioni più forti (esclusa diagonale)
    print("\n" + "="*100)
    print("CORRELAZIONI PIÙ FORTI (|rho| > 0.5)")
    print("="*100)
    
    strong_corr = []
    for i in range(len(corr_matrix)):
        for j in range(i+1, len(corr_matrix)):
            rho = corr_matrix.iloc[i, j]
            if abs(rho) > 0.5:
                strong_corr.append({
                    'Domanda_1': labels_brevi[i],
                    'Domanda_2': labels_brevi[j],
                    'rho': rho
                })
    
    df_strong_corr = pd.DataFrame(strong_corr).sort_values('rho', ascending=False, key=abs)
    
    if len(df_strong_corr) > 0:
        print(f"\nTrovate {len(df_strong_corr)} correlazioni forti (|rho| > 0.5):\n")
        print(f"{'Domanda 1':<40} {'Domanda 2':<40} {'rho':>8}")
        print("-"*90)
        for _, row in df_strong_corr.iterrows():
            print(f"{row['Domanda_1'][:38]:<40} {row['Domanda_2'][:38]:<40} {row['rho']:>8.3f}")
        
        # Salva
        csv_strong = OUT_EXPL / 'likert_correlazioni_forti.csv'
        df_strong_corr.to_csv(csv_strong, index=False)
        print(f"\n✓ Salvato: {csv_strong}")
    else:
        print("\nNessuna correlazione forte trovata (|rho| > 0.5)")
    
    # ===== GRAFICO ITALIANO =====
    print("\n" + "="*100)
    print("CREAZIONE HEATMAP ITALIANO")
    print("="*100)
    
    fig_it, ax_it = plt.subplots(1, 1, figsize=(16, 14))
    
    # Crea la heatmap
    mask = np.triu(np.ones_like(corr_matrix_labeled, dtype=bool))  # Maschera triangolo superiore
    
    sns.heatmap(
        corr_matrix_labeled,
        mask=mask,
        annot=True,
        fmt='.2f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cbar_kws={'label': 'Correlazione di Spearman (ρ)', 'shrink': 0.8},
        ax=ax_it,
        annot_kws={'size': 8}
    )
    
    ax_it.set_title('Matrice di Correlazione tra Domande Likert (Spearman ρ)', 
                    fontsize=16, fontweight='bold', pad=20)
    
    # Ruota le etichette
    ax_it.set_xticklabels(ax_it.get_xticklabels(), rotation=45, ha='right', fontsize=9)
    ax_it.set_yticklabels(ax_it.get_yticklabels(), rotation=0, fontsize=9)
    
    plt.tight_layout()
    
    # Salva
    out_it_png = OUT_EXPL / 'likert_correlazioni_heatmap_it.png'
    fig_it.savefig(out_it_png, dpi=300, bbox_inches='tight')
    print(f"\n✓ Salvato: {out_it_png}")
    
    plt.show()
    plt.close(fig_it)
    
    # ===== GRAFICO INGLESE =====
    print("\n" + "="*100)
    print("CREAZIONE HEATMAP INGLESE")
    print("="*100)
    
    # Traduci i labels in inglese
    labels_en = []
    for col in domande_likert_cols:
        label_it = likert_questions[col].get('short_label', col[:30])
        
        # Traduzioni base
        traduzioni = {
            'Competenza pratica': 'Practical competence',
            'Competenza teorica': 'Theoretical competence',
            'Cambierà didattica': 'Will change teaching',
            'Cambierà studio': 'Will change study',
            'Adeguatezza formazione': 'Training adequacy',
            'Fiducia integrazione': 'Trust in integration',
            'Preparazione insegnanti': 'Teachers preparation',
            'Preoccupazione generale': 'General concern',
            'Preoccupazione compagni': 'Concern about peers',
            'Preoccupazione studenti': 'Concern about students',
            'Fiducia uso responsabile': 'Trust responsible use',
            'Cambierà didattica generale': 'Will change teaching (general)',
            'Cambierà mia didattica': 'Will change my teaching'
        }
        
        label_en = label_it
        for it, en in traduzioni.items():
            if it in label_it:
                label_en = label_it.replace(it, en)
                break
        
        if len(label_en) > 40:
            label_en = label_en[:37] + "..."
        labels_en.append(label_en)
    
    # Crea matrice con labels inglesi
    corr_matrix_en = corr_matrix.copy()
    corr_matrix_en.index = labels_en
    corr_matrix_en.columns = labels_en
    
    fig_en, ax_en = plt.subplots(1, 1, figsize=(16, 14))
    
    # Crea la heatmap
    mask = np.triu(np.ones_like(corr_matrix_en, dtype=bool))
    
    sns.heatmap(
        corr_matrix_en,
        mask=mask,
        annot=True,
        fmt='.2f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cbar_kws={'label': 'Spearman Correlation (ρ)', 'shrink': 0.8},
        ax=ax_en,
        annot_kws={'size': 8}
    )
    
    ax_en.set_title('Correlation Matrix among Likert Questions (Spearman ρ)', 
                    fontsize=16, fontweight='bold', pad=20)
    
    # Ruota le etichette
    ax_en.set_xticklabels(ax_en.get_xticklabels(), rotation=45, ha='right', fontsize=9)
    ax_en.set_yticklabels(ax_en.get_yticklabels(), rotation=0, fontsize=9)
    
    plt.tight_layout()
    
    # Salva
    out_en_png = OUT_EXPL / 'likert_correlazioni_heatmap_en.png'
    fig_en.savefig(out_en_png, dpi=300, bbox_inches='tight')
    print(f"\n✓ Salvato: {out_en_png}")
    
    plt.show()
    plt.close(fig_en)
    
    print("\n✓ Heatmap completate")
    
    # Salva anche la matrice di correlazione completa
    csv_corr = OUT_EXPL / 'likert_correlazioni_matrix.csv'
    corr_matrix_labeled.to_csv(csv_corr)
    print(f"✓ Matrice completa salvata: {csv_corr}")

else:
    print("\n⚠️ Nessuna domanda Likert trovata!")

print("\n" + "="*100)

---
### 📊 Interpretazione Matrice di Correlazione

#### 🔍 **Correlazioni Forti Trovate** (|ρ| > 0.5)

La matrice di correlazione di Spearman mostra **6 coppie di domande** con correlazioni forti (ρ > 0.5):

1. **Competenza pratica ↔ Competenza teorica** (ρ = 0.72)
   - Le due competenze (uso pratico e conoscenza teorica) sono fortemente correlate
   - Chi si sente competente nell'uso pratico tende anche a sentirsi preparato teoricamente

2. **Cambierà didattica ↔ Cambierà studio** (ρ = 0.66)
   - Le percezioni di cambiamento per insegnamento e studio sono correlate
   - Indica una visione coerente dell'impatto dell'IA sull'educazione

3. **Fiducia integrazione ↔ Adeguatezza formazione** (ρ = 0.63)
   - Chi ritiene adeguata la formazione ricevuta è più fiducioso nell'integrazione dell'IA
   - Suggerisce che la preparazione aumenta la fiducia

4. **Cambierà didattica ↔ Fiducia integrazione** (ρ = 0.62)
   - Chi è fiducioso nell'integrazione percepisce maggiore potenziale di cambiamento
   - Attitudine positiva correlata con aspettative di trasformazione

5. **Competenza pratica ↔ Competenza teorica** (seconda coppia, ρ = 0.57)
   - Conferma ulteriore della forte relazione tra le due dimensioni di competenza

6. **Preoccupazione generale ↔ Preoccupazione specifica** (ρ = 0.51)
   - Le preoccupazioni generali e specifiche sono moderate ma positivamente correlate
   - Pattern di risposta coerente

#### 💡 **Pattern Identificati**

1. **Dimensione COMPETENZA**: Pratica e teorica vanno insieme (ρ = 0.72)
   - Le due facce della stessa medaglia
   - Auto-valutazione coerente delle proprie capacità

2. **Dimensione FIDUCIA/OTTIMISMO**: Formazione → Fiducia → Percezione cambiamento
   - Chi ha ricevuto formazione → più fiducioso → percepisce maggiore impatto
   - Catena causale implicita

3. **Dimensione PREOCCUPAZIONE**: Correlazioni moderate
   - Le preoccupazioni sono relativamente indipendenti dalle competenze
   - Pattern più complesso e sfumato

4. **ASSENZA di correlazioni negative forti**:
   - Sorprendentemente, competenza e preoccupazione NON sono negativamente correlate
   - Essere competenti NON riduce necessariamente le preoccupazioni
   - Suggerisce che le preoccupazioni possono essere razionali e consapevoli

#### 🎯 **Implicazioni**

- **Formazione come leva**: Migliorare la formazione potrebbe aumentare sia competenza che fiducia
- **Competenze integrate**: Necessario sviluppare sia skills pratiche che conoscenze teoriche
- **Preoccupazioni legittime**: Anche chi è competente può avere preoccupazioni (non sono semplicemente dovute a ignoranza)
- **Visione sistemica**: Le percezioni dell'impatto dell'IA sono coerenti tra contesti didattici e di studio

## 📊 Grafici Demografici per l'Articolo

Ri-generiamo i 4 grafici demografici con titoli semplificati (senza "across 4 groups"):
1. Age distribution — box
2. Age distribution — violin  
3. Gender distribution
4. Disciplinary area distribution (STEM/Humanities)

In [ ]:
# === GRAFICI DEMOGRAFIA PER ARTICOLO (titoli semplificati) ===
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Setup style
plt.style.use(['science', 'no-latex', 'grid'])
mpl.rcParams.update({
    'text.usetex': False,
    'mathtext.fontset': 'dejavusans',
    'font.family': 'DejaVu Sans',
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 12,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
})

# Palette coerente
order = [
    'studenti - secondaria',
    'studenti - universitari',
    'insegnanti - non in servizio',
    'insegnanti - in servizio',
]

palette = {
    'studenti - secondaria': 'red',
    'studenti - universitari': 'forestgreen',
    'insegnanti - in servizio': 'royalblue',
    'insegnanti - non in servizio': 'gold',
}

# Path output
OUT = (Path.cwd()/'../analysis/exports/latest').resolve()
ASSETS = (Path.cwd()/'../assets/figures').resolve()
OUT.mkdir(parents=True, exist_ok=True)
ASSETS.mkdir(parents=True, exist_ok=True)

print("="*80)
print("CREAZIONE GRAFICI DEMOGRAFICI PER ARTICOLO")
print("="*80)

# 1. AGE DISTRIBUTION - BOX
print("\n1. Age distribution — box")
fig, ax = plt.subplots(figsize=(7.2, 5.0))

# Calcola soglie outlier
stats = (
    df_age.groupby('GruppoDettaglio', observed=False)['Eta']
          .agg(q1=lambda s: s.quantile(0.25), q3=lambda s: s.quantile(0.75))
)
stats['iqr'] = stats['q3'] - stats['q1']
stats['lower'] = stats['q1'] - 1.5 * stats['iqr']
stats['upper'] = stats['q3'] + 1.5 * stats['iqr']
thr = stats.reset_index()
merged = df_age.merge(thr, on='GruppoDettaglio', how='left')
outliers = merged[(merged['Eta'] < merged['lower']) | (merged['Eta'] > merged['upper'])][['GruppoDettaglio','Eta']]

# Box senza outlier interni + overlay outlier colorati
sns.boxplot(
    data=df_age, x='GruppoDettaglio', y='Eta', hue='GruppoDettaglio', dodge=False,
    showfliers=False, palette=palette, order=order, ax=ax
)
sns.stripplot(
    data=outliers, x='GruppoDettaglio', y='Eta', hue='GruppoDettaglio', dodge=False,
    order=order, hue_order=order, palette=palette, jitter=0.08, size=2.6, marker='o', alpha=0.7, linewidth=0, ax=ax
)
ax.set_xlabel('Gruppo')
ax.set_ylabel('Età (anni)')
ax.set_title('Age distribution — box')
leg = ax.get_legend()
if leg is not None:
    leg.remove()
ax.grid(True, axis='y', alpha=0.25)
ax.set_axisbelow(True)
plt.xticks(rotation=10, ha='right')
sns.despine(ax=ax)
plt.tight_layout()

p_box_png = ASSETS / 'age_distribution_box.png'
p_box_svg = ASSETS / 'age_distribution_box.svg'
fig.savefig(p_box_png); fig.savefig(p_box_svg)
plt.show()
plt.close(fig)
print(f"   ✓ Salvato: {p_box_png}")

# 2. AGE DISTRIBUTION - VIOLIN
print("\n2. Age distribution — violin")
fig, ax = plt.subplots(figsize=(7.2, 5.0))
sns.violinplot(
    data=df_age, x='GruppoDettaglio', y='Eta', hue='GruppoDettaglio', dodge=False,
    inner='quartile', cut=0, density_norm='width', palette=palette, order=order, ax=ax
)
ax.set_xlabel('Gruppo')
ax.set_ylabel('Età (anni)')
ax.set_title('Age distribution — violin')
leg = ax.get_legend()
if leg is not None:
    leg.remove()
ax.grid(True, axis='y', alpha=0.25)
ax.set_axisbelow(True)
plt.xticks(rotation=10, ha='right')
sns.despine(ax=ax)
plt.tight_layout()

p_violin_png = ASSETS / 'age_distribution_violin.png'
p_violin_svg = ASSETS / 'age_distribution_violin.svg'
fig.savefig(p_violin_png); fig.savefig(p_violin_svg)
plt.show()
plt.close(fig)
print(f"   ✓ Salvato: {p_violin_png}")

print("\n✓ Grafici età completati!")

In [ ]:
# 3. GENDER DISTRIBUTION
print("\n3. Gender distribution")

# Leggi i conteggi salvati in precedenza
counts_gender = pd.read_csv(OUT / 'counts_genere_per_gruppo.csv', index_col=0)

# Crea stacked bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Colori per genere (neutri)
colors_gender = {'Maschio': '#4575b4', 'Femmina': '#d73027', 'Non risponde': '#cccccc'}

# Prepara dati per plotting
x_pos = np.arange(len(order))
width = 0.6

# Bottom tracker per stacking
bottom = np.zeros(len(order))

for gen in ['Maschio', 'Femmina', 'Non risponde']:
    if gen in counts_gender.columns:
        values = [counts_gender.loc[g, gen] if g in counts_gender.index else 0 for g in order]
        ax.bar(x_pos, values, width, label=gen, bottom=bottom, color=colors_gender.get(gen, '#999'))
        bottom += values

ax.set_xticks(x_pos)
ax.set_xticklabels(order, rotation=15, ha='right')
ax.set_xlabel('Gruppo')
ax.set_ylabel('Conteggio')
ax.set_title('Gender distribution')
ax.legend(loc='upper right', frameon=False)
ax.grid(True, axis='y', alpha=0.25)
ax.set_axisbelow(True)
sns.despine(ax=ax)
plt.tight_layout()

p_gender_png = ASSETS / 'gender_distribution.png'
p_gender_svg = ASSETS / 'gender_distribution.svg'
fig.savefig(p_gender_png); fig.savefig(p_gender_svg)
plt.show()
plt.close(fig)
print(f"   ✓ Salvato: {p_gender_png}")

# 4. DISCIPLINARY AREA DISTRIBUTION  
print("\n4. Disciplinary area distribution (STEM/Humanities)")

# Leggi i conteggi salvati in precedenza
counts_area = pd.read_csv(OUT / 'counts_area_per_gruppo.csv', index_col=0)

# Crea stacked bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Colori per area (coerenti con tema)
colors_area = {'STEM': '#2ca02c', 'Umanistiche': '#ff7f0e', 'Non risponde': '#cccccc'}

# Prepara dati per plotting
bottom = np.zeros(len(order))

for area in ['STEM', 'Umanistiche', 'Non risponde']:
    if area in counts_area.columns:
        values = [counts_area.loc[g, area] if g in counts_area.index else 0 for g in order]
        ax.bar(x_pos, values, width, label=area, bottom=bottom, color=colors_area.get(area, '#999'))
        bottom += values

ax.set_xticks(x_pos)
ax.set_xticklabels(order, rotation=15, ha='right')
ax.set_xlabel('Gruppo')
ax.set_ylabel('Conteggio')
ax.set_title('Disciplinary area distribution (STEM/Humanities)')
ax.legend(loc='upper right', frameon=False)
ax.grid(True, axis='y', alpha=0.25)
ax.set_axisbelow(True)
sns.despine(ax=ax)
plt.tight_layout()

p_area_png = ASSETS / 'area_distribution.png'
p_area_svg = ASSETS / 'area_distribution.svg'
fig.savefig(p_area_png); fig.savefig(p_area_svg)
plt.show()
plt.close(fig)
print(f"   ✓ Salvato: {p_area_png}")

print("\n" + "="*80)
print("✓ TUTTI I 4 GRAFICI DEMOGRAFICI CREATI!")
print("="*80)
print(f"\nFile salvati in: {ASSETS}")
print("  1. age_distribution_box.png/.svg")
print("  2. age_distribution_violin.png/.svg")
print("  3. gender_distribution.png/.svg")
print("  4. area_distribution.png/.svg")

In [ ]:
import asyncio
import edge_tts
from pathlib import Path

# Testo dell'introduzione italiana
introduzione_text = """
L'intelligenza artificiale generativa (GenAI) sta entrando nelle scuole e università italiane spontaneamente, senza una pianificazione pedagogica strutturata. A differenza di precedenti strumenti digitali che recuperano o visualizzano contenuti esistenti (motori di ricerca, database, sistemi di gestione dell'apprendimento), la GenAI ricostruisce attivamente la conoscenza a ogni interazione. Quando ad esempio uno studente interroga ChatGPT sulla Rivoluzione Francese, non riceve un documento o una fonte (in alcuni casi la riceve dopo), ma una narrazione appena sintetizzata—una che fonde molteplici fonti, seleziona certe cornici interpretative rispetto ad altre, e si presenta con una voce autorevole.

Questo processo è genuinamente creativo, nel senso espresso da Bruno Munari. Infatti, nella sua opera "Fantasia", definisce la creatività come ricombinazione di elementi esistenti in nuove configurazioni—un processo di selezione, connessione e sintesi piuttosto che invenzione ex nihilo. La GenAI opera precisamente attraverso tali processi ricombinatori. 

Questa capacità creativa—che nei modelli è più propriamente associata al termine "generativa"—consente a questi strumenti di svolgere molti compiti cognitivi e intellettuali di alto livello che fino a ieri erano prerogativa esclusiva dell'essere umano: la scrittura di saggi, la sintesi di documenti complessi, la traduzione tra lingue, la produzione di codice software, la selezione e gerarchizzazione di notizie, l'elaborazione di strategie argomentative. Tutte attività che richiedevano giudizio umano, esperienza consolidata e capacità di discernimento. Quando uno studente chiede a ChatGPT di riassumere un capitolo di filosofia o di proporre una struttura per un tema, il sistema esegue istantaneamente operazioni di selezione delle informazioni rilevanti, gerarchizzazione dei concetti e costruzione di nessi logici, producendo un testo strutturato e coerente.

Ma è proprio questa capacità a generare preoccupazioni profonde. Alcuni critici avvertono che la GenAI potrebbe essere semplicemente un "pappagallo stocastico"—un pattern-matcher senza genuina comprensione che riproduce schemi statistici dei dati di addestramento. Questi sistemi incorporano bias sistematici che riflettono e amplificano le disuguaglianze presenti nei dati—bias di genere che associano determinate professioni a specifici sessi, bias razziali che perpetuano stereotipi su culture e gruppi etnici, bias socioeconomici che privilegiano prospettive dominanti marginalizzando visioni alternative. Quando uno studente chiede a un'IA di descrivere "un ingegnere" o "un'infermiera", le risposte riflettono sistematicamente questi pregiudizi, naturalizzandoli come verità neutre e poco trasparenti.

Accanto a questo, il problema della trasparenza si lega a quello della accountability. Nei media tradizionali—libri, film, televisione, giornali—il lavoro di cura della conoscenza era svolto da autori, editori, curatori e gatekeeper giornalistici che facevano scelte deliberate e ne portavano la responsabilità. I gatekeeper, in particolare, svolgevano un ruolo cruciale di filtro e validazione: decidevano quali notizie meritassero attenzione, quale rilevanza attribuire agli eventi, quali fonti fossero affidabili. Questo processo, pur non immune da bias, era trasparente nelle sue responsabilità—si poteva identificare chi aveva preso le decisioni editoriali e chiamarlo a rispondere delle sue scelte. Con la GenAI, questo compito fondamentalmente umano di cura della conoscenza è delegato a meccanismi computazionali che operano secondo pattern statistici, senza intento autoriale, accountability o criteri di selezione trasparenti.

Un'altra questione riguarda la dimensione epistemologica, legata al modo in cui conosciamo il mondo. Questi sistemi non si limitano a impacchettare informazioni e darcele in maniera che sembra neutra: modificano attivamente il nostro modo di percepire la realtà, ci fanno vedere le cose diversamente, ci spingono verso ragionamenti che non avremmo sviluppato autonomamente. Quando uno studente chiede a ChatGPT di spiegare un concetto complesso, non riceve solo dati—riceve una particolare interpretazione, una specifica cornice cognitiva, un modo di pensare che diventa invisibilmente parte del suo processo di apprendimento. Il rischio è che la delega sistematica dei processi cognitivi possa atrofizzare le nostre capacità di pensiero critico, di analisi autonoma, di sintesi originale—trasformandoci da pensatori attivi in consumatori passivi di elaborazioni algoritmiche.

Strettamente connesso a questa trasformazione epistemologica è il problema della velocità e dell'efficienza come valori dominanti. Se la delega cognitiva modifica cosa pensiamo, l'accelerazione radicale elimina il tempo necessario per pensare. In una società sempre più competitiva e orientata alla produttività immediata—dove non solo gli studenti ma anche professionisti, ricercatori e lavoratori di ogni settore sono sottoposti a pressioni crescenti per "fare di più in meno tempo"—la GenAI amplifica la spinta verso tempi di risposta sempre più ridotti. Uno studente che un tempo avrebbe impiegato ore o giorni per elaborare una tesina—leggendo fonti, prendendo appunti, riorganizzando le idee, scrivendo bozze successive—può ora ottenere un testo completo in pochi secondi. Un professionista che dedicava settimane ad analizzare dati e preparare report può ora delegare l'intero processo a un sistema automatico. Questa accelerazione radicale elimina quello che gli educatori chiamano "tempo di incubazione": quel periodo apparentemente improduttivo ma cognitivamente cruciale in cui le idee sedimentano, si connettono, maturano. La riflessione profonda richiede lentezza; richiede momenti di sospensione, di dubbio, di ritorno critico sul proprio ragionamento—pause che nel contesto lavorativo contemporaneo vengono sempre più percepite come lussi ingiustificabili. La GenAI, offrendo soluzioni istantanee, rischia di far percepire questa lentezza riflessiva non come una risorsa cognitiva essenziale ma come un'inefficienza da eliminare. Il messaggio implicito è chiaro: perché impiegare tre giorni quando puoi ottenere lo stesso risultato in tre minuti? Ma il risultato è davvero lo stesso? Il processo conta, non solo il prodotto—e un processo che salta la fase della rielaborazione personale produce apprendimenti superficiali, privi di quella sedimentazione che trasforma l'informazione in conoscenza autentica.

Si aggiunge a queste preoccupazioni la prospettiva della sostituzione dell'insegnante. Alcuni visionari tecnologici immaginano che l'IA possa non solo automatizzare compiti intellettuali, ma sostituire completamente la figura del docente. Bill Gates, all'evento ASU+GSV di San Diego, ha prospettato che entro un anno e mezzo chatbot come ChatGPT potranno insegnare a leggere e scrivere ai bambini con l'efficacia di un tutor umano. Questa visione trova eco anche nella fiction, come nella serie Star Wars: Skeleton Crew, dove bambini interagiscono con droidi educatori in contesti di apprendimento completamente automatizzati. Al di là delle questioni tecniche—se questi sistemi possano svolgere adeguatamente compiti che una volta erano di competenza umana—emerge una domanda pedagogica fondamentale: quando una macchina si sostituisce alla relazione educativa, che effetti ha questo processo sulla formazione intellettuale e umana?

Questo crea una situazione paradossale. La GenAI democratizza l'accesso a capacità digitali avanzate—studenti ed educatori possono ora creare siti web dinamici, dashboard interattive, simulazioni di valutazione ed esperienze di apprendimento personalizzate (come interazioni basate su avatar con figure storiche o letterarie) che prima richiedevano competenze tecniche specializzate. Tuttavia questa rapida diffusione avviene in un contesto dove la comprensione critica rimane limitata: gli utenti interagiscono intensivamente con sistemi i cui processi decisionali e limiti rimangono in gran parte oscuri.

Vista la complessità del fenomeno—che Morin definirebbe irriducibile a semplificazioni—è comprensibile che le posizioni oscillino tra "integrati" e "apocalittici", spesso nella stessa persona: chi vede nell'IA una rivoluzione pedagogica democratizzante può al contempo temere l'erosione del pensiero critico; chi riconosce i rischi per il rapporto educativo umano può contemporaneamente apprezzarne le potenzialità. È in questo clima di incertezza e ambivalenza—dove entusiasmo e preoccupazione coesistono senza sintesi definitiva—che studenti, docenti e istituzioni navigano quotidianamente l'integrazione della GenAI.
"""

# Configurazione output
output_dir = Path("../output/audio")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "introduzione_italiana.mp3"

# Voce italiana (scegliamo una voce naturale)
VOICE = "it-IT-ElsaNeural"  # Voce femminile italiana di alta qualità

print(f"Generazione audio dell'introduzione...")
print(f"Voce: {VOICE}")
print(f"Output: {output_file}")
print(f"Lunghezza testo: {len(introduzione_text)} caratteri")
print(f"Parole: circa {len(introduzione_text.split())} parole")
print(f"Durata stimata: circa {len(introduzione_text.split()) / 150:.1f} minuti (a 150 parole/minuto)")

# Funzione asincrona per generare l'audio
async def generate_audio():
    communicate = edge_tts.Communicate(introduzione_text, VOICE)
    await communicate.save(str(output_file))

# Esegui la generazione
await generate_audio()

print(f"\n✓ Audio generato con successo!")
print(f"File salvato in: {output_file}")
print(f"Dimensione file: {output_file.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Debug: ispeziona colonne DF per identificare età e gruppo
cols = list(DF.columns)
print('Numero colonne DF:', len(cols))
for i, c in enumerate(cols[:80], 1):
    print(f"{i:3d}. {c}")


In [ ]:
# Debug: trova possibili colonne per età e gruppo
import re, unicodedata

def norm(s):
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c)).lower()

age_candidates = []
grp_candidates = []
for c in DF.columns:
    n = norm(str(c))
    if any(k in n for k in ['eta', 'età', 'age']):
        age_candidates.append(c)
    if any(k in n for k in ['gruppo', 'group', 'dettaglio', 'detail']):
        grp_candidates.append(c)
print('Age candidates:', age_candidates)
print('Group candidates:', grp_candidates)


In [ ]:
# Debug: individua colonne numeriche candidate per età per percentuale di valori numerici
import pandas as pd
candidates = []
for c in DF.columns:
    s = pd.to_numeric(DF[c], errors='coerce')
    valid = s.between(10, 100).sum()
    total = s.notna().sum()
    if valid >= 10:  # almeno 10 valori plausibili
        candidates.append((c, int(valid), int(total)))

candidates = sorted(candidates, key=lambda x: (-x[1], -x[2]))
print('Colonne plausibili per età (validi tra 10 e 100):')
for c, v, t in candidates[:10]:
    print(f" - {c} -> validi:{v} su nonNa:{t}")
    sample = pd.to_numeric(DF[c], errors='coerce').dropna().astype(int).drop_duplicates().sort_values().tolist()[:10]
    print('   esempi:', sample)
